## What “clusters” (entities) mean

A **cluster / entity** is a **connected component** in a graph:

- **Nodes = addresses**
- **Edges = “same controller” links** inferred from transaction patterns

A **cluster size** is simply: **how many addresses became connected** through these inferred links.

---

## Algorithm overview (step-by-step)

### 0) Define two time windows

Two overlapping windows are used:

- **Index window** `[INDEX_START, ANALYSIS_END)`  
  Build an outpoint index so inputs can be resolved to addresses.
- **Analysis window** `[ANALYSIS_START, ANALYSIS_END)`  
  Apply heuristics and add unions (one year of clustering).

Why this is necessary: **Bitcoin inputs do not contain an address**.  
An input only references a previous output (`prev_txid`, `prev_vout`). The address lives in that previous output.

---

### 1) Build an outpoint index (outputs → address)

For each day/file in the **index window**, take rows with `dir == "out"` and store:

- key: `(txid, n)` where `n` is output index (vout)
- value: `address`

This creates a local map: **(prev_txid, prev_vout) → address**, which allows later recovery of input addresses.

---

### 2) Resolve input addresses (prevouts → address)

For each day/file in the **analysis window**, for every row with `dir == "in"`:

- `spend_txid` = spending transaction id
- `(prev_txid, prev_vout)` = the referenced output being spent

Then:

- count total inputs per spending tx: `input_utxo_count[spend_txid] += 1`
- look up `(prev_txid, prev_vout)` in the outpoint index to obtain the **input address**
- group resolved input addresses per spending tx:  
  `inputs_by_txid[spend_txid] = {addr1, addr2, ...}`

Important:
- `input_utxo_count` counts **all inputs** (even unresolved)
- `inputs_by_txid` contains only **resolved input addresses**

---

### 3) Group outputs per transaction

Still within the analysis window, group `dir == "out"` rows by `txid` and collect:

- `out_addrs: [address, ...]`
- `out_values: [value, ...]`

Now each transaction can be analyzed using:
- input UTXO count,
- resolved input addresses,
- output addresses + output amounts.

---

### 4) Per transaction: apply filters + heuristics → add “same owner” edges

For each transaction `txid` that has outputs and at least one input:

#### 4.1 Skip transactions with insufficient structure
- no outputs → skip
- `input_utxo_count[txid] == 0` → skip

#### 4.2 Skip collaborative patterns (CoinJoin-like)
A simple CoinJoin-ish filter flags a transaction if:
- `n_in_utxos >= 3`
- `n_out >= 3`
- at least 3 outputs share exactly the same amount

If flagged:
- **do not add any unions**
- only mark outputs as “seen” (for later “new output” checks)

Goal: avoid linking unrelated participants.

#### 4.3 Multi-input heuristic (high-signal link)
If `n_in_utxos >= 2`:
- take the **resolved unique input addresses** for this tx
- if there are ≥2 distinct resolved input addresses → connect (union) them all
- if only 1 resolved input address → mark it as participating, but no additional links can be created

Interpretation: spending multiple inputs in one transaction usually requires control over all corresponding private keys (or custody over them), so they are treated as belonging to the same entity.

#### 4.4 Change heuristic (riskier link)
Attempt to identify exactly one output that is likely “change” back to the spender:

Candidate output must satisfy all:
- output address is **not** one of the resolved input addresses (not a self-send)
- output address type matches the **majority input type** (prefix-based: `1...`, `3...`, `bc1...`)
- output address is **new as an output** (has not appeared before as an output)

If *exactly one* candidate remains:
- label it as change
- connect (union) it to an input address (e.g., the first resolved input)

Finally:
- mark all outputs as “seen” to support future “new output” tests.

---

### 5) After scanning all files: compress unions → entity IDs → cluster sizes

After processing the entire analysis window:

1) compute each address’s Union-Find root
2) assign a contiguous `entity_id` per root
3) compute cluster sizes (e.g., via `bincount(entity_id)`)
4) report:
   - largest cluster size and fraction
   - top-K cluster sizes
   - quantiles (median/90th/99th)
   - heuristic coverage (how many nodes were linked by which heuristic)
5) optionally write mapping `(address → entity_id)` to Parquet (ideally in chunks)

---

## How to interpret “less permissive union” results

The typical outcome is **heavy-tailed**:

- many clusters of size 1–10
- fewer clusters of size 10–1000
- very few clusters of size 10k+

Sometimes there is a **giant component** (largest cluster far larger than the rest).  
This can be:
- **plausible**: large custodial/service wallets consolidating many deposits
- **a red flag**: false change links “bridge” unrelated clusters, causing cascade merges

Tightening change inference typically causes:
- fewer unions,
- more fragmentation (more clusters),
- reduced dominance of the largest cluster (if change links were the main bridge source).

If the largest cluster stays huge even with strict change rules, it is often driven by multi-input consolidation patterns of large services, or by collaborative transactions not caught by the filter.

In [ ]:
# =============================================================================
# Reporting helpers (all prints live in functions below)
# =============================================================================

from __future__ import annotations

from collections import Counter
import math
import sqlite3
import numpy as np
import polars as pl

OUTPUT_DIR_VALUES = {"out", "vout", "output", "o"}
INPUT_DIR_VALUES  = {"in", "vin", "input", "i"}


# -----------------------------
# Sanity checks + prevout join sanity
# -----------------------------

def run_sanity_checks(
    n_nodes: int,
    node_to_entity: list[int] | np.ndarray | None = None,
    cluster_size_counter: Counter | None = None,
    prevout_lookups: int | None = None,
    prevout_hits: int | None = None,
    top_k: int = 20,
) -> None:
    """
    Prints:
      - largest cluster fraction of nodes
      - top cluster sizes
      - entity count stats
      - optional prevout hit-rate
    Provide either node_to_entity OR cluster_size_counter.
    """
    if cluster_size_counter is None:
        if node_to_entity is None:
            raise ValueError("Provide either node_to_entity or cluster_size_counter.")
        cluster_size_counter = Counter(list(node_to_entity))

    sizes = list(cluster_size_counter.values())
    if not sizes:
        print("[SANITY] No clusters found (sizes empty).", flush=True)
        return

    sizes_sorted = sorted(sizes, reverse=True)
    total_nodes_from_sizes = sum(sizes_sorted)
    largest = sizes_sorted[0]
    frac = largest / total_nodes_from_sizes if total_nodes_from_sizes else float("nan")

    print("\n[SANITY] Cluster summary", flush=True)
    print(f"  UF nodes (n_nodes): {n_nodes:,}", flush=True)
    print(f"  Total nodes from cluster sizes: {total_nodes_from_sizes:,}", flush=True)
    if total_nodes_from_sizes != n_nodes:
        print("  [WARN] sum(cluster_sizes) != n_nodes  -> mismatch suggests a bug in mapping logic.", flush=True)
    print(f"  Entities (clusters): {len(cluster_size_counter):,}", flush=True)
    print(f"  Largest cluster size: {largest:,}", flush=True)
    print(f"  Largest cluster fraction of nodes: {frac:.2%}", flush=True)

    print(f"\n[SANITY] Top {top_k} cluster sizes:", flush=True)
    print(" ", sizes_sorted[:top_k], flush=True)

    def pct(p: float) -> int:
        s_asc = sorted(sizes_sorted)
        idx = max(0, min(len(s_asc) - 1, math.ceil(p * len(s_asc)) - 1))
        return int(s_asc[idx])

    med = int(sorted(sizes_sorted)[len(sizes_sorted) // 2])
    print("\n[SANITY] Quick distribution stats", flush=True)
    print(f"  Median cluster size: {med:,}", flush=True)
    print(f"  90th percentile cluster size: {pct(0.90):,}", flush=True)
    print(f"  99th percentile cluster size: {pct(0.99):,}", flush=True)

    if prevout_lookups is not None and prevout_hits is not None:
        rate = (prevout_hits / prevout_lookups) if prevout_lookups else float("nan")
        print("\n[SANITY] Prevout lookup hit-rate (DB)", flush=True)
        print(f"  Lookups:  {prevout_lookups:,}", flush=True)
        print(f"  Hits:     {prevout_hits:,}", flush=True)
        print(f"  Hit-rate: {rate:.2%}", flush=True)


def prevout_join_sanity_polars(one_io_parquet: str) -> None:
    """
    Pure-Polars sanity on a single parquet:
      - prev_vout integer-likeness
      - duplicate outpoints (txid,n)
      - left join vin->vout inside same parquet
      - match explosion check
    Note: This join only resolves prevouts that point to outputs present in this same file.
    Your real pipeline resolves against the outpoint DB across days.
    """
    df = pl.read_parquet(one_io_parquet, columns=["dir","txid","n","prev_txid","prev_vout","address","value"])
    df = df.with_columns(pl.col("dir").cast(pl.Utf8, strict=False).str.to_lowercase().alias("dir"))

    vout = (
        df.filter(pl.col("dir").is_in(list(OUTPUT_DIR_VALUES)))
          .select([
              pl.col("txid").alias("out_txid"),
              pl.col("n").cast(pl.Int64, strict=False).alias("out_n"),
              pl.col("address").alias("out_address"),
              pl.col("value").alias("out_value"),
          ])
          .filter(pl.col("out_txid").is_not_null() & pl.col("out_n").is_not_null())
    )

    vin = (
        df.filter(pl.col("dir").is_in(list(INPUT_DIR_VALUES)))
          .select([
              pl.col("txid").alias("spend_txid"),
              pl.col("prev_txid").alias("prev_txid"),
              pl.col("prev_vout").alias("prev_vout"),
          ])
          .filter(pl.col("spend_txid").is_not_null() & pl.col("prev_txid").is_not_null())
    )

    print("\n[PREVOUT-SANITY | POLARS]", flush=True)
    print("File:", one_io_parquet, flush=True)
    print("Rows:", {"vin": vin.height, "vout": vout.height}, flush=True)

    vin_nonnull = vin.filter(pl.col("prev_vout").is_not_null())
    if vin_nonnull.height > 0:
        frac_integerlike = (
            vin_nonnull
            .with_columns(((pl.col("prev_vout") - pl.col("prev_vout").floor()).abs() < 1e-9).alias("is_intlike"))
            .select(pl.col("is_intlike").mean())
            .item()
        )
        print(f"prev_vout integer-like fraction: {float(frac_integerlike):.6f}", flush=True)

    vin2 = vin.with_columns(
        pl.when(pl.col("prev_vout").is_not_null())
          .then(pl.col("prev_vout").cast(pl.Int64, strict=False))
          .otherwise(None)
          .alias("prev_vout_i64")
    )

    dupe_outpoints = (
        vout.group_by(["out_txid","out_n"])
            .len()
            .filter(pl.col("len") > 1)
            .height
    )
    print("Duplicate outpoints (should be 0):", dupe_outpoints, flush=True)

    joined = (
        vin2.join(
            vout,
            left_on=["prev_txid","prev_vout_i64"],
            right_on=["out_txid","out_n"],
            how="left",
        )
    )

    resolved = joined.filter(pl.col("out_address").is_not_null()).height
    lookups  = vin.height
    print(f"Prevout resolution hit-rate (within-file): {resolved}/{lookups} = {resolved/lookups:.2%}" if lookups else "No vin rows", flush=True)

    if joined.height > 0:
        max_matches_per_input = (
            joined.group_by(["spend_txid","prev_txid","prev_vout_i64"])
                  .len()
                  .select(pl.col("len").max())
                  .item()
        )
        print("Max matches per (spend_txid, prev_txid, prev_vout):", int(max_matches_per_input) if max_matches_per_input is not None else None, flush=True)

    sample_unresolved = (
        joined.filter(pl.col("out_address").is_null())
              .select(["spend_txid","prev_txid","prev_vout_i64"])
              .head(5)
    )
    print("\nSample unresolved inputs (within-file):", flush=True)
    print(sample_unresolved, flush=True)


def prevout_join_sanity_db(
    conn: sqlite3.Connection,
    one_io_parquet: str,
    *,
    sample_limit: int,
    hybrid_threshold: int,
    join_chunk: int,
) -> None:
    """
    Sanity-check using your ACTUAL outpoint DB resolution:
      - reads vin from a single parquet
      - casts prev_vout -> int64
      - looks up in DB using lookup_outpoints_hybrid(conn, keys, hybrid_threshold, join_chunk)
      - prints DB hit-rate and sample unresolved keys
    Uses only a sample of inputs (sample_limit) to keep it cheap.
    """
    df = pl.read_parquet(one_io_parquet, columns=["dir","txid","prev_txid","prev_vout"])
    df = df.with_columns(pl.col("dir").cast(pl.Utf8, strict=False).str.to_lowercase().alias("dir"))

    vin = (
        df.filter(pl.col("dir").is_in(list(INPUT_DIR_VALUES)))
          .select([
              pl.col("txid").alias("spend_txid"),
              pl.col("prev_txid").alias("prev_txid"),
              pl.col("prev_vout").alias("prev_vout"),
          ])
          .filter(pl.col("prev_txid").is_not_null() & pl.col("prev_vout").is_not_null())
          .head(sample_limit)
    )

    print("\n[PREVOUT-SANITY | DB]", flush=True)
    print("File:", one_io_parquet, flush=True)
    print("Vin sample rows:", vin.height, flush=True)

    if vin.height == 0:
        print("No vin rows in sample.", flush=True)
        return

    frac_integerlike = (
        vin.with_columns(((pl.col("prev_vout") - pl.col("prev_vout").floor()).abs() < 1e-9).alias("is_intlike"))
           .select(pl.col("is_intlike").mean())
           .item()
    )
    print(f"prev_vout integer-like fraction (sample): {float(frac_integerlike):.6f}", flush=True)

    vin2 = vin.with_columns(
        pl.col("prev_vout").cast(pl.Int64, strict=False).alias("prev_vout_i64")
    ).filter(pl.col("prev_vout_i64").is_not_null())

    keys = [
        (prev_txid, int(prev_n))
        for prev_txid, prev_n in vin2.select(["prev_txid", "prev_vout_i64"]).iter_rows(named=False, buffer_size=200_000)
    ]
    if not keys:
        print("No usable keys after casting prev_vout.", flush=True)
        return

    # NOTE: pass both knobs (your current version was missing them)
    hits = lookup_outpoints_hybrid(conn, keys, hybrid_threshold=hybrid_threshold, join_chunk=join_chunk)
    resolved = len(hits)
    lookups = len(keys)

    print(f"DB prevout hit-rate (sample): {resolved}/{lookups} = {resolved/lookups:.2%}", flush=True)

    unresolved = []
    for k in keys[:5000]:
        if k not in hits:
            unresolved.append(k)
            if len(unresolved) >= 10:
                break
    if unresolved:
        print("Sample unresolved (prev_txid, prev_n):", flush=True)
        for k in unresolved:
            print(" ", k, flush=True)
    else:
        print("No unresolved keys in first 5000 keys (good sign).", flush=True)


# -----------------------------
# Evidence composition helpers
# -----------------------------

def compute_top_cluster_evidence_composition(
    *,
    node_to_entity: np.ndarray,
    multi_change_flags: bytearray,
    top_entity_id: int,
) -> dict[str, float | int]:
    """
    Evidence bits (same semantics as address_confidence.parquet evidence_bits):
      bit 1 -> multi-input evidence
      bit 2 -> change-output evidence
      bit 4 -> change-anchor evidence

    Returns counts + shares for the top cluster, including bit2-only share.
    """
    n_nodes = int(node_to_entity.size)
    if n_nodes == 0:
        return {
            "top_entity_id": int(top_entity_id),
            "n_top": 0,
            "n_bit1": 0, "n_bit2": 0, "n_bit4": 0,
            "n_bit2_only": 0,
            "share_bit1": float("nan"),
            "share_bit2": float("nan"),
            "share_bit4": float("nan"),
            "share_bit2_only": float("nan"),
        }

    if len(multi_change_flags) < n_nodes:
        raise ValueError(f"multi_change_flags length {len(multi_change_flags)} < n_nodes {n_nodes}")

    bits = np.frombuffer(multi_change_flags, dtype=np.uint8, count=n_nodes)
    mask = (node_to_entity == int(top_entity_id))
    n_top = int(mask.sum())
    if n_top == 0:
        return {
            "top_entity_id": int(top_entity_id),
            "n_top": 0,
            "n_bit1": 0, "n_bit2": 0, "n_bit4": 0,
            "n_bit2_only": 0,
            "share_bit1": float("nan"),
            "share_bit2": float("nan"),
            "share_bit4": float("nan"),
            "share_bit2_only": float("nan"),
        }

    top_bits = bits[mask]
    n_bit1 = int(np.count_nonzero(top_bits & 1))
    n_bit2 = int(np.count_nonzero(top_bits & 2))
    n_bit4 = int(np.count_nonzero(top_bits & 4))
    n_bit2_only = int(np.count_nonzero(top_bits == 2))

    share_bit1 = n_bit1 / n_top
    share_bit2 = n_bit2 / n_top
    share_bit4 = n_bit4 / n_top
    share_bit2_only = n_bit2_only / n_top

    return {
        "top_entity_id": int(top_entity_id),
        "n_top": n_top,
        "n_bit1": n_bit1,
        "n_bit2": n_bit2,
        "n_bit4": n_bit4,
        "n_bit2_only": n_bit2_only,
        "share_bit1": float(share_bit1),
        "share_bit2": float(share_bit2),
        "share_bit4": float(share_bit4),
        "share_bit2_only": float(share_bit2_only),
    }


# -----------------------------
# Final report printer (calls sanity if requested)
# -----------------------------

def print_final_report(
    *,
    n_nodes: int,
    n_entities: int,
    node_to_entity: np.ndarray,
    counts: np.ndarray,
    cluster_sizes: np.ndarray,
    stats: dict[str, int],
    multi_change_flags: bytearray,
    outputs: OutputConfig,
    guards: MergeGuardParams,
    enable_constraints_diag: bool,
    constraint_unique_pairs: set[tuple[int, int]],
    constraint_gate_evals_total: int,
    constraint_pairs_repeated: int,
    change_merge_votes: dict[tuple[int, int], int],
    prevout_lookups: int,
    prevout_hits: int,

    # sanity hooks
    run_sanity: bool,
    sanity_top_k: int,
    run_prevout_db_sanity: bool,
    conn: sqlite3.Connection | None,
    sanity_parquet: str | None,
    prevout_sanity_sample_limit: int,
    prevout_hybrid_threshold: int,
    prevout_lookup_chunk: int,
) -> int:
    """
    All final prints centralized here.
    Optionally prints sanity checks and DB prevout sanity.
    Returns top_entity_id (largest cluster).
    """
    print("\nFinalizing entity mapping (compressing components)...", flush=True)
    print(f"Number of unique addresses with UF nodes: {n_nodes:,}", flush=True)

    hit_rate = prevout_hits / max(1, prevout_lookups)
    print(f"[INFO] prevout lookups: {prevout_lookups:,}  hits: {prevout_hits:,}  hit-rate: {hit_rate:.3%}", flush=True)
    print(f"Number of entities (clusters): {n_entities:,}", flush=True)

    if n_entities == 0:
        print("[WARN] No entities computed.", flush=True)
        return -1

    # Top clusters
    top_k = min(outputs.top_k_clusters_print, n_entities)
    top_ids = np.argpartition(counts, -top_k)[-top_k:]
    top_ids_sorted = top_ids[np.argsort(counts[top_ids])[::-1]]
    top_entity_id = int(top_ids_sorted[0])

    print("\n[TOP CLUSTERS]", flush=True)
    for rank, eid in enumerate(top_ids_sorted, start=1):
        sz = int(counts[eid])
        frac = (sz / n_nodes) if n_nodes else float("nan")
        print(f"  #{rank:02d}  entity_id={int(eid):>8d}  size={sz:>10,d}  frac={frac:>7.2%}", flush=True)

    # Cluster stats
    largest = int(cluster_sizes.max()) if cluster_sizes.size else 0
    largest_frac = (largest / n_nodes) if n_nodes else float("nan")
    q50, q90, q99 = np.quantile(cluster_sizes, [0.5, 0.9, 0.99]) if cluster_sizes.size else (0, 0, 0)

    print("\n[CLUSTER STATS]", flush=True)
    print(f"  Entities: {n_entities:,}", flush=True)
    print(f"  Largest cluster size: {largest:,}", flush=True)
    print(f"  Largest cluster fraction of nodes: {largest_frac:.2%}", flush=True)
    print(f"  Median cluster size: {int(q50)}", flush=True)
    print(f"  90th percentile: {int(q90)}", flush=True)
    print(f"  99th percentile: {int(q99)}", flush=True)

    # Evidence composition inside top cluster
    comp = compute_top_cluster_evidence_composition(
        node_to_entity=node_to_entity,
        multi_change_flags=multi_change_flags,
        top_entity_id=top_entity_id,
    )

    print("\n[TOP CLUSTER EVIDENCE COMPOSITION]", flush=True)
    print(f"  Top entity_id: {comp['top_entity_id']:,}", flush=True)
    print(f"  Addresses in top cluster: {comp['n_top']:,}", flush=True)
    print(f"  bit1 (multi-input)   : {comp['n_bit1']:,}  share={comp['share_bit1']:.2%}", flush=True)
    print(f"  bit2 (change-output) : {comp['n_bit2']:,}  share={comp['share_bit2']:.2%}", flush=True)
    print(f"  bit4 (change-anchor) : {comp['n_bit4']:,}  share={comp['share_bit4']:.2%}", flush=True)
    print(f"  bit2-only            : {comp['n_bit2_only']:,}  share={comp['share_bit2_only']:.2%}", flush=True)
    print("  (collapse signature often: very high bit2-only share + low bit1 share)", flush=True)

    # Heuristic coverage (node-level)
    flags_view = multi_change_flags[:n_nodes]
    n_addrs_multi = sum(1 for v in flags_view if (v & 1))
    n_addrs_change_out = sum(1 for v in flags_view if (v & 2))
    n_addrs_change_anchor = sum(1 for v in flags_view if (v & 4))
    n_addrs_touched = sum(1 for v in flags_view if (v & 7))

    print("\n[HEURISTIC COVERAGE (node-level)]", flush=True)
    print(f"  Total txs processed (>=1 input UTXO): {stats['n_txs_total']:,}", flush=True)
    print(f"  Mixing-like skipped: {stats['n_txs_coinjoin_flagged']:,}", flush=True)
    print(f"  Multi-input txs (SAFE policy applied): {stats['n_txs_with_multiinput']:,}", flush=True)
    print(f"  Change detected (tight): {stats['n_txs_with_change_detected']:,}", flush=True)
    print(f"  Nodes marked multi-input: {n_addrs_multi:,}", flush=True)
    print(f"  Nodes marked change-output: {n_addrs_change_out:,}", flush=True)
    print(f"  Nodes marked change-anchor: {n_addrs_change_anchor:,}", flush=True)
    print(f"  Nodes touched by any heuristic: {n_addrs_touched:,}", flush=True)

    # Union diagnostics
    print("\n[UNION DIAGNOSTICS]", flush=True)
    print(f"  Union attempts: {stats['union_attempts']:,}", flush=True)
    print(f"  Unions applied: {stats['unions_applied']:,}", flush=True)
    print(f"    - applied via H1:     {stats['unions_applied_h1']:,}", flush=True)
    print(f"    - applied via CHANGE: {stats['unions_applied_change']:,}", flush=True)

    print(f"  skip_same_component (already merged): {stats['skip_same_component']:,}", flush=True)
    print(f"    - same_component via H1:     {stats['skip_same_component_h1']:,}", flush=True)
    print(f"    - same_component via CHANGE: {stats['skip_same_component_change']:,}", flush=True)

    print(f"  Unions skipped by guards/votes/constraints: {stats['unions_skipped']:,}", flush=True)
    print(f"    - skipped (H1):     {stats['unions_skipped_h1']:,}", flush=True)
    print(f"    - skipped (CHANGE): {stats['unions_skipped_change']:,}", flush=True)

    print("  Skip breakdown:", flush=True)
    print(f"    - skip_abs_cap: {stats['skip_abs_cap']:,}  (H1={stats['skip_abs_cap_h1']:,}, CHANGE={stats['skip_abs_cap_change']:,})", flush=True)
    print(f"    - skip_ratio_guard: {stats['skip_ratio_guard']:,}  (CHANGE-only)", flush=True)
    print(f"    - skip_constraint: {stats['skip_constraint']:,}  (CHANGE-only; vote pending)", flush=True)
    print(f"    - skip_degree_guard: {stats['skip_degree_guard']:,}  (CHANGE-only)", flush=True)
    print(f"    - skip_vote_pending: {stats['skip_vote_pending']:,}", flush=True)
    print(f"    - skip_vote_failed:  {stats['skip_vote_failed']:,}", flush=True)

    # Constraint diagnostics (optional)
    if enable_constraints_diag:
        print("\n[CONSTRAINT DIAGNOSTICS]", flush=True)
        print(f"  Vote-gating pairs seen (unique): {len(constraint_unique_pairs):,}", flush=True)
        print(f"  Vote-gating evaluations (total): {constraint_gate_evals_total:,}", flush=True)
        print(f"  Vote-gating pairs that repeated: {constraint_pairs_repeated:,}", flush=True)
        active_pairs = sum(1 for _k, v in change_merge_votes.items() if v > 0)
        print(f"  Vote pairs tracked (total): {active_pairs:,}", flush=True)

    # sanity checks invoked here
    if run_sanity:
        run_sanity_checks(
            n_nodes=n_nodes,
            node_to_entity=node_to_entity,
            prevout_lookups=prevout_lookups,
            prevout_hits=prevout_hits,
            top_k=sanity_top_k,
        )

    if run_prevout_db_sanity:
        if conn is None or sanity_parquet is None:
            print("[SANITY] prevout DB sanity requested but conn or sanity_parquet is None; skipping.", flush=True)
        else:
            prevout_join_sanity_db(
                conn,
                sanity_parquet,
                sample_limit=prevout_sanity_sample_limit,
                hybrid_threshold=prevout_hybrid_threshold,
                join_chunk=prevout_lookup_chunk,
            )

    return top_entity_id

### PathsConfig

- `parquet_io_glob = "/media/vatereal/Main/parquet/io/day=*/io-*.parquet"`
  1) **Name + value set:** `parquet_io_glob = "/media/vatereal/Main/parquet/io/day=*/io-*.parquet"`
  2) **Reflects:** Input discovery pattern (which parquet files exist in-scope).
  3) **Update higher/lower:** Not numeric; change when dataset layout/naming changes (different root, partitions, filenames).

- `outputs_dir = "/media/vatereal/Main/outputs"`
  1) **Name + value set:** `outputs_dir = "/media/vatereal/Main/outputs"`
  2) **Reflects:** Where all artifacts are written (SQLite DB, parquet mapping, logs).
  3) **Update higher/lower:** Not numeric; move to faster storage (NVMe/SSD) for major speedups in SQLite + Parquet writes.

- `outpoint_db_path = "/media/vatereal/Main/outputs/outpoints_2014.sqlite"`
  1) **Name + value set:** `outpoint_db_path = outputs_dir / "outpoints_2014.sqlite"`
  2) **Reflects:** SQLite cache of outpoints used for prevout resolution (drives hit-rate and speed).
  3) **Update higher/lower:** Not numeric; keep year/window-specific; place on fast disk; avoid reusing across different settings.

- `entity_map_out_path = "/media/vatereal/Main/outputs/entities_multiinput_change_2014.parquet"`
  1) **Name + value set:** `entity_map_out_path = outputs_dir / "entities_multiinput_change_2014.parquet"`
  2) **Reflects:** Final artifact: `address -> entity_id` mapping output.
  3) **Update higher/lower:** Not numeric; version outputs when heuristics/guards change to preserve comparability.

---

### AnalysisConfig

- `analysis_start = date(2014, 1, 1)`
  1) **Name + value set:** `analysis_start = 2014-01-01`
  2) **Reflects:** Inclusive start of clustering window.
  3) **Update higher/lower:** Not numeric; move earlier/later to target a different time period.

- `analysis_end_exclusive = date(2015, 1, 1)`
  1) **Name + value set:** `analysis_end_exclusive = 2015-01-01 (exclusive)`
  2) **Reflects:** Exclusive end of clustering window.
  3) **Update higher/lower:** Not numeric; set to `year+1` for 1-year runs; extend for multi-year runs.

- `outpoint_db_lookback_days = 365`
  1) **Name + value set:** `outpoint_db_lookback_days = 365`
  2) **Reflects:** How far back to preload outputs so analysis inputs can resolve prevouts.
  3) **Update higher/lower:**
     - **Higher:** improves prevout hit-rate / change eligibility when UTXOs are old.
     - **Lower:** faster preload but more unresolved inputs (more skipped txs).

- `value_unit_mode = "infer"`
  1) **Name + value set:** `value_unit_mode = "infer"`
  2) **Reflects:** Whether `value` is interpreted as BTC vs sats (infer by magnitude).
  3) **Update higher/lower:**
     - Use `"btc"` / `"sats"` if schema is known (safer).
     - Keep `"infer"` only if dataset is consistent and you trust inference.

---

### OutputConfig

- `reset_outpoint_db = True`
  1) **Name + value set:** `reset_outpoint_db = True`
  2) **Reflects:** Whether DB is rebuilt from scratch (reproducibility).
  3) **Update higher/lower:**
     - **False:** only when reusing a DB built from the exact same input/window/lookback/settings.
     - **True:** when iterating heuristics or debugging.

- `write_entity_mapping = True`
  1) **Name + value set:** `write_entity_mapping = True`
  2) **Reflects:** Whether to write `address -> entity_id` parquet.
  3) **Update higher/lower:**
     - **False:** faster diagnostics runs.
     - **True:** final outputs.

- `entity_map_compression = "zstd"`
  1) **Name + value set:** `entity_map_compression = "zstd"`
  2) **Reflects:** Output size vs write speed tradeoff.
  3) **Update higher/lower:**
     - `"snappy"` for faster writes / larger file.
     - `"zstd"` for smaller files / usually fine speed.

- `entity_write_batch = 1_000_000`
  1) **Name + value set:** `entity_write_batch = 1,000,000`
  2) **Reflects:** Rows buffered per parquet write (speed vs memory).
  3) **Update higher/lower:**
     - **Higher:** fewer write calls (faster) if RAM allows.
     - **Lower:** reduces memory spikes.

- `top_k_clusters_print = 20`
  1) **Name + value set:** `top_k_clusters_print = 20`
  2) **Reflects:** Log verbosity for top clusters.
  3) **Update higher/lower:** Higher for debugging; lower for cleaner logs.

---

### PlotConfig

- `enable_plots = True`
  1) **Name + value set:** `enable_plots = True`
  2) **Reflects:** Whether to run matplotlib diagnostics (no clustering effect).
  3) **Update higher/lower:** False for headless/throughput runs; True when iterating heuristics.

- `zipf_top_k = 200_000`
  1) **Name + value set:** `zipf_top_k = 200,000`
  2) **Reflects:** How many top clusters appear in the rank-size plot.
  3) **Update higher/lower:** Higher for more tail detail; lower if plotting is slow.

- `focus_max_size = 2048`
  1) **Name + value set:** `focus_max_size = 2048`
  2) **Reflects:** Upper cutoff for the “zoomed” histogram region.
  3) **Update higher/lower:** Lower to zoom in on small wallets; higher to inspect mid-sized clusters.

- `log_bins = 40`
  1) **Name + value set:** `log_bins = 40`
  2) **Reflects:** Histogram resolution in log-space.
  3) **Update higher/lower:** Higher for finer detail (can get noisy); lower for smoother plots.

---

### PerfConfig

- `outpoint_commit_every_rows = 500_000`
  1) **Name + value set:** `outpoint_commit_every_rows = 500,000`
  2) **Reflects:** SQLite commit granularity (throughput vs stall/WAL behavior).
  3) **Update higher/lower:**
     - **Higher:** fewer commits (often faster) if stable.
     - **Lower:** smaller transactions if WAL growth/stalls appear.

- `prevout_hybrid_threshold = 5_000`
  1) **Name + value set:** `prevout_hybrid_threshold = 5,000`
  2) **Reflects:** Legacy lookup mode switch (mostly relevant to sanity helper).
  3) **Update higher/lower:** Usually leave as-is under Patch 3; only matters if legacy path is used.

- `prevout_lookup_chunk = 50_000`
  1) **Name + value set:** `prevout_lookup_chunk = 50,000`
  2) **Reflects:** Chunk size for legacy join buffer (sanity helper).
  3) **Update higher/lower:** Increase if using legacy path and RAM allows; decrease if SQLite temp memory pressure.

- `iter_buffer_in = 200_000`
  1) **Name + value set:** `iter_buffer_in = 200,000`
  2) **Reflects:** Polars→Python iteration chunk size for VIN extraction.
  3) **Update higher/lower:** Higher for speed if RAM allows; lower to reduce memory spikes.

- `iter_buffer_out = 200_000`
  1) **Name + value set:** `iter_buffer_out = 200,000`
  2) **Reflects:** Polars→Python iteration chunk size for VOUT/outpoint indexing.
  3) **Update higher/lower:** Same as `iter_buffer_in`.

- `iter_buffer_grouped = 50_000`
  1) **Name + value set:** `iter_buffer_grouped = 50,000`
  2) **Reflects:** Iteration chunk size for per-tx joined rows.
  3) **Update higher/lower:** Higher for speed if stable; lower if joined rows are heavy.

- `gc_every_n_files = 100`
  1) **Name + value set:** `gc_every_n_files = 100`
  2) **Reflects:** How often to trigger GC to control RSS.
  3) **Update higher/lower:** Lower if memory creeps upward; higher if GC pauses are noticeable.

- `tqdm_mininterval = 2.0`
  1) **Name + value set:** `tqdm_mininterval = 2.0`
  2) **Reflects:** Min seconds between tqdm refreshes.
  3) **Update higher/lower:** Higher reduces overhead; lower increases UI responsiveness.

- `tqdm_miniters = 50`
  1) **Name + value set:** `tqdm_miniters = 50`
  2) **Reflects:** Min iterations between tqdm refreshes.
  3) **Update higher/lower:** Higher reduces overhead; lower updates more frequently.

- `tqdm_postfix_every = 250`
  1) **Name + value set:** `tqdm_postfix_every = 250`
  2) **Reflects:** How often postfix status is updated.
  3) **Update higher/lower:** Higher reduces overhead; lower gives more live context.

- `use_sql_prevout_join = True`
  1) **Name + value set:** `use_sql_prevout_join = True`
  2) **Reflects:** Patch 3: set-based vinbuf→outpoints join in SQLite.
  3) **Update higher/lower:** Keep True for this build’s correctness/perf; False not supported in this patched version.

- `vinbuf_insert_chunk = 1_000_000`
  1) **Name + value set:** `vinbuf_insert_chunk = 1,000,000`
  2) **Reflects:** Batch size for inserting VIN rows into SQLite temp table.
  3) **Update higher/lower:** Higher for speed if RAM allows; lower if temp memory pressure/stalls.

- `vinbuf_fetch_chunk = 1_000_000`
  1) **Name + value set:** `vinbuf_fetch_chunk = 1,000,000`
  2) **Reflects:** Batch size for fetching aggregated join results from SQLite.
  3) **Update higher/lower:** Higher reduces fetch loops; lower reduces peak memory during transport.

---

### SanityConfig

- `run_sanity_checks = True`
  1) **Name + value set:** `run_sanity_checks = True`
  2) **Reflects:** Prints cluster summaries and prevout hit-rate diagnostics.
  3) **Update higher/lower:** Disable for max throughput once stable; enable during tuning.

- `run_prevout_db_sanity = True`
  1) **Name + value set:** `run_prevout_db_sanity = True`
  2) **Reflects:** Additional sampled prevout hit-rate test.
  3) **Update higher/lower:** Disable to save time once DB logic is trusted.

- `prevout_sanity_sample_limit = 200_000`
  1) **Name + value set:** `prevout_sanity_sample_limit = 200,000`
  2) **Reflects:** Sample size (confidence vs time).
  3) **Update higher/lower:** Higher for stronger confidence; lower for quick smoke checks.

- `prevout_sanity_parquet = None`
  1) **Name + value set:** `prevout_sanity_parquet = None`
  2) **Reflects:** Which parquet file is used for sanity sampling (None = auto-selected).
  3) **Update higher/lower:** Not numeric; set explicitly to compare runs consistently.

---

### HeuristicToggles

- `enable_coinjoin_filter = True`
  1) **Name + value set:** `enable_coinjoin_filter = True`
  2) **Reflects:** Whether mixing-like transactions are excluded from clustering.
  3) **Update higher/lower:** Disable for maximum linkage (more false merges); enable for conservative clustering.

- `enable_multi_input = True`
  1) **Name + value set:** `enable_multi_input = True`
  2) **Reflects:** Whether H1 multi-input unions are applied.
  3) **Update higher/lower:** Disable to study change-only behavior; enable for additional linkage.

- `enable_change = True`
  1) **Name + value set:** `enable_change = True`
  2) **Reflects:** Whether change heuristic unions are applied.
  3) **Update higher/lower:** Disable to study pure H1; enable for stronger wallet growth linkage.

- `enable_merge_guards = True`
  1) **Name + value set:** `enable_merge_guards = True`
  2) **Reflects:** Whether guardrails/caps/ratio/votes are enforced.
  3) **Update higher/lower:** Disable only for experiments; enable to avoid catastrophic bridging.

- `precreate_nodes_for_all_output_addrs = False`
  1) **Name + value set:** `precreate_nodes_for_all_output_addrs = False`
  2) **Reflects:** Whether every output address gets a UF node regardless of later resolution.
  3) **Update higher/lower:** True for maximal node universe; False to reduce UF size/RAM.

- `create_nodes_for_all_resolved_inputs = True`
  1) **Name + value set:** `create_nodes_for_all_resolved_inputs = True`
  2) **Reflects:** Preserve node universe for all resolved input addresses.
  3) **Update higher/lower:** Keep True for comparability/completeness; False only to reduce memory.

---

### HeuristicParams

- `dust_sats = 546`
  1) **Name + value set:** `dust_sats = 546`
  2) **Reflects:** Dust cutoff for “spendable” output logic.
  3) **Update higher/lower:** Higher if dust spam pollutes patterns; lower rarely.

- `max_fee_abs_sats = 50_000_000`
  1) **Name + value set:** `max_fee_abs_sats = 50,000,000`
  2) **Reflects:** Absolute fee sanity cap for change detection.
  3) **Update higher/lower:** Higher if high-fee regimes; lower for stricter filtering.

- `max_fee_frac = 0.05`
  1) **Name + value set:** `max_fee_frac = 0.05`
  2) **Reflects:** Fee cap as fraction of total input value.
  3) **Update higher/lower:** Higher if fee spikes expected; lower to be conservative.

- `max_change_inputs_utxos = 10`
  1) **Name + value set:** `max_change_inputs_utxos = 10`
  2) **Reflects:** Max input UTXOs to allow change heuristic.
  3) **Update higher/lower:** Higher includes consolidations (riskier); lower makes change stricter.

- `max_change_spendable_outs = 2`
  1) **Name + value set:** `max_change_spendable_outs = 2`
  2) **Reflects:** Max spendable outputs allowed for change heuristic.
  3) **Update higher/lower:** Higher increases coverage but risk; lower reduces false positives and coverage.

- `change_require_2_outputs = True`
  1) **Name + value set:** `change_require_2_outputs = True`
  2) **Reflects:** Require exactly 2 spendable outputs for change detection.
  3) **Update higher/lower:** False to allow 3-output logic; True for tightest behavior.

- `multi_input_policy = "one_output"`
  1) **Name + value set:** `multi_input_policy = "one_output"`
  2) **Reflects:** H1 fires only if there is exactly 1 spendable output (very conservative).
  3) **Update higher/lower:** `"one_or_two_nonmix"` increases coverage but raises false-positive risk.

- `enable_optimal_change_constraint = True`
  1) **Name + value set:** `enable_optimal_change_constraint = True`
  2) **Reflects:** Extra constraint intended to mimic coin selection optimality.
  3) **Update higher/lower:** Disable if too restrictive; enable for conservative change inference.

- `optimal_change_slack_sats = 0`
  1) **Name + value set:** `optimal_change_slack_sats = 0`
  2) **Reflects:** Tolerance around optimal-change inequality.
  3) **Update higher/lower:** Increase slightly if near-miss rejects due to rounding/fees; keep 0 for strictness.

---

### MergeGuardParams

- `max_merged_component_size = 1_500_000`
  1) **Name + value set:** `max_merged_component_size = 1,500,000`
  2) **Reflects:** Hard fail-safe cap on entity size (limits worst-case damage).
  3) **Update higher/lower:** Higher to allow very large services; lower to reduce catastrophic merge impact.

- `merge_ratio_guard = True`
  1) **Name + value set:** `merge_ratio_guard = True`
  2) **Reflects:** Enables lopsided-merge protection on CHANGE unions.
  3) **Update higher/lower:** Disable only if you accept bridging risk; otherwise keep enabled.

- `merge_ratio_max = 200.0`
  1) **Name + value set:** `merge_ratio_max = 200.0`
  2) **Reflects:** Maximum `s_big/s_small` allowed when ratio guard applies.
  3) **Update higher/lower:** Higher allows more growth (more risk); lower blocks more merges (more conservative).

- `merge_ratio_big_cluster_min = 50_000`
  1) **Name + value set:** `merge_ratio_big_cluster_min = 50,000`
  2) **Reflects:** Only apply ratio guard after a cluster is “big”.
  3) **Update higher/lower:** Higher delays ratio checks; lower/0 applies ratio checks earlier.

- `merge_ratio_small_floor = 50`
  1) **Name + value set:** `merge_ratio_small_floor = 50`
  2) **Reflects:** Patch 1: ratio guard ignores tiny components (prevents freezing on singleton change).
  3) **Update higher/lower:** Higher reduces freezing further; lower makes guard stricter against small attachments.

- `enable_change_merge_votes = True`
  1) **Name + value set:** `enable_change_merge_votes = True`
  2) **Reflects:** Enables voting gate for big–big CHANGE merges (bridging defense).
  3) **Update higher/lower:** Disable for maximum merge permissiveness; enable to protect large clusters.

- `change_vote_min_side = 25_000`
  1) **Name + value set:** `change_vote_min_side = 25,000`
  2) **Reflects:** Votes trigger only if `min(sa, sb)` exceeds this (both sides big).
  3) **Update higher/lower:** Higher limits voting to very large merges; lower protects medium clusters too.

- `change_vote_merged_trigger = 0`
  1) **Name + value set:** `change_vote_merged_trigger = 0`
  2) **Reflects:** Optional trigger by merged size (0 = disabled).
  3) **Update higher/lower:** Set >0 (e.g., 100k) if you want voting on huge merged results even with uneven sides.

- `change_votes_required = 2`
  1) **Name + value set:** `change_votes_required = 2`
  2) **Reflects:** Number of repeated observations required before allowing a voted merge.
  3) **Update higher/lower:** Higher = more conservative; lower = faster merging (riskier).

- `ratio_guard_sample_n = 1000`
  1) **Name + value set:** `ratio_guard_sample_n = 1000`
  2) **Reflects:** How many ratio-guard events to log for diagnostics (no behavior change).
  3) **Update higher/lower:** Higher yields more evidence (bigger logs); 0 disables logging overhead.

- `ratio_guard_sample_path = "/media/vatereal/Main/outputs/ratio_guard_samples_2014.tsv"`
  1) **Name + value set:** `ratio_guard_sample_path = outputs_dir / "ratio_guard_samples_2014.tsv"`
  2) **Reflects:** Where diagnostic ratio-guard TSV is written.
  3) **Update higher/lower:** Not numeric; make run-specific to avoid overwriting/comparing mixed runs.

- `debug_large_change_merges = False`
  1) **Name + value set:** `debug_large_change_merges = False`
  2) **Reflects:** Extra logging for large CHANGE merges (debug-only).
  3) **Update higher/lower:** Enable only when investigating suspicious merges.

- `debug_change_merge_min = 50_000`
  1) **Name + value set:** `debug_change_merge_min = 50,000`
  2) **Reflects:** Threshold for debug log entries when debug is enabled.
  3) **Update higher/lower:** Lower captures more events; higher captures only very large events.

- `debug_change_merge_log_path = "/media/vatereal/Main/outputs/large_change_merges_2014.tsv"`
  1) **Name + value set:** `debug_change_merge_log_path = outputs_dir / "large_change_merges_2014.tsv"`
  2) **Reflects:** Where large-merge debug logs are written.
  3) **Update higher/lower:** Not numeric; keep run-specific to avoid mixing logs across experiments.


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import annotations

import gc
import glob
import math
import os
import random
import sqlite3
import sys
from collections import Counter
from dataclasses import dataclass
from datetime import date, timedelta
from pathlib import Path
from typing import Literal

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm.auto import tqdm


# =============================================================================
# Determinism helpers
# =============================================================================

def set_determinism(seed: int) -> None:
    """
    Makes the pipeline *more reproducible* across runs.

    Notes:
      - This pipeline is fundamentally deterministic given a deterministic stream order.
      - The main nondeterminism you observed comes from:
          * SQLite group_concat(DISTINCT ...) ordering (fixed by sorting lists post-transport)
          * Polars unique() order (fixed by sorting before insertion)
      - We still set seeds and env to reduce incidental variation.
    """
    os.environ.setdefault("PYTHONHASHSEED", str(seed))
    random.seed(seed)
    np.random.seed(seed)


# =============================================================================
# Config objects (NO DEFAULTS — everything must be explicit)
# =============================================================================

ValueUnitMode = Literal["infer", "btc", "sats"]
MultiInputPolicy = Literal["one_output", "one_or_two_nonmix"]


@dataclass(frozen=True)
class PathsConfig:
    parquet_io_glob: str
    outputs_dir: Path
    outpoint_db_path: Path
    entity_map_out_path: Path


@dataclass(frozen=True)
class AnalysisConfig:
    analysis_start: date
    analysis_end_exclusive: date
    outpoint_db_lookback_days: int
    value_unit_mode: ValueUnitMode


@dataclass(frozen=True)
class OutputConfig:
    reset_outpoint_db: bool
    write_entity_mapping: bool
    entity_map_compression: str
    entity_write_batch: int
    top_k_clusters_print: int


@dataclass(frozen=True)
class PlotConfig:
    enable_plots: bool
    zipf_top_k: int
    focus_max_size: int
    log_bins: int


@dataclass(frozen=True)
class PerfConfig:
    outpoint_commit_every_rows: int
    prevout_hybrid_threshold: int
    prevout_lookup_chunk: int
    iter_buffer_in: int
    iter_buffer_out: int
    iter_buffer_grouped: int
    gc_every_n_files: int
    tqdm_mininterval: float
    tqdm_miniters: int
    tqdm_postfix_every: int

    # Patch 3: set-based prevout join path (SQLite join + SQLite aggregates + Polars transport)
    use_sql_prevout_join: bool
    vinbuf_insert_chunk: int
    vinbuf_fetch_chunk: int


@dataclass(frozen=True)
class SanityConfig:
    run_sanity_checks: bool
    run_prevout_db_sanity: bool
    prevout_sanity_sample_limit: int
    prevout_sanity_parquet: str | None


@dataclass(frozen=True)
class HeuristicToggles:
    enable_coinjoin_filter: bool
    enable_multi_input: bool
    enable_change: bool
    enable_merge_guards: bool
    precreate_nodes_for_all_output_addrs: bool
    create_nodes_for_all_resolved_inputs: bool


@dataclass(frozen=True)
class HeuristicParams:
    dust_sats: int
    max_fee_abs_sats: int
    max_fee_frac: float
    max_change_inputs_utxos: int
    max_change_spendable_outs: int
    change_require_2_outputs: bool
    multi_input_policy: MultiInputPolicy
    enable_optimal_change_constraint: bool
    optimal_change_slack_sats: int


UltraVoteRule = tuple[int, int]  # (threshold, votes_required)


@dataclass(frozen=True)
class MergeGuardParams:
    # Patch A: Safety ceiling (raised; still recommended as a fuse)
    max_merged_component_size: int

    # Ratio guard (CHANGE only) + ignore tiny components (singleton change)
    merge_ratio_guard: bool
    merge_ratio_max: float
    merge_ratio_big_cluster_min: int
    merge_ratio_small_floor: int

    # Big–big gating (CHANGE only): require multiple "confirmations" per component-pair
    # (Patch A: strengthened tiers above 250k/500k/1M via ultra_change_vote_rules)
    enable_change_merge_votes: bool
    change_vote_min_side: int
    change_vote_merged_trigger: int
    change_votes_required: int
    ultra_change_vote_rules: tuple[UltraVoteRule, ...]

    # Patch B: alternative confirmation signal when pair repetition is rare
    # "change component attempts to attach to too many distinct large anchors"
    enable_change_degree_guard: bool
    change_degree_large_min: int
    change_degree_small_min: int
    change_degree_max_distinct_large: int
    change_degree_set_cap: int

    # Diagnostics: ratio-guard sample log
    ratio_guard_sample_n: int
    ratio_guard_sample_path: Path

    # Patch B: constraint event log (vote pending + degree guard)
    constraint_log_n: int
    constraint_log_path: Path

    # Debug logging of large merges (optional)
    debug_large_change_merges: bool
    debug_change_merge_min: int
    debug_change_merge_log_path: Path


@dataclass(frozen=True)
class ConfidenceConfig:
    """
    Proxy "probability" that an address is *meaningfully clustered* (diagnostic / downstream weight).
    This does NOT claim statistical calibration; it's an evidence-based score in [0,1].
    """
    enable_confidence_proxy: bool
    confidence_out_path: Path
    confidence_compression: str
    confidence_write_batch: int

    # scoring knobs
    size_norm: int
    size_bonus_max: float
    p_singleton: float
    p_base_0: float
    p_base_1: float
    p_base_2plus: float

    include_cluster_size: bool
    include_evidence_bits: bool


# =============================================================================
# Utility helpers
# =============================================================================

def extract_day_from_path(path: str) -> date | None:
    p = Path(path)
    for part in p.parts:
        if part.startswith("day="):
            day_str = part.split("=", 1)[1]
            try:
                return date.fromisoformat(day_str)
            except ValueError:
                return None
    return None


def addr_type(addr: str) -> str:
    if addr.startswith("1"):
        return "p2pkh"
    if addr.startswith("3"):
        return "p2sh"
    if addr.startswith("bc1q"):
        return "bech32_p2wpkh"
    if addr.startswith("bc1p"):
        return "taproot"
    return "other"


def infer_value_unit_from_sample(sample_vals: list[float]) -> str:
    if not sample_vals:
        return "btc"
    mx = max(sample_vals)
    return "sats" if mx >= 1e6 else "btc"


class UnionFind:
    """
    Union-Find with rank, path compression, component sizes,
    plus Patch B metadata: per-root set of distinct "large anchors" that a CHANGE component tried to attach to.
    """
    def __init__(self):
        self.parent: list[int] = []
        self.rank: list[int] = []
        self.size: list[int] = []

        # Patch B: root -> set(anchor_root) (only meaningful for "change side" components)
        self.change_large_anchor_roots: dict[int, set[int]] = {}

    def make_set(self) -> int:
        idx = len(self.parent)
        self.parent.append(idx)
        self.rank.append(0)
        self.size.append(1)
        return idx

    def find(self, x: int) -> int:
        parent = self.parent
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union_roots(self, rx: int, ry: int, *, change_degree_set_cap: int) -> int:
        """
        Union roots (rx, ry) and return new root.
        Merges Patch B sets deterministically, capping to change_degree_set_cap.
        """
        if rx == ry:
            return rx

        parent = self.parent
        rank = self.rank
        size = self.size

        # union by rank; deterministic tie-break already given by call order in your stream
        if rank[rx] < rank[ry]:
            parent[rx] = ry
            size[ry] += size[rx]
            new_root, old_root = ry, rx
        elif rank[rx] > rank[ry]:
            parent[ry] = rx
            size[rx] += size[ry]
            new_root, old_root = rx, ry
        else:
            parent[ry] = rx
            rank[rx] += 1
            size[rx] += size[ry]
            new_root, old_root = rx, ry

        # Patch B: merge anchor-root sets
        sa = self.change_large_anchor_roots.get(new_root)
        sb = self.change_large_anchor_roots.get(old_root)

        if sa is None and sb is None:
            return new_root

        merged: set[int] = set()
        if sa:
            merged |= sa
        if sb:
            merged |= sb

        if len(merged) > change_degree_set_cap:
            merged = set(sorted(merged)[:change_degree_set_cap])

        self.change_large_anchor_roots[new_root] = merged
        self.change_large_anchor_roots.pop(old_root, None)
        return new_root


def _count_dupe_values(values: list[int], dust_sats: int) -> tuple[int, int]:
    vals = [v for v in values if v >= dust_sats]
    if not vals:
        return 0, 0
    cnt = Counter(vals)
    return max(cnt.values()), len(cnt)


def detect_mixing_like(n_in_utxos: int, spendable_outs: list[tuple[str, int]], dust_sats: int) -> bool:
    # Conservative "mixing-like" flags: equal-amount patterns and symmetry.
    n_out = len(spendable_outs)
    if n_in_utxos < 3 or n_out < 3:
        return False

    out_vals = [v for (_a, v) in spendable_outs]
    out_types = [addr_type(a) for (a, _v) in spendable_outs if a is not None]
    unique_vals = len(set(out_vals))
    unique_types = len(set(out_types)) if out_types else 0
    max_dupe, distinct_vals_non_dust = _count_dupe_values(out_vals, dust_sats)

    if max_dupe >= 2 and distinct_vals_non_dust >= 2:
        return True
    if abs(n_in_utxos - n_out) <= 1 and min(n_in_utxos, n_out) >= 3:
        return True
    if n_out >= 4 and unique_vals <= (n_out // 2):
        return True
    if n_out >= 4 and unique_types == 1 and max_dupe >= 2:
        return True
    return False


def multi_input_is_safe(policy: MultiInputPolicy, n_in_utxos: int, n_spendable_out: int, is_mixing_like: bool) -> bool:
    if n_in_utxos < 2:
        return False
    if policy == "one_output":
        return n_spendable_out == 1
    if policy == "one_or_two_nonmix":
        if n_spendable_out == 1:
            return True
        if n_spendable_out == 2 and not is_mixing_like:
            return True
        return False
    raise ValueError(f"Unknown multi_input_policy={policy}")


# =============================================================================
# SQLite outpoint DB + TEMP join buffers
# =============================================================================

def open_outpoint_db(db_path: Path, reset: bool) -> sqlite3.Connection:
    if reset and db_path.exists():
        db_path.unlink()

    conn = sqlite3.connect(str(db_path))
    cur = conn.cursor()

    cur.execute("PRAGMA journal_mode=WAL;")
    cur.execute("PRAGMA synchronous=OFF;")
    cur.execute("PRAGMA temp_store=MEMORY;")
    cur.execute("PRAGMA cache_size=-2000000;")   # ~2GB cache (negative => KB)
    cur.execute("PRAGMA wal_autocheckpoint=5000;")

    conn.isolation_level = None
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS outpoints (
            txid TEXT NOT NULL,
            n    INTEGER NOT NULL,
            address TEXT NOT NULL,
            value_sats INTEGER NOT NULL,
            PRIMARY KEY(txid, n)
        ) WITHOUT ROWID;
        """
    )
    return conn


def init_lookup_tables(conn: sqlite3.Connection) -> None:
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS keybuf;")
    cur.execute(
        """
        CREATE TEMP TABLE keybuf (
            txid TEXT NOT NULL,
            n    INTEGER NOT NULL
        );
        """
    )

    cur.execute("DROP TABLE IF EXISTS vinbuf;")
    cur.execute(
        """
        CREATE TEMP TABLE vinbuf (
            spend_txid TEXT NOT NULL,
            prev_txid  TEXT NOT NULL,
            prev_n     INTEGER NOT NULL
        );
        """
    )


def reset_vinbuf(conn: sqlite3.Connection) -> None:
    conn.execute("DELETE FROM vinbuf;")


def insert_outpoints_no_commit(conn: sqlite3.Connection, rows: list[tuple[str, int, str, int]]) -> None:
    if rows:
        conn.executemany(
            "INSERT OR IGNORE INTO outpoints(txid, n, address, value_sats) VALUES (?, ?, ?, ?);",
            rows,
        )


# Legacy hybrid lookup kept for sanity check helper
def lookup_outpoints_or(conn: sqlite3.Connection, keys: list[tuple[str, int]], chunk_size: int) -> dict[tuple[str, int], tuple[str, int]]:
    if not keys:
        return {}
    keys = list(dict.fromkeys(keys))
    cur = conn.cursor()
    out: dict[tuple[str, int], tuple[str, int]] = {}

    for i in range(0, len(keys), chunk_size):
        chunk = keys[i : i + chunk_size]
        where = " OR ".join(["(txid=? AND n=?)"] * len(chunk))
        params = [p for k in chunk for p in k]
        cur.execute(f"SELECT txid, n, address, value_sats FROM outpoints WHERE {where};", params)
        for txid, n, address, value_sats in cur.fetchall():
            out[(txid, int(n))] = (address, int(value_sats))
    return out


def lookup_outpoints_join(conn: sqlite3.Connection, keys: list[tuple[str, int]], chunk_size: int) -> dict[tuple[str, int], tuple[str, int]]:
    if not keys:
        return {}
    keys = list(dict.fromkeys(keys))
    cur = conn.cursor()
    out: dict[tuple[str, int], tuple[str, int]] = {}

    for i in range(0, len(keys), chunk_size):
        chunk = keys[i : i + chunk_size]
        cur.execute("DELETE FROM keybuf;")
        cur.executemany("INSERT INTO keybuf(txid, n) VALUES (?, ?);", chunk)
        cur.execute(
            """
            SELECT k.txid, k.n, o.address, o.value_sats
            FROM keybuf k
            JOIN outpoints o
              ON o.txid = k.txid AND o.n = k.n;
            """
        )
        for txid, n, address, value_sats in cur.fetchall():
            out[(txid, int(n))] = (address, int(value_sats))
    return out


def lookup_outpoints_hybrid(conn: sqlite3.Connection, keys: list[tuple[str, int]], hybrid_threshold: int, join_chunk: int) -> dict[tuple[str, int], tuple[str, int]]:
    if not keys:
        return {}
    if len(keys) < hybrid_threshold:
        return lookup_outpoints_or(conn, keys, chunk_size=500)
    return lookup_outpoints_join(conn, keys, chunk_size=join_chunk)


# =============================================================================
# Patch 3: Resolve + aggregate inputs inside SQLite, transport to Polars
# =============================================================================

def fetch_prevouts_aggs_polars(conn: sqlite3.Connection, fetch_chunk: int) -> pl.DataFrame:
    """
    Returns one row per spend_txid:

      txid: str
      resolved_cnt: int
      sum_inputs_sats: int
      min_input_sats: int
      in_addrs: list[str]   (DISTINCT addresses among resolved inputs for that tx)

    Note: group_concat() uses commas; Bitcoin addresses do not contain commas.
    """
    cur = conn.cursor()
    cur.execute(
        """
        SELECT
            v.spend_txid                                 AS txid,
            COUNT(*)                                      AS resolved_cnt,
            SUM(o.value_sats)                             AS sum_inputs_sats,
            MIN(o.value_sats)                             AS min_input_sats,
            group_concat(DISTINCT o.address)              AS in_addrs_csv
        FROM vinbuf v
        JOIN outpoints o
          ON o.txid = v.prev_txid AND o.n = v.prev_n
        GROUP BY v.spend_txid;
        """
    )

    dfs: list[pl.DataFrame] = []
    while True:
        rows = cur.fetchmany(fetch_chunk)
        if not rows:
            break
        dfs.append(
            pl.DataFrame(
                rows,
                schema=["txid", "resolved_cnt", "sum_inputs_sats", "min_input_sats", "in_addrs_csv"],
                orient="row",
            )
        )

    if not dfs:
        return pl.DataFrame(
            schema={
                "txid": pl.Utf8,
                "resolved_cnt": pl.Int64,
                "sum_inputs_sats": pl.Int64,
                "min_input_sats": pl.Int64,
                "in_addrs": pl.List(pl.Utf8),
            }
        )

    df = pl.concat(dfs, how="vertical", rechunk=True)

    df = df.with_columns(
        pl.col("in_addrs_csv")
        .fill_null("")
        .str.split(",")
        .alias("in_addrs")
    ).drop("in_addrs_csv")

    # Edge-case: NULL -> "" -> [""] should become []
    df = df.with_columns(
        pl.when(pl.col("in_addrs") == [""])
        .then(pl.lit([], dtype=pl.List(pl.Utf8)))
        .otherwise(pl.col("in_addrs"))
        .alias("in_addrs")
    )

    # Determinism: sort the per-tx list of input addresses
    df = df.with_columns(pl.col("in_addrs").list.sort().alias("in_addrs"))

    return df


# =============================================================================
# Reporting helpers (prints centralized below)
# =============================================================================

OUTPUT_DIR_VALUES = {"out", "vout", "output", "o"}
INPUT_DIR_VALUES  = {"in", "vin", "input", "i"}


def run_sanity_checks(
    n_nodes: int,
    node_to_entity: list[int] | np.ndarray | None = None,
    cluster_size_counter: Counter | None = None,
    prevout_lookups: int | None = None,
    prevout_hits: int | None = None,
    top_k: int = 20,
) -> None:
    """
    Prints:
      - largest cluster fraction of nodes
      - top cluster sizes
      - entity count stats
      - optional prevout hit-rate
    Provide either node_to_entity OR cluster_size_counter.
    """
    if cluster_size_counter is None:
        if node_to_entity is None:
            raise ValueError("Provide either node_to_entity or cluster_size_counter.")
        cluster_size_counter = Counter(list(node_to_entity))

    sizes = list(cluster_size_counter.values())
    if not sizes:
        print("[SANITY] No clusters found (sizes empty).", flush=True)
        return

    sizes_sorted = sorted(sizes, reverse=True)
    total_nodes_from_sizes = sum(sizes_sorted)
    largest = sizes_sorted[0]
    frac = largest / total_nodes_from_sizes if total_nodes_from_sizes else float("nan")

    print("\n[SANITY] Cluster summary", flush=True)
    print(f"  UF nodes (n_nodes): {n_nodes:,}", flush=True)
    print(f"  Total nodes from cluster sizes: {total_nodes_from_sizes:,}", flush=True)
    if total_nodes_from_sizes != n_nodes:
        print("  [WARN] sum(cluster_sizes) != n_nodes  -> mismatch suggests a bug in mapping logic.", flush=True)
    print(f"  Entities (clusters): {len(cluster_size_counter):,}", flush=True)
    print(f"  Largest cluster size: {largest:,}", flush=True)
    print(f"  Largest cluster fraction of nodes: {frac:.2%}", flush=True)

    print(f"\n[SANITY] Top {top_k} cluster sizes:", flush=True)
    print(" ", sizes_sorted[:top_k], flush=True)

    def pct(p: float) -> int:
        s_asc = sorted(sizes_sorted)
        idx = max(0, min(len(s_asc) - 1, math.ceil(p * len(s_asc)) - 1))
        return int(s_asc[idx])

    med = int(sorted(sizes_sorted)[len(sizes_sorted) // 2])
    print("\n[SANITY] Quick distribution stats", flush=True)
    print(f"  Median cluster size: {med:,}", flush=True)
    print(f"  90th percentile cluster size: {pct(0.90):,}", flush=True)
    print(f"  99th percentile cluster size: {pct(0.99):,}", flush=True)

    if prevout_lookups is not None and prevout_hits is not None:
        rate = (prevout_hits / prevout_lookups) if prevout_lookups else float("nan")
        print("\n[SANITY] Prevout lookup hit-rate (DB)", flush=True)
        print(f"  Lookups:  {prevout_lookups:,}", flush=True)
        print(f"  Hits:     {prevout_hits:,}", flush=True)
        print(f"  Hit-rate: {rate:.2%}", flush=True)


def prevout_join_sanity_polars(one_io_parquet: str) -> None:
    """
    Pure-Polars sanity on a single parquet:
      - prev_vout integer-likeness
      - duplicate outpoints (txid,n)
      - left join vin->vout inside same parquet
      - match explosion check
    Note: This join only resolves prevouts that point to outputs present in this same file.
    Your real pipeline resolves against the outpoint DB across days.
    """
    df = pl.read_parquet(one_io_parquet, columns=["dir","txid","n","prev_txid","prev_vout","address","value"])
    df = df.with_columns(pl.col("dir").cast(pl.Utf8, strict=False).str.to_lowercase().alias("dir"))

    vout = (
        df.filter(pl.col("dir").is_in(list(OUTPUT_DIR_VALUES)))
          .select([
              pl.col("txid").alias("out_txid"),
              pl.col("n").cast(pl.Int64, strict=False).alias("out_n"),
              pl.col("address").alias("out_address"),
              pl.col("value").alias("out_value"),
          ])
          .filter(pl.col("out_txid").is_not_null() & pl.col("out_n").is_not_null())
    )

    vin = (
        df.filter(pl.col("dir").is_in(list(INPUT_DIR_VALUES)))
          .select([
              pl.col("txid").alias("spend_txid"),
              pl.col("prev_txid").alias("prev_txid"),
              pl.col("prev_vout").alias("prev_vout"),
          ])
          .filter(pl.col("spend_txid").is_not_null() & pl.col("prev_txid").is_not_null())
    )

    print("\n[PREVOUT-SANITY | POLARS]", flush=True)
    print("File:", one_io_parquet, flush=True)
    print("Rows:", {"vin": vin.height, "vout": vout.height}, flush=True)

    vin_nonnull = vin.filter(pl.col("prev_vout").is_not_null())
    if vin_nonnull.height > 0:
        frac_integerlike = (
            vin_nonnull
            .with_columns(((pl.col("prev_vout") - pl.col("prev_vout").floor()).abs() < 1e-9).alias("is_intlike"))
            .select(pl.col("is_intlike").mean())
            .item()
        )
        print(f"prev_vout integer-like fraction: {float(frac_integerlike):.6f}", flush=True)

    vin2 = vin.with_columns(
        pl.when(pl.col("prev_vout").is_not_null())
          .then(pl.col("prev_vout").cast(pl.Int64, strict=False))
          .otherwise(None)
          .alias("prev_vout_i64")
    )

    dupe_outpoints = (
        vout.group_by(["out_txid","out_n"])
            .len()
            .filter(pl.col("len") > 1)
            .height
    )
    print("Duplicate outpoints (should be 0):", dupe_outpoints, flush=True)

    joined = (
        vin2.join(
            vout,
            left_on=["prev_txid","prev_vout_i64"],
            right_on=["out_txid","out_n"],
            how="left",
        )
    )

    resolved = joined.filter(pl.col("out_address").is_not_null()).height
    lookups  = vin.height
    print(f"Prevout resolution hit-rate (within-file): {resolved}/{lookups} = {resolved/lookups:.2%}" if lookups else "No vin rows", flush=True)

    if joined.height > 0:
        max_matches_per_input = (
            joined.group_by(["spend_txid","prev_txid","prev_vout_i64"])
                  .len()
                  .select(pl.col("len").max())
                  .item()
        )
        print("Max matches per (spend_txid, prev_txid, prev_vout):", int(max_matches_per_input) if max_matches_per_input is not None else None, flush=True)

    sample_unresolved = (
        joined.filter(pl.col("out_address").is_null())
              .select(["spend_txid","prev_txid","prev_vout_i64"])
              .head(5)
    )
    print("\nSample unresolved inputs (within-file):", flush=True)
    print(sample_unresolved, flush=True)


def prevout_join_sanity_db(
    conn: sqlite3.Connection,
    one_io_parquet: str,
    *,
    sample_limit: int,
    hybrid_threshold: int,
    join_chunk: int,
) -> None:
    """
    Sanity-check using your ACTUAL outpoint DB resolution:
      - reads vin from a single parquet
      - casts prev_vout -> int64
      - looks up in DB using lookup_outpoints_hybrid(conn, keys, hybrid_threshold, join_chunk)
      - prints DB hit-rate and sample unresolved keys
    Uses only a sample of inputs (sample_limit) to keep it cheap.
    """
    df = pl.read_parquet(one_io_parquet, columns=["dir","txid","prev_txid","prev_vout"])
    df = df.with_columns(pl.col("dir").cast(pl.Utf8, strict=False).str.to_lowercase().alias("dir"))

    vin = (
        df.filter(pl.col("dir").is_in(list(INPUT_DIR_VALUES)))
          .select([
              pl.col("txid").alias("spend_txid"),
              pl.col("prev_txid").alias("prev_txid"),
              pl.col("prev_vout").alias("prev_vout"),
          ])
          .filter(pl.col("prev_txid").is_not_null() & pl.col("prev_vout").is_not_null())
          .head(sample_limit)
    )

    print("\n[PREVOUT-SANITY | DB]", flush=True)
    print("File:", one_io_parquet, flush=True)
    print("Vin sample rows:", vin.height, flush=True)

    if vin.height == 0:
        print("No vin rows in sample.", flush=True)
        return

    frac_integerlike = (
        vin.with_columns(((pl.col("prev_vout") - pl.col("prev_vout").floor()).abs() < 1e-9).alias("is_intlike"))
           .select(pl.col("is_intlike").mean())
           .item()
    )
    print(f"prev_vout integer-like fraction (sample): {float(frac_integerlike):.6f}", flush=True)

    vin2 = vin.with_columns(
        pl.col("prev_vout").cast(pl.Int64, strict=False).alias("prev_vout_i64")
    ).filter(pl.col("prev_vout_i64").is_not_null())

    keys = [
        (prev_txid, int(prev_n))
        for prev_txid, prev_n in vin2.select(["prev_txid", "prev_vout_i64"]).iter_rows(named=False, buffer_size=200_000)
    ]
    if not keys:
        print("No usable keys after casting prev_vout.", flush=True)
        return

    hits = lookup_outpoints_hybrid(conn, keys, hybrid_threshold=hybrid_threshold, join_chunk=join_chunk)
    resolved = len(hits)
    lookups = len(keys)

    print(f"DB prevout hit-rate (sample): {resolved}/{lookups} = {resolved/lookups:.2%}", flush=True)

    unresolved = []
    for k in keys[:5000]:
        if k not in hits:
            unresolved.append(k)
            if len(unresolved) >= 10:
                break
    if unresolved:
        print("Sample unresolved (prev_txid, prev_n):", flush=True)
        for k in unresolved:
            print(" ", k, flush=True)
    else:
        print("No unresolved keys in first 5000 keys (good sign).", flush=True)


def compute_top_cluster_evidence_composition(
    *,
    node_to_entity: np.ndarray,
    multi_change_flags: bytearray,
    top_entity_id: int,
) -> dict[str, float | int]:
    """
    Evidence bits:
      bit 1 -> multi-input evidence
      bit 2 -> change-output evidence
      bit 4 -> change-anchor evidence

    Returns counts + shares for the top cluster, including bit2-only share.
    """
    n_nodes = int(node_to_entity.size)
    if n_nodes == 0:
        return {
            "top_entity_id": int(top_entity_id),
            "n_top": 0,
            "n_bit1": 0, "n_bit2": 0, "n_bit4": 0,
            "n_bit2_only": 0,
            "share_bit1": float("nan"),
            "share_bit2": float("nan"),
            "share_bit4": float("nan"),
            "share_bit2_only": float("nan"),
        }

    if len(multi_change_flags) < n_nodes:
        raise ValueError(f"multi_change_flags length {len(multi_change_flags)} < n_nodes {n_nodes}")

    bits = np.frombuffer(multi_change_flags, dtype=np.uint8, count=n_nodes)
    mask = (node_to_entity == int(top_entity_id))
    n_top = int(mask.sum())
    if n_top == 0:
        return {
            "top_entity_id": int(top_entity_id),
            "n_top": 0,
            "n_bit1": 0, "n_bit2": 0, "n_bit4": 0,
            "n_bit2_only": 0,
            "share_bit1": float("nan"),
            "share_bit2": float("nan"),
            "share_bit4": float("nan"),
            "share_bit2_only": float("nan"),
        }

    top_bits = bits[mask]
    n_bit1 = int(np.count_nonzero(top_bits & 1))
    n_bit2 = int(np.count_nonzero(top_bits & 2))
    n_bit4 = int(np.count_nonzero(top_bits & 4))
    n_bit2_only = int(np.count_nonzero(top_bits == 2))

    share_bit1 = n_bit1 / n_top
    share_bit2 = n_bit2 / n_top
    share_bit4 = n_bit4 / n_top
    share_bit2_only = n_bit2_only / n_top

    return {
        "top_entity_id": int(top_entity_id),
        "n_top": n_top,
        "n_bit1": n_bit1,
        "n_bit2": n_bit2,
        "n_bit4": n_bit4,
        "n_bit2_only": n_bit2_only,
        "share_bit1": float(share_bit1),
        "share_bit2": float(share_bit2),
        "share_bit4": float(share_bit4),
        "share_bit2_only": float(share_bit2_only),
    }


# =============================================================================
# Plot helpers
# =============================================================================

def _log_bins(max_val: int, bins: int) -> np.ndarray:
    max_val = max(1, int(max_val))
    return np.logspace(0, math.log10(max_val), num=bins)


def plot_hist_logbins(sizes: np.ndarray, title: str, color: str, bins: int, max_x: int | None) -> None:
    if sizes.size == 0:
        print(f"[PLOT] Skipping empty plot: {title}", flush=True)
        return
    if max_x is None:
        max_x = int(sizes.max())
    b = _log_bins(max_x, bins)
    plt.figure(figsize=(11, 6))
    plt.hist(sizes, bins=b, log=True, color=color, edgecolor="black", linewidth=0.7)
    plt.xscale("log")
    plt.xlabel("Cluster size (addresses per entity)")
    plt.ylabel("Frequency (log scale)")
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", linewidth=0.35, alpha=0.6)
    plt.tight_layout()
    plt.show()
    plt.close()


def plot_zipf(sizes: np.ndarray, title: str, color: str, top_k: int) -> None:
    if sizes.size == 0:
        print(f"[PLOT] Skipping empty plot: {title}", flush=True)
        return
    top_k = min(top_k, sizes.size)
    top = np.partition(sizes, -top_k)[-top_k:]
    top_sorted = np.sort(top)[::-1]
    ranks = np.arange(1, top_sorted.size + 1)

    plt.figure(figsize=(11, 6))
    plt.plot(ranks, top_sorted, color=color, linewidth=1.3)
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel("Rank (log scale)")
    plt.ylabel("Cluster size (log scale)")
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", linewidth=0.35, alpha=0.6)
    plt.tight_layout()
    plt.show()
    plt.close()


# =============================================================================
# Main executor (explicit-parameter entrypoint)
# =============================================================================

def run_entity_clustering(
    paths: PathsConfig,
    analysis: AnalysisConfig,
    outputs: OutputConfig,
    plots: PlotConfig,
    perf: PerfConfig,
    sanity: SanityConfig,
    toggles: HeuristicToggles,
    params: HeuristicParams,
    guards: MergeGuardParams,
    confidence: ConfidenceConfig,
    *,
    determinism_seed: int,
) -> None:
    """
    Address clustering with:
      - H1 multi-input heuristic under a configurable safety policy
      - Change heuristic (2-output default)
      - Mixing-like filter
      - Merge guards:
          * absolute max component size (all unions)
          * ratio guard ONLY for CHANGE unions (+ small-floor to avoid freezing on singletons)
          * big–big vote-gating with ultra-large strengthening (Patch A)
          * change-degree guard (Patch B) for bridge-y change components
      - Patch 3:
          * prevout resolution via set-based SQLite JOIN (vinbuf → outpoints)
          * per-tx input aggregates computed in SQLite, transported to Polars
      - Node universe:
          * create UF nodes for all resolved input addresses (toggle)
      - Diagnostics:
          * skip reason counters
          * ratio-guard sample log
          * constraint event log (vote pending + degree guard) + uniqueness/repeat diagnostics (Patch B)
      - Confidence proxy output (optional)
    """
    set_determinism(determinism_seed)

    try:
        sys.stdout.reconfigure(line_buffering=True)
        sys.stderr.reconfigure(line_buffering=True)
    except Exception:
        pass

    pl.Config.set_tbl_rows(20)
    pl.Config.set_fmt_str_lengths(80)

    paths.outputs_dir.mkdir(parents=True, exist_ok=True)

    io_paths = sorted(glob.glob(paths.parquet_io_glob))
    print("Parquet file counts:", {"io": len(io_paths)}, flush=True)
    print(f"[INFO] Analysis window: {analysis.analysis_start} .. {analysis.analysis_end_exclusive} (exclusive)", flush=True)

    index_start = analysis.analysis_start - timedelta(days=analysis.outpoint_db_lookback_days)
    print(f"[INFO] Outpoint DB preload start: {index_start} .. {analysis.analysis_end_exclusive} (exclusive)", flush=True)
    print(f"[INFO] Outpoint DB path: {paths.outpoint_db_path} (reset={outputs.reset_outpoint_db})", flush=True)

    eligible: list[tuple[str, date]] = []
    for path in io_paths:
        d = extract_day_from_path(path)
        if d is None:
            continue
        if index_start <= d < analysis.analysis_end_exclusive:
            eligible.append((path, d))

    n_preload_total = sum(1 for _p, d in eligible if d < analysis.analysis_start)
    n_analysis_total = len(eligible) - n_preload_total
    print(f"[INFO] Eligible files in window: {len(eligible)}  (preload={n_preload_total}, analysis={n_analysis_total})", flush=True)

    if not eligible:
        print("[WARN] No eligible parquet files found for the requested window.", flush=True)
        return

    # Pick sanity parquet
    sanity_parquet = sanity.prevout_sanity_parquet
    if sanity_parquet is None:
        for p, d in eligible:
            if analysis.analysis_start <= d < analysis.analysis_end_exclusive:
                sanity_parquet = p
                break
        if sanity_parquet is None and eligible:
            sanity_parquet = eligible[0][0]
    if sanity.run_sanity_checks and sanity_parquet:
        print(f"[INFO] Sanity parquet selected: {sanity_parquet}", flush=True)

    conn = open_outpoint_db(paths.outpoint_db_path, reset=outputs.reset_outpoint_db)
    init_lookup_tables(conn)

    conn.execute("BEGIN;")
    pending_outpoint_rows = 0

    uf = UnionFind()
    addr_to_id: dict[str, int] = {}

    # Flags (diagnostic)
    # bit 1: multi-input evidence
    # bit 2: change-output evidence
    # bit 4: change-anchor evidence
    multi_change_flags = bytearray()
    seen_output_flags = bytearray()

    seen_output_addrs: set[str] | None
    if toggles.precreate_nodes_for_all_output_addrs:
        seen_output_addrs = None
    else:
        seen_output_addrs = set()

    def ensure_flag_capacity(idx: int) -> None:
        needed = idx + 1
        cur = len(seen_output_flags)
        if cur < needed:
            delta = needed - cur
            seen_output_flags.extend(b"\x00" * delta)
            multi_change_flags.extend(b"\x00" * delta)

    def get_addr_id(addr: str) -> int:
        idx = addr_to_id.get(addr)
        if idx is None:
            idx = uf.make_set()
            addr_to_id[addr] = idx
            ensure_flag_capacity(idx)
        return idx

    # value unit mode
    if analysis.value_unit_mode == "btc":
        value_unit: str | None = "btc"
    elif analysis.value_unit_mode == "sats":
        value_unit = "sats"
    elif analysis.value_unit_mode == "infer":
        value_unit = None
    else:
        raise ValueError(f"Unknown value_unit_mode={analysis.value_unit_mode}")

    def value_expr_to_sats() -> pl.Expr:
        nonlocal value_unit
        if value_unit is None:
            return (pl.col("value").cast(pl.Float64, strict=False) * 100_000_000).round(0).cast(pl.Int64, strict=False)
        if value_unit == "sats":
            return pl.col("value").cast(pl.Float64, strict=False).round(0).cast(pl.Int64, strict=False)
        return (pl.col("value").cast(pl.Float64, strict=False) * 100_000_000).round(0).cast(pl.Int64, strict=False)

    stats = {
        "n_files_indexed": 0,
        "n_files_analyzed": 0,
        "n_txs_total": 0,
        "n_txs_coinjoin_flagged": 0,
        "n_txs_with_multiinput": 0,
        "n_txs_with_change_detected": 0,
        "n_prevout_lookups": 0,
        "n_prevout_hits": 0,

        "union_attempts": 0,
        "unions_applied": 0,
        "unions_applied_h1": 0,
        "unions_applied_change": 0,

        "skip_same_component": 0,
        "skip_same_component_h1": 0,
        "skip_same_component_change": 0,

        "unions_skipped": 0,
        "unions_skipped_h1": 0,
        "unions_skipped_change": 0,

        "skip_abs_cap": 0,
        "skip_abs_cap_h1": 0,
        "skip_abs_cap_change": 0,

        "skip_ratio_guard": 0,
        "skip_ratio_guard_change": 0,

        # vote-gating
        "skip_constraint": 0,
        "skip_constraint_change": 0,

        # Patch B: degree-guard
        "skip_degree_guard": 0,
        "skip_degree_guard_change": 0,

        "skip_vote_pending": 0,
        "skip_vote_failed": 0,
    }

    # vote-gating store (unordered (root_a, root_b))
    change_merge_votes: dict[tuple[int, int], int] = {}

    # Patch B: constraint diagnostics
    constraints_added = 0
    constraint_gate_evals_total = 0
    constraint_pair_counts: dict[tuple[int, int], int] = {}
    constraint_unique_pairs: set[tuple[int, int]] = set()
    constraint_pairs_repeated = 0

    # ratio-guard sample log state
    ratio_guard_samples_written = 0
    ratio_guard_sample_fh = None
    try:
        if guards.ratio_guard_sample_n > 0:
            ratio_guard_sample_fh = open(guards.ratio_guard_sample_path, "w", encoding="utf-8")
            ratio_guard_sample_fh.write(
                "txid\tsa\tsb\ts_big\ts_small\tratio\tchange_side\tbig_side\tbig_is_change\tchange_comp_size\tinput_comp_size\n"
            )
    except Exception:
        ratio_guard_sample_fh = None

    # constraint log state (Patch B)
    constraint_log_written = 0
    constraint_log_fh = None
    try:
        if guards.constraint_log_n > 0:
            constraint_log_fh = open(guards.constraint_log_path, "w", encoding="utf-8")
            constraint_log_fh.write(
                "txid\tevent\treason\tsa\tsb\tmerged\tmin_side\tbig_side\tratio\t"
                "votes\tvotes_required\tkey_r0\tkey_r1\ttriggered_by\tbig_is_change\t"
                "pair_seen\tis_repeat\tchange_root\tanchor_root\tchange_distinct_large\n"
            )
    except Exception:
        constraint_log_fh = None

    def _votes_required_for_sizes(min_side: int, big_side: int, merged: int) -> int:
        # base requirement
        req = guards.change_votes_required
        # Patch A: ultra tiers (take maximum requirement among triggered thresholds)
        for thr, req_thr in guards.ultra_change_vote_rules:
            if big_side >= thr or merged >= thr:
                if req_thr > req:
                    req = req_thr
        return req

    def _log_constraint_event(
        *,
        txid: str | None,
        event: str,  # "VOTE_PENDING" | "DEGREE_GUARD"
        reason: str,  # "CHANGE" or "H1" (here: only CHANGE)
        sa: int,
        sb: int,
        merged: int,
        min_side: int,
        big_side: int,
        ratio: float,
        votes: int,
        votes_required: int,
        key_r0: int,
        key_r1: int,
        triggered_by: str,
        big_is_change: int,
        pair_seen: int,
        is_repeat: int,
        change_root: int,
        anchor_root: int,
        change_distinct_large: int,
    ) -> None:
        nonlocal constraint_log_written
        if (
            constraint_log_fh is None
            or constraint_log_written >= guards.constraint_log_n
            or txid is None
        ):
            return
        constraint_log_fh.write(
            f"{txid}\t{event}\t{reason}\t{sa}\t{sb}\t{merged}\t{min_side}\t{big_side}\t{ratio}\t"
            f"{votes}\t{votes_required}\t{key_r0}\t{key_r1}\t{triggered_by}\t{big_is_change}\t"
            f"{pair_seen}\t{is_repeat}\t{change_root}\t{anchor_root}\t{change_distinct_large}\n"
        )
        constraint_log_written += 1

    def union_guarded(a: int, b: int, reason: Literal["H1", "CHANGE"], txid: str | None) -> bool:
        nonlocal ratio_guard_samples_written, constraint_log_written
        nonlocal constraints_added, constraint_gate_evals_total, constraint_pairs_repeated

        stats["union_attempts"] += 1
        ra = uf.find(a)
        rb = uf.find(b)
        if ra == rb:
            stats["skip_same_component"] += 1
            if reason == "H1":
                stats["skip_same_component_h1"] += 1
            else:
                stats["skip_same_component_change"] += 1
            return False

        sa = uf.size[ra]
        sb = uf.size[rb]
        merged = sa + sb

        # No guards path
        if not toggles.enable_merge_guards:
            uf.union_roots(ra, rb, change_degree_set_cap=guards.change_degree_set_cap)
            stats["unions_applied"] += 1
            if reason == "H1":
                stats["unions_applied_h1"] += 1
            else:
                stats["unions_applied_change"] += 1
            return True

        # Absolute cap (all unions)
        if merged > guards.max_merged_component_size:
            stats["unions_skipped"] += 1
            if reason == "H1":
                stats["unions_skipped_h1"] += 1
                stats["skip_abs_cap_h1"] += 1
            else:
                stats["unions_skipped_change"] += 1
                stats["skip_abs_cap_change"] += 1
            stats["skip_abs_cap"] += 1
            return False

        # Ratio guard ONLY for CHANGE (with small-floor)
        if guards.merge_ratio_guard and reason == "CHANGE":
            s_big = sa if sa >= sb else sb
            s_small = sb if sa >= sb else sa

            if s_small >= guards.merge_ratio_small_floor and s_big >= guards.merge_ratio_big_cluster_min:
                ratio = (s_big / s_small) if s_small > 0 else float("inf")
                if ratio > guards.merge_ratio_max:
                    stats["unions_skipped"] += 1
                    stats["unions_skipped_change"] += 1
                    stats["skip_ratio_guard"] += 1
                    stats["skip_ratio_guard_change"] += 1

                    # Sample log (assumes CHANGE call is (anchor=a, change=b))
                    if (
                        ratio_guard_sample_fh is not None
                        and ratio_guard_samples_written < guards.ratio_guard_sample_n
                        and txid is not None
                    ):
                        big_root = ra if sa >= sb else rb
                        big_is_change = 1 if big_root == rb else 0
                        big_side = "b" if big_is_change else "a"

                        ratio_guard_sample_fh.write(
                            f"{txid}\t{sa}\t{sb}\t{s_big}\t{s_small}\t{ratio}\t"
                            f"b\t{big_side}\t{big_is_change}\t{sb}\t{sa}\n"
                        )
                        ratio_guard_samples_written += 1
                    return False

        # Patch B: change-degree guard (CHANGE-only)
        if guards.enable_change_degree_guard and reason == "CHANGE":
            # Convention in pipeline: a=anchor, b=change
            anchor_root = ra
            change_root = rb
            anchor_size = sa
            change_size = sb

            change_distinct_large = 0
            if (anchor_size >= guards.change_degree_large_min) and (change_size >= guards.change_degree_small_min):
                s = uf.change_large_anchor_roots.get(change_root)
                if s is None:
                    s = set()
                    uf.change_large_anchor_roots[change_root] = s
                s.add(anchor_root)
                if len(s) > guards.change_degree_set_cap:
                    uf.change_large_anchor_roots[change_root] = set(sorted(s)[:guards.change_degree_set_cap])
                    s = uf.change_large_anchor_roots[change_root]
                change_distinct_large = len(s)

                if change_distinct_large > guards.change_degree_max_distinct_large:
                    stats["unions_skipped"] += 1
                    stats["unions_skipped_change"] += 1
                    stats["skip_degree_guard"] += 1
                    stats["skip_degree_guard_change"] += 1

                    s_big = max(sa, sb)
                    s_small = min(sa, sb)
                    ratio = (s_big / s_small) if s_small > 0 else float("inf")
                    big_root = ra if sa >= sb else rb
                    big_is_change = 1 if big_root == rb else 0
                    r0, r1 = (anchor_root, change_root) if anchor_root < change_root else (change_root, anchor_root)

                    _log_constraint_event(
                        txid=txid,
                        event="DEGREE_GUARD",
                        reason=reason,
                        sa=sa, sb=sb,
                        merged=merged,
                        min_side=min(sa, sb),
                        big_side=max(sa, sb),
                        ratio=ratio,
                        votes=0,
                        votes_required=0,
                        key_r0=r0,
                        key_r1=r1,
                        triggered_by="degree_guard",
                        big_is_change=big_is_change,
                        pair_seen=0,
                        is_repeat=0,
                        change_root=change_root,
                        anchor_root=anchor_root,
                        change_distinct_large=change_distinct_large,
                    )
                    return False

        # Big–big vote-gating ONLY for CHANGE
        if guards.enable_change_merge_votes and reason == "CHANGE":
            min_side = sa if sa <= sb else sb
            big_side = sa if sa >= sb else sb

            triggered = False
            triggered_by = "none"

            if guards.change_vote_min_side > 0 and min_side >= guards.change_vote_min_side:
                triggered = True
                triggered_by = "min_side"
            if (not triggered) and guards.change_vote_merged_trigger > 0 and merged >= guards.change_vote_merged_trigger:
                triggered = True
                triggered_by = "merged"

            if triggered:
                r0, r1 = (ra, rb) if ra < rb else (rb, ra)
                k = (r0, r1)

                constraint_gate_evals_total += 1
                prev_seen = constraint_pair_counts.get(k, 0)
                pair_seen = prev_seen + 1
                constraint_pair_counts[k] = pair_seen
                if pair_seen == 1:
                    constraints_added += 1
                    constraint_unique_pairs.add(k)
                elif pair_seen == 2:
                    constraint_pairs_repeated += 1
                is_repeat = 1 if pair_seen > 1 else 0

                prev_votes = change_merge_votes.get(k, 0)
                votes = prev_votes + 1
                change_merge_votes[k] = votes

                votes_required = _votes_required_for_sizes(min_side=min_side, big_side=big_side, merged=merged)

                if votes < votes_required:
                    stats["unions_skipped"] += 1
                    stats["unions_skipped_change"] += 1
                    stats["skip_constraint"] += 1
                    stats["skip_constraint_change"] += 1
                    stats["skip_vote_pending"] += 1

                    s_big = max(sa, sb)
                    s_small = min(sa, sb)
                    ratio = (s_big / s_small) if s_small > 0 else float("inf")
                    big_root = ra if sa >= sb else rb
                    big_is_change = 1 if big_root == rb else 0

                    _log_constraint_event(
                        txid=txid,
                        event="VOTE_PENDING",
                        reason=reason,
                        sa=sa, sb=sb,
                        merged=merged,
                        min_side=min_side,
                        big_side=big_side,
                        ratio=ratio,
                        votes=votes,
                        votes_required=votes_required,
                        key_r0=r0,
                        key_r1=r1,
                        triggered_by=triggered_by,
                        big_is_change=big_is_change,
                        pair_seen=pair_seen,
                        is_repeat=is_repeat,
                        change_root=rb,
                        anchor_root=ra,
                        change_distinct_large=0,
                    )
                    return False

        # Debug log of large CHANGE merges that pass all guards (optional)
        if guards.debug_large_change_merges and reason == "CHANGE" and merged >= guards.debug_change_merge_min and txid is not None:
            try:
                with open(guards.debug_change_merge_log_path, "a", encoding="utf-8") as f:
                    f.write(f"{txid}\t{sa}\t{sb}\t{merged}\n")
            except Exception:
                pass

        uf.union_roots(ra, rb, change_degree_set_cap=guards.change_degree_set_cap)
        stats["unions_applied"] += 1
        if reason == "H1":
            stats["unions_applied_h1"] += 1
        else:
            stats["unions_applied_change"] += 1
        return True

    # tqdm: force stdout
    pbar = tqdm(
        eligible,
        total=len(eligible),
        unit="file",
        dynamic_ncols=True,
        mininterval=perf.tqdm_mininterval,
        miniters=perf.tqdm_miniters,
        smoothing=0,
        desc="Processing parquet",
        file=sys.stdout,
        leave=True,
    )

    DIR_NEEDS_NORMALIZATION: bool | None = None

    def log(msg: str) -> None:
        tqdm.write(msg, file=sys.stdout)

    def mark_outputs_seen(outputs_list: list[tuple[str | None, int]]) -> None:
        if toggles.precreate_nodes_for_all_output_addrs:
            for a, _v in outputs_list:
                if a is None:
                    continue
                a_id = get_addr_id(a)
                seen_output_flags[a_id] = 1
        else:
            assert seen_output_addrs is not None
            for a, _v in outputs_list:
                if a is not None:
                    seen_output_addrs.add(a)

    for i, (path, file_day) in enumerate(pbar, start=1):
        in_analysis = (analysis.analysis_start <= file_day < analysis.analysis_end_exclusive)

        if (i % perf.tqdm_postfix_every) == 0:
            pbar.set_postfix_str(f"{'analysis' if in_analysis else 'preload'} day={file_day}")

        if in_analysis:
            stats["n_files_analyzed"] += 1
            if stats["n_files_analyzed"] <= 5:
                log(f"[{stats['n_files_analyzed']}] Processing {path} (day={file_day}) ...")
            elif (stats["n_files_analyzed"] % 250) == 0:
                log(f"[{stats['n_files_analyzed']}] Processing ... (day={file_day})")
        else:
            stats["n_files_indexed"] += 1
            if stats["n_files_indexed"] <= 3:
                log(f"[preload {stats['n_files_indexed']}] Indexing outputs only: {path} (day={file_day}) ...")

        df = pl.read_parquet(path, columns=["dir", "txid", "n", "prev_txid", "prev_vout", "address", "value"])

        if DIR_NEEDS_NORMALIZATION is None:
            dir_uniques = df.select(pl.col("dir").cast(pl.Utf8, strict=False).unique()).to_series().to_list()
            DIR_NEEDS_NORMALIZATION = any(
                (d is not None and str(d).lower() in ("in", "out") and str(d) not in ("in", "out")) for d in dir_uniques
            )
            if in_analysis and stats["n_files_analyzed"] == 1:
                log(f"  [debug] distinct dir values in first processed file: {dir_uniques}")

        if DIR_NEEDS_NORMALIZATION:
            df = df.with_columns(pl.col("dir").cast(pl.Utf8).str.to_lowercase().alias("dir"))

        # Infer unit once if needed
        if value_unit is None:
            sample = (
                df.filter(pl.col("dir") == "out")
                .select(pl.col("value").cast(pl.Float64, strict=False))
                .drop_nulls()
                .head(5000)
                .to_series()
                .to_list()
            )
            value_unit = infer_value_unit_from_sample(sample)
            log(f"[INFO] Detected value unit: {value_unit}  (all internal amount logic uses satoshis)")

        # 1) Index outputs into outpoints DB
        out_all = (
            df.filter(pl.col("dir") == "out")
            .filter(pl.col("txid").is_not_null())
            .filter(pl.col("n").is_not_null())
            .select(
                pl.col("txid"),
                pl.col("n").cast(pl.Int64, strict=False).alias("n"),
                pl.col("address"),
                value_expr_to_sats().alias("value_sats"),
            )
        )

        out_for_db = out_all.filter(pl.col("address").is_not_null()).filter(pl.col("value_sats").is_not_null())
        if out_for_db.height > 0:
            rows = [
                (txid, int(n), addr, int(vs))
                for txid, n, addr, vs in out_for_db.select(["txid", "n", "address", "value_sats"]).iter_rows(
                    named=False, buffer_size=perf.iter_buffer_out
                )
            ]
            insert_outpoints_no_commit(conn, rows)
            pending_outpoint_rows += len(rows)

            if pending_outpoint_rows >= perf.outpoint_commit_every_rows:
                conn.execute("COMMIT;")
                conn.execute("BEGIN;")
                pending_outpoint_rows = 0

            if toggles.precreate_nodes_for_all_output_addrs:
                # Determinism: sort unique output addrs before node creation
                out_u = out_for_db.select(pl.col("address").unique()).to_series().to_list()
                for addr in sorted(out_u):
                    _ = get_addr_id(addr)

        if not in_analysis:
            del df, out_all, out_for_db
            if (i % perf.gc_every_n_files) == 0:
                gc.collect()
            continue

        # 2) Build VIN (spend_txid, prev_txid, prev_n)
        in_df = (
            df.filter(pl.col("dir") == "in")
            .filter(pl.col("txid").is_not_null())
            .filter(pl.col("prev_txid").is_not_null())
            .filter(pl.col("prev_vout").is_not_null())
            .select(
                pl.col("txid").alias("spend_txid"),
                pl.col("prev_txid"),
                pl.col("prev_vout").cast(pl.Int64, strict=False).alias("prev_n"),
            )
            .filter(pl.col("prev_n").is_not_null())
        )

        del df
        if (i % perf.gc_every_n_files) == 0:
            gc.collect()

        if in_df.height == 0:
            del in_df
            del out_all, out_for_db
            if (i % perf.gc_every_n_files) == 0:
                gc.collect()
            continue

        # n_in_utxos per spend tx
        in_counts = (
            in_df.group_by("spend_txid")
            .len()
            .rename({"spend_txid": "txid", "len": "n_in_utxos"})
        )

        # 2a) Resolve prevouts via SQLite join + SQLite aggregates
        if not perf.use_sql_prevout_join:
            raise RuntimeError("perf.use_sql_prevout_join=False is not supported in this build; set it True.")

        reset_vinbuf(conn)

        cur = conn.cursor()
        insert_sql = "INSERT INTO vinbuf(spend_txid, prev_txid, prev_n) VALUES (?, ?, ?);"
        batch: list[tuple[str, str, int]] = []
        total_vin = 0

        for spend_txid, prev_txid, prev_n in in_df.select(["spend_txid", "prev_txid", "prev_n"]).iter_rows(
            named=False, buffer_size=perf.iter_buffer_in
        ):
            batch.append((str(spend_txid), str(prev_txid), int(prev_n)))
            if len(batch) >= perf.vinbuf_insert_chunk:
                cur.executemany(insert_sql, batch)
                total_vin += len(batch)
                batch.clear()

        if batch:
            cur.executemany(insert_sql, batch)
            total_vin += len(batch)
            batch.clear()

        stats["n_prevout_lookups"] += total_vin

        resolved_aggs = fetch_prevouts_aggs_polars(conn, fetch_chunk=perf.vinbuf_fetch_chunk)

        if resolved_aggs.height > 0:
            resolved_inputs_sum = resolved_aggs.select(pl.col("resolved_cnt").sum()).item()
            stats["n_prevout_hits"] += int(resolved_inputs_sum) if resolved_inputs_sum is not None else 0

        del in_df

        # Node universe patch: create nodes for ALL resolved input addresses
        if toggles.create_nodes_for_all_resolved_inputs and resolved_aggs.height > 0:
            u = (
                resolved_aggs
                .select(pl.col("in_addrs").explode().drop_nulls().unique())
                .to_series()
                .to_list()
            )
            for a in sorted(u):
                _ = get_addr_id(a)

        # 3) Outputs grouped per tx
        if out_all.height == 0:
            del out_all, out_for_db, in_counts, resolved_aggs
            if (i % perf.gc_every_n_files) == 0:
                gc.collect()
            continue

        vout_grouped = (
            out_all.group_by("txid")
            .agg(
                pl.col("address").alias("out_addrs_all"),
                pl.col("value_sats").alias("out_values_sats_all"),
            )
        )
        del out_all, out_for_db

        # Join tx-level inputs + resolved stats to outputs
        tx_joined = (
            vout_grouped
            .join(in_counts, on="txid", how="inner")
            .join(resolved_aggs, on="txid", how="left")
        )
        del vout_grouped, in_counts, resolved_aggs

        for (
            txid,
            out_addrs_all,
            out_values_all,
            n_in_utxos,
            resolved_cnt,
            sum_inputs_sats,
            min_input_sats,
            in_addrs,
        ) in tx_joined.select(
            ["txid", "out_addrs_all", "out_values_sats_all", "n_in_utxos", "resolved_cnt", "sum_inputs_sats", "min_input_sats", "in_addrs"]
        ).iter_rows(named=False, buffer_size=perf.iter_buffer_grouped):

            if not out_addrs_all or not out_values_all:
                continue

            n_in_utxos_i = int(n_in_utxos) if n_in_utxos is not None else 0
            if n_in_utxos_i == 0:
                continue

            stats["n_txs_total"] += 1

            outputs_list: list[tuple[str | None, int]] = []
            sum_outputs_sats = 0
            for a, v in zip(out_addrs_all, out_values_all):
                if v is None:
                    continue
                vi = int(v)
                outputs_list.append((a, vi))
                sum_outputs_sats += vi

            spendable = [(a, v) for (a, v) in outputs_list if a is not None and v >= params.dust_sats]
            if not spendable:
                mark_outputs_seen(outputs_list)
                continue

            # Mixing-like filter
            is_mixing_like = False
            if toggles.enable_coinjoin_filter:
                is_mixing_like = detect_mixing_like(n_in_utxos_i, spendable, dust_sats=params.dust_sats)
                if is_mixing_like:
                    stats["n_txs_coinjoin_flagged"] += 1
                    mark_outputs_seen(outputs_list)
                    continue

            if in_addrs is None:
                mark_outputs_seen(outputs_list)
                continue

            in_addrs_list = list(in_addrs)  # already list-sorted in fetch_prevouts_aggs_polars()
            if not in_addrs_list:
                mark_outputs_seen(outputs_list)
                continue

            # H1 multi-input unions
            if toggles.enable_multi_input:
                safe = multi_input_is_safe(params.multi_input_policy, n_in_utxos_i, len(spendable), is_mixing_like)
                if safe:
                    stats["n_txs_with_multiinput"] += 1
                    if len(in_addrs_list) >= 2:
                        in_ids = [get_addr_id(a) for a in in_addrs_list]
                        for idx in in_ids:
                            multi_change_flags[idx] |= 1
                        first_id = in_ids[0]
                        for idx in in_ids[1:]:
                            union_guarded(first_id, idx, reason="H1", txid=str(txid))
                    else:
                        idx = get_addr_id(in_addrs_list[0])
                        multi_change_flags[idx] |= 1

            # CHANGE unions
            if toggles.enable_change:
                n_spendable_out = len(spendable)

                if params.change_require_2_outputs and n_spendable_out != 2:
                    mark_outputs_seen(outputs_list)
                    continue
                if n_in_utxos_i > params.max_change_inputs_utxos:
                    mark_outputs_seen(outputs_list)
                    continue
                if n_spendable_out > params.max_change_spendable_outs:
                    mark_outputs_seen(outputs_list)
                    continue

                resolved_cnt_i = int(resolved_cnt) if resolved_cnt is not None else 0
                if resolved_cnt_i != n_in_utxos_i:
                    mark_outputs_seen(outputs_list)
                    continue

                in_types = [addr_type(a) for a in in_addrs_list]
                if len(set(in_types)) != 1:
                    mark_outputs_seen(outputs_list)
                    continue
                majority_type = in_types[0]

                spendable_addrs = [a for (a, _v) in spendable]
                if len(spendable_addrs) != len(set(spendable_addrs)):
                    mark_outputs_seen(outputs_list)
                    continue

                in_addr_set_fast = set(in_addrs_list)
                candidates: list[tuple[str, int]] = []
                for a, v in spendable:
                    if a in in_addr_set_fast:
                        continue
                    if addr_type(a) != majority_type:
                        continue
                    if toggles.precreate_nodes_for_all_output_addrs:
                        a_id = get_addr_id(a)
                        if seen_output_flags[a_id]:
                            continue
                    else:
                        assert seen_output_addrs is not None
                        if a in seen_output_addrs:
                            continue
                    candidates.append((a, v))

                if len(candidates) != 1:
                    mark_outputs_seen(outputs_list)
                    continue

                change_addr, change_val = candidates[0]

                sum_in_i = int(sum_inputs_sats) if sum_inputs_sats is not None else 0
                fee = sum_in_i - sum_outputs_sats
                if sum_in_i <= 0 or fee < 0:
                    mark_outputs_seen(outputs_list)
                    continue
                if fee > params.max_fee_abs_sats or fee > int(params.max_fee_frac * sum_in_i):
                    mark_outputs_seen(outputs_list)
                    continue

                spend_vals = [v for (_a, v) in spendable]
                if n_spendable_out == 2:
                    other_val = max(v for (_a, v) in spendable if _a != change_addr)
                    if not (change_val < other_val):
                        mark_outputs_seen(outputs_list)
                        continue
                elif n_spendable_out == 3:
                    if change_val != min(spend_vals):
                        mark_outputs_seen(outputs_list)
                        continue
                else:
                    mark_outputs_seen(outputs_list)
                    continue

                if params.enable_optimal_change_constraint:
                    mn = int(min_input_sats) if min_input_sats is not None else None
                    if mn is None:
                        mark_outputs_seen(outputs_list)
                        continue
                    if change_val > (mn - fee + params.optimal_change_slack_sats):
                        mark_outputs_seen(outputs_list)
                        continue

                change_id = get_addr_id(change_addr)
                multi_change_flags[change_id] |= 2

                anchor_id = get_addr_id(in_addrs_list[0])
                multi_change_flags[anchor_id] |= 4

                stats["n_txs_with_change_detected"] += 1
                union_guarded(anchor_id, change_id, reason="CHANGE", txid=str(txid))

            mark_outputs_seen(outputs_list)

        del tx_joined
        if (i % perf.gc_every_n_files) == 0:
            gc.collect()

    pbar.close()
    conn.execute("COMMIT;")

    # Close diagnostic file handles
    try:
        if ratio_guard_sample_fh is not None:
            ratio_guard_sample_fh.close()
            log(f"[DIAG] Wrote ratio-guard sample log: {guards.ratio_guard_sample_path}  (n={ratio_guard_samples_written})")
    except Exception:
        pass
    try:
        if constraint_log_fh is not None:
            constraint_log_fh.close()
            log(f"[DIAG] Wrote constraint-event log: {guards.constraint_log_path}  (n={constraint_log_written})")
    except Exception:
        pass

    # Finalize entity mapping (compute only), then report via print_final_report()
    n_nodes = len(addr_to_id)
    if n_nodes == 0:
        print("No union-find nodes created.", flush=True)
        conn.close()
        return

    node_to_entity = np.empty(n_nodes, dtype=np.int32)
    root_to_entity: dict[int, int] = {}
    next_entity_id = 0
    for node in range(n_nodes):
        root = uf.find(node)
        ent = root_to_entity.get(root)
        if ent is None:
            ent = next_entity_id
            root_to_entity[root] = ent
            next_entity_id += 1
        node_to_entity[node] = ent

    n_entities = next_entity_id
    counts = np.bincount(node_to_entity, minlength=n_entities).astype(np.int64)
    cluster_sizes = counts[counts > 0]

    _ = print_final_report(
        n_nodes=n_nodes,
        n_entities=n_entities,
        node_to_entity=node_to_entity,
        counts=counts,
        cluster_sizes=cluster_sizes,
        stats=stats,
        multi_change_flags=multi_change_flags,
        outputs=outputs,
        guards=guards,
        enable_constraints_diag=(guards.enable_change_merge_votes or guards.enable_change_degree_guard),
        constraint_unique_pairs=constraint_unique_pairs,
        constraint_gate_evals_total=constraint_gate_evals_total,
        constraint_pairs_repeated=constraint_pairs_repeated,
        change_merge_votes=change_merge_votes,
        prevout_lookups=stats["n_prevout_lookups"],
        prevout_hits=stats["n_prevout_hits"],
        run_sanity=sanity.run_sanity_checks,
        sanity_top_k=20,
        run_prevout_db_sanity=(sanity.run_prevout_db_sanity and sanity_parquet is not None),
        conn=conn,
        sanity_parquet=sanity_parquet,
        prevout_sanity_sample_limit=sanity.prevout_sanity_sample_limit,
        prevout_hybrid_threshold=perf.prevout_hybrid_threshold,
        prevout_lookup_chunk=perf.prevout_lookup_chunk,
    )

    # Write mapping (address -> entity_id)
    if outputs.write_entity_mapping:
        print(f"\n[WRITE] Writing entity mapping to: {paths.entity_map_out_path}", flush=True)
        if paths.entity_map_out_path.exists():
            paths.entity_map_out_path.unlink()

        writer: pq.ParquetWriter | None = None
        batch_addrs: list[str] = []
        batch_eids: list[int] = []
        written = 0

        for addr, node_id in addr_to_id.items():
            batch_addrs.append(addr)
            batch_eids.append(int(node_to_entity[node_id]))

            if len(batch_addrs) >= outputs.entity_write_batch:
                table = pa.table({"address": batch_addrs, "entity_id": batch_eids})
                if writer is None:
                    writer = pq.ParquetWriter(
                        str(paths.entity_map_out_path),
                        table.schema,
                        compression=outputs.entity_map_compression,
                        use_dictionary=True,
                    )
                writer.write_table(table)
                written += len(batch_addrs)
                batch_addrs.clear()
                batch_eids.clear()
                print(f"  [WRITE] rows written: {written:,}", flush=True)

        if batch_addrs:
            table = pa.table({"address": batch_addrs, "entity_id": batch_eids})
            if writer is None:
                writer = pq.ParquetWriter(
                    str(paths.entity_map_out_path),
                    table.schema,
                    compression=outputs.entity_map_compression,
                    use_dictionary=True,
                )
            writer.write_table(table)
            written += len(batch_addrs)
            print(f"  [WRITE] rows written: {written:,}", flush=True)

        if writer is not None:
            writer.close()

        print("[WRITE] Done.", flush=True)

    # Confidence proxy output (optional)
    if confidence.enable_confidence_proxy:
        print(f"\n[CONF] Writing confidence proxy to: {confidence.confidence_out_path}", flush=True)
        if confidence.confidence_out_path.exists():
            confidence.confidence_out_path.unlink()

        def proxy_prob(node_id: int) -> float:
            eid = int(node_to_entity[node_id])
            sz = int(counts[eid])
            if sz <= 1:
                return float(confidence.p_singleton)

            bits = int(multi_change_flags[node_id])
            e = 0
            if bits & 1:
                e += 1
            if bits & 2:
                e += 1
            if bits & 4:
                e += 1

            if e <= 0:
                base = float(confidence.p_base_0)
            elif e == 1:
                base = float(confidence.p_base_1)
            else:
                base = float(confidence.p_base_2plus)

            denom = math.log1p(max(1, confidence.size_norm))
            frac = (math.log1p(sz) / denom) if denom > 0 else 1.0
            frac = min(1.0, max(0.0, frac))
            p = base + float(confidence.size_bonus_max) * frac
            return float(min(0.999, max(0.001, p)))

        writer2: pq.ParquetWriter | None = None
        b_addr: list[str] = []
        b_eid: list[int] = []
        b_p: list[float] = []
        b_sz: list[int] = []
        b_bits: list[int] = []
        written2 = 0

        for addr, node_id in addr_to_id.items():
            eid = int(node_to_entity[node_id])
            b_addr.append(addr)
            b_eid.append(eid)
            b_p.append(proxy_prob(node_id))
            if confidence.include_cluster_size:
                b_sz.append(int(counts[eid]))
            if confidence.include_evidence_bits:
                b_bits.append(int(multi_change_flags[node_id]))

            if len(b_addr) >= confidence.confidence_write_batch:
                data = {"address": b_addr, "entity_id": b_eid, "p_clustered_proxy": b_p}
                if confidence.include_cluster_size:
                    data["cluster_size"] = b_sz
                if confidence.include_evidence_bits:
                    data["evidence_bits"] = b_bits
                table = pa.table(data)
                if writer2 is None:
                    writer2 = pq.ParquetWriter(
                        str(confidence.confidence_out_path),
                        table.schema,
                        compression=confidence.confidence_compression,
                        use_dictionary=True,
                    )
                writer2.write_table(table)
                written2 += len(b_addr)
                b_addr.clear()
                b_eid.clear()
                b_p.clear()
                b_sz.clear()
                b_bits.clear()
                print(f"  [CONF] rows written: {written2:,}", flush=True)

        if b_addr:
            data = {"address": b_addr, "entity_id": b_eid, "p_clustered_proxy": b_p}
            if confidence.include_cluster_size:
                data["cluster_size"] = b_sz
            if confidence.include_evidence_bits:
                data["evidence_bits"] = b_bits
            table = pa.table(data)
            if writer2 is None:
                writer2 = pq.ParquetWriter(
                    str(confidence.confidence_out_path),
                    table.schema,
                    compression=confidence.confidence_compression,
                    use_dictionary=True,
                )
            writer2.write_table(table)
            written2 += len(b_addr)
            print(f"  [CONF] rows written: {written2:,}", flush=True)

        if writer2 is not None:
            writer2.close()

        print("[CONF] Done.", flush=True)

    # Plots
    if plots.enable_plots:
        print("\n[PLOT] Generating plots...", flush=True)

        largest = int(cluster_sizes.max()) if cluster_sizes.size else 0

        plot_hist_logbins(
            sizes=cluster_sizes,
            title=f"Entity Cluster Size Distribution ({analysis.analysis_start.year}) — log bins (all clusters)",
            color="tab:blue",
            bins=plots.log_bins,
            max_x=None,
        )

        sizes_excl = cluster_sizes[cluster_sizes != largest] if largest > 0 else cluster_sizes
        plot_hist_logbins(
            sizes=sizes_excl,
            title=f"Entity Cluster Size Distribution ({analysis.analysis_start.year}) — excluding largest cluster",
            color="tab:orange",
            bins=plots.log_bins,
            max_x=None,
        )

        sizes_focus = cluster_sizes[cluster_sizes <= plots.focus_max_size]
        plot_hist_logbins(
            sizes=sizes_focus,
            title=f"Entity Cluster Sizes ≤ {plots.focus_max_size} ({analysis.analysis_start.year}) — log bins",
            color="tab:green",
            bins=plots.log_bins,
            max_x=plots.focus_max_size,
        )

        plot_zipf(
            sizes=cluster_sizes,
            title=f"Rank–Size (Zipf) Plot ({analysis.analysis_start.year}) — top {min(plots.zipf_top_k, cluster_sizes.size):,} clusters",
            color="tab:red",
            top_k=plots.zipf_top_k,
        )

    conn.close()
    print("[INFO] Done.", flush=True)


if __name__ == "__main__":
    # --- Paths (explicit) ---
    outputs_dir = Path("/media/vatereal/Main/outputs")
    analysis_year = 2014


    paths = PathsConfig(
        parquet_io_glob=str(Path("/media/vatereal/Main/parquet") / "io/day=*/io-*.parquet"),
        outputs_dir=outputs_dir,
        outpoint_db_path=outputs_dir / f"outpoints_{analysis_year}.sqlite",
        entity_map_out_path=outputs_dir / f"entities_multiinput_change_{analysis_year}.parquet",
    )

    # --- Analysis window (explicit) ---
    analysis = AnalysisConfig(
        analysis_start=date(analysis_year, 1, 1),
        analysis_end_exclusive=date(analysis_year + 1, 1, 1),
        outpoint_db_lookback_days=365,
        value_unit_mode="infer",
    )

    # --- Output behavior (explicit) ---
    outputs = OutputConfig(
        reset_outpoint_db=True,
        write_entity_mapping=True,
        entity_map_compression="zstd",
        entity_write_batch=1_000_000,
        top_k_clusters_print=20,
    )

    # --- Plot behavior (explicit) ---
    plots = PlotConfig(
        enable_plots=True,
        zipf_top_k=200_000,
        focus_max_size=2048,
        log_bins=40,
    )

    # --- Performance knobs (explicit) ---
    perf = PerfConfig(
        outpoint_commit_every_rows=500_000,
        prevout_hybrid_threshold=5_000,
        prevout_lookup_chunk=50_000,
        iter_buffer_in=200_000,
        iter_buffer_out=200_000,
        iter_buffer_grouped=50_000,
        gc_every_n_files=100,
        tqdm_mininterval=2.0,
        tqdm_miniters=50,
        tqdm_postfix_every=250,

        # Patch 3 settings (required):
        use_sql_prevout_join=True,
        vinbuf_insert_chunk=1_000_000,
        vinbuf_fetch_chunk=1_000_000,
    )

    # --- Sanity checks (explicit) ---
    sanity = SanityConfig(
        run_sanity_checks=True,
        run_prevout_db_sanity=True,
        prevout_sanity_sample_limit=200_000,
        prevout_sanity_parquet=None,
    )

    # --- Heuristic toggles (explicit) ---
    toggles = HeuristicToggles(
        enable_coinjoin_filter=True,
        enable_multi_input=True,
        enable_change=True,
        enable_merge_guards=True,
        precreate_nodes_for_all_output_addrs=False,
        create_nodes_for_all_resolved_inputs=True,
    )

    # --- Heuristic params (explicit) ---
    params = HeuristicParams(
        dust_sats=546,
        max_fee_abs_sats=50_000_000,
        max_fee_frac=0.05,
        max_change_inputs_utxos=10,
        max_change_spendable_outs=2,
        change_require_2_outputs=True,
        multi_input_policy="one_output",
        enable_optimal_change_constraint=True,
        optimal_change_slack_sats=0,
    )

    # --- Merge guards (explicit) ---
    guards = MergeGuardParams(
        # Patch A: allow mega-entities (still a ceiling/fuse; tune as desired)
        max_merged_component_size=10_000_000,

        # Ratio guard ignores tiny components
        merge_ratio_guard=True,
        merge_ratio_max=200.0,
        merge_ratio_big_cluster_min=50_000,
        merge_ratio_small_floor=50,

        # Big–big gating (CHANGE only): min_side is "both sides >= this"
        enable_change_merge_votes=True,
        change_vote_min_side=25_000,
        change_vote_merged_trigger=0,
        change_votes_required=2,

        # Patch A: stricter votes above thresholds (250k/500k/1M)
        ultra_change_vote_rules=((250_000, 3), (500_000, 4), (1_000_000, 5)),

        # Patch B: degree guard (alternative confirmation signal)
        enable_change_degree_guard=True,
        change_degree_large_min=25_000,
        change_degree_small_min=50,
        change_degree_max_distinct_large=3,
        change_degree_set_cap=64,

        # Ratio-guard sample log
        ratio_guard_sample_n=1000,
        ratio_guard_sample_path=outputs_dir / f"ratio_guard_samples_{analysis_year}.tsv",

        # Patch B: constraint event log (vote pending + degree guard)
        constraint_log_n=200_000,
        constraint_log_path=outputs_dir / f"constraint_events_{analysis_year}.tsv",

        debug_large_change_merges=False,
        debug_change_merge_min=250_000,
        debug_change_merge_log_path=outputs_dir / f"large_change_merges_{analysis_year}.tsv",
    )

    # --- Confidence proxy output (explicit) ---
    confidence = ConfidenceConfig(
        enable_confidence_proxy=True,
        confidence_out_path=outputs_dir / f"address_confidence_{analysis_year}.parquet",
        confidence_compression="zstd",
        confidence_write_batch=1_000_000,

        size_norm=100_000,
        size_bonus_max=0.15,
        p_singleton=0.02,
        p_base_0=0.25,
        p_base_1=0.65,
        p_base_2plus=0.85,

        include_cluster_size=True,
        include_evidence_bits=True,
    )

    run_entity_clustering(
        paths=paths,
        analysis=analysis,
        outputs=outputs,
        plots=plots,
        perf=perf,
        sanity=sanity,
        toggles=toggles,
        params=params,
        guards=guards,
        confidence=confidence,
        determinism_seed=1337,
    )

## Logistic change model: implementation + feature set

### Objective
The pipeline uses a **binary classifier** to decide, for a given transaction, **which candidate output is the change output** (if any). It replaces brittle hard rules with:

- a **feature extractor** (transaction → feature vector),
- a **logistic regression scorer** (features → probability),
- a **decision policy** (probability thresholds + abstention).

The output of this decision drives **CHANGE unions** in the union-find clustering.

---

## Where the model is used in the pipeline

For each transaction eligible for change detection:

1. **Candidate generation**  
   Restrict to spendable outputs that:
   - are not one of the input addresses,
   - have output address type matching the (uniform) input type,
   - satisfy the “2-output” structure (if enabled),
   - are distinct and non-dust.

2. **Score candidates**  
   For each candidate output $c$, compute features $x(c)$, then score with logit:
   
   $$p(c) = \sigma\left(b + \sum_i w_i x_i(c)\right)$$
   
   where $\sigma(z) = \frac{1}{1+e^{-z}}$.

3. **Select best + abstain if ambiguous**  
   Let $p_{best}$ be max probability and $p_{2nd}$ runner-up.
   - If $p_{best} - p_{2nd} < \text{min\_p\_gap}$, abstain (no change union).

4. **Apply dynamic acceptance threshold**  
   The threshold increases with the current **anchor cluster size**:
   
   $$p_{\text{accept}} = \text{clamp}\left(p_0 + \alpha \log(1 + S_{anchor}),\ p_{\min},\ p_{\max}\right)$$
   
   If $p_{best} < p_{\text{accept}}$, abstain.

5. **If accepted: perform CHANGE union**  
   - Mark evidence bits:
     - change output: bit2
     - change anchor: bit4
   - Union(anchor\_addr, change\_addr) with CHANGE reason (subject to guards).

---

## Logistic regression implementation details

### Scoring function
The model computes a linear score:

$$z = \text{intercept} + \sum_i w_i x_i$$

Then converts to probability with a numerically stable sigmoid:

- If $z \ge 0$: $\sigma(z)=\frac{1}{1+e^{-z}}$
- Else: $\sigma(z)=\frac{e^z}{1+e^z}$

This avoids overflow for large $|z|$.

### Missing weights
If a feature exists in the extracted vector but has no coefficient entry, it contributes **0**.

---

## Feature extraction

Each candidate output gets its own feature vector, derived from transaction context + candidate-vs-other comparisons.

### Core structural features

| Feature name | Type | Meaning |
|---|---:|---|
| `n_in` | float | Number of input UTXOs spent by the tx |
| `n_out` | float | Number of spendable outputs considered |
| `in_type_uniform` | {0,1} | All input address types are identical |
| `out_type_match_in` | {0,1} | Candidate output type matches input type |
| `is_mixing_like` | {0,1} | Mixing-like pattern detected (penalized) |

> Note: The pipeline *already* tends to filter out some non-uniform / mixing-like cases before scoring; but the feature is kept so the model can encode this as a strong negative signal if such cases slip through.

---

### Fee / conservation features

| Feature name | Type | Meaning |
|---|---:|---|
| `fee_frac` | float | $\frac{\text{fee}}{\text{sum\_inputs}}$ |
| `log_fee` | float | normalized $\log(1+\text{fee})$ |
| `sum_in_gt_sum_out` | {0,1} | sanity indicator (inputs exceed outputs) |

**Normalization used**  
To keep magnitudes stable across time/scale:

$$\text{log\_fee} = \frac{\log(1+\text{fee})}{\log(1+\text{fee\_norm})}$$

with `fee_norm = cfg.log1p_fee_norm` (example: \(10^6\)).

---

### Candidate-specific features

| Feature name | Type | Meaning |
|---|---:|---|
| `cand_new` | {0,1} | candidate address has not been seen as an output before (“fresh”) |
| `cand_is_min_out` | {0,1} | candidate value is the smaller of the two |
| `cand_lt_other` | {0,1} | candidate value strictly less than the other output |
| `log_cand` | float | normalized \( \log(1+\text{candidate value}) \) |
| `log_other` | float | normalized \( \log(1+\text{other output value}) \) |

Normalization:
$$\text{log\_cand} = \frac{\log(1+\text{cand\_value})}{\log(1+\text{value\_norm})}$$

$$\text{log\_other} = \frac{\log(1+\text{other\_value})}{\log(1+\text{value\_norm})}$$

with `value_norm = cfg.log1p_value_norm` (example: \(10^8\)).

---

### “Optimal change” feature (optional but implemented)

| Feature name | Type | Meaning |
|---|---:|---|
| `optimal_ok` | {0,1} | candidate satisfies a soft “optimal change” inequality |

Definition:
- Let `min_input_sats` be the minimum input value among spent UTXOs.
- Candidate is “optimal-ok” if:

$$\text{cand\_value} \le \text{min\_input} - \text{fee} + \text{slack}$$

with `slack = cfg.optimal_change_slack_sats` (often 0).

This encodes a classic heuristic as a **feature**, not a hard filter.

---

## Decision policy hyperparameters (model-adjacent, not coefficients)

These are not weights; they govern when the model’s output is trusted:

- `min_p_gap`: require separation between best and runner-up candidate
- `p_accept_base`, `p_accept_log_slope`: threshold schedule based on anchor cluster size
- `p_accept_min`, `p_accept_max`: clamp bounds on the schedule

This makes the system more conservative for attaching change to already-large entities.

---

## Evidence bits and what they mean downstream

The model doesn’t directly “assign entities”; it triggers evidence + unions:

- **bit2 (change-output)**: address was predicted as change output at least once
- **bit4 (change-anchor)**: anchor/input address participated in a change prediction
- (Separately) **bit1** comes from multi-input heuristic (H1)

These bits are persisted in `address_confidence_YYYY.parquet` and used for diagnostics such as top-cluster evidence composition.


In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import annotations

import gc
import glob
import math
import os
import random
import sqlite3
import sys
from collections import Counter
from dataclasses import dataclass
from datetime import date, timedelta
from pathlib import Path
from typing import Literal, Optional

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm.auto import tqdm


# =============================================================================
# Determinism helpers
# =============================================================================

def set_determinism(seed: int) -> None:
    """
    Makes the pipeline *more reproducible* across runs.
    """
    os.environ.setdefault("PYTHONHASHSEED", str(seed))
    random.seed(seed)
    np.random.seed(seed)


# =============================================================================
# Patch 2: Online quantile calibrator (year-adaptive acceptance, no magic caps)
# =============================================================================

def _clamp(x: float, lo: float, hi: float) -> float:
    return float(max(lo, min(hi, x)))


class OnlineQuantileCalibrator:
    """
    Rolling quantile estimator over a fixed-size ring buffer, updated periodically.

    Deterministic: uses numpy partition on a copy of the current buffer window.
    """

    def __init__(
        self,
        *,
        quantile: float,
        window: int,
        update_every: int,
        warmup_min_samples: int,
        init_value: float,
        lo: float,
        hi: float,
    ):
        if not (0.0 < quantile < 1.0):
            raise ValueError("quantile must be in (0,1)")
        if window <= 0:
            raise ValueError("window must be > 0")

        self.q = float(quantile)
        self.window = int(window)
        self.update_every = int(update_every)
        self.warmup_min = int(warmup_min_samples)
        self.lo = float(lo)
        self.hi = float(hi)

        self.buf = np.empty(self.window, dtype=np.float32)
        self.count = 0
        self.idx = 0
        self.value = _clamp(float(init_value), self.lo, self.hi)

    def update(self, x: float) -> None:
        self.buf[self.idx] = float(x)
        self.idx = (self.idx + 1) % self.window
        self.count += 1

        if self.count < self.warmup_min:
            return
        if (self.count % self.update_every) != 0:
            return

        n = min(self.count, self.window)
        tmp = self.buf[:n].copy()
        k = int(max(0, min(n - 1, int(self.q * (n - 1)))))
        v = float(np.partition(tmp, k)[k])
        self.value = _clamp(v, self.lo, self.hi)


class ReservoirSampler:
    """
    Deterministic reservoir sampling given a fixed RNG seed.
    """

    def __init__(self, max_samples: int, seed: int):
        self.max = int(max_samples)
        self.n_seen = 0
        self.rng = np.random.default_rng(int(seed))

    def consider(self) -> Optional[int]:
        """
        Returns:
          - None => drop
          - idx  => store/replace at idx
        """
        self.n_seen += 1
        if self.max <= 0:
            return None
        if self.n_seen <= self.max:
            return self.n_seen - 1
        j = int(self.rng.integers(0, self.n_seen))
        if j < self.max:
            return j
        return None


class LogitEvalCollector:
    """
    Stores (score,label) pairs with reservoir sampling to cap memory.
    """

    def __init__(self, max_samples: int, seed: int):
        self.sampler = ReservoirSampler(max_samples=max_samples, seed=seed)
        self.scores: list[float] = []
        self.labels: list[int] = []

    def add(self, score: float, label: int) -> None:
        idx = self.sampler.consider()
        if idx is None:
            return
        if idx == len(self.scores):
            self.scores.append(float(score))
            self.labels.append(int(label))
        else:
            self.scores[idx] = float(score)
            self.labels[idx] = int(label)

    def arrays(self) -> tuple[np.ndarray | None, np.ndarray | None]:
        if not self.scores:
            return None, None
        return np.asarray(self.scores, dtype=np.float32), np.asarray(self.labels, dtype=np.int8)


# =============================================================================
# Config objects (NO DEFAULTS — everything must be explicit)
# =============================================================================

ValueUnitMode = Literal["infer", "btc", "sats"]
MultiInputPolicy = Literal["one_output", "one_or_two_nonmix"]


@dataclass(frozen=True)
class PathsConfig:
    parquet_io_glob: str
    outputs_dir: Path
    outpoint_db_path: Path
    entity_map_out_path: Path


@dataclass(frozen=True)
class AnalysisConfig:
    analysis_start: date
    analysis_end_exclusive: date
    outpoint_db_lookback_days: int
    value_unit_mode: ValueUnitMode


@dataclass(frozen=True)
class OutputConfig:
    reset_outpoint_db: bool
    write_entity_mapping: bool
    entity_map_compression: str
    entity_write_batch: int
    top_k_clusters_print: int


@dataclass(frozen=True)
class PlotConfig:
    enable_plots: bool
    zipf_top_k: int
    focus_max_size: int
    log_bins: int

    # NEW: logit performance plots
    logit_eval_max_samples: int
    logit_calibration_bins: int
    logit_roc_points: int

    # saving / displaying
    save_plots: bool
    show_plots: bool


@dataclass(frozen=True)
class PerfConfig:
    outpoint_commit_every_rows: int
    prevout_hybrid_threshold: int
    prevout_lookup_chunk: int
    iter_buffer_in: int
    iter_buffer_out: int
    iter_buffer_grouped: int
    gc_every_n_files: int
    tqdm_mininterval: float
    tqdm_miniters: int
    tqdm_postfix_every: int

    # Patch 3: set-based prevout join path (SQLite join + SQLite aggregates + Polars transport)
    use_sql_prevout_join: bool
    vinbuf_insert_chunk: int
    vinbuf_fetch_chunk: int


@dataclass(frozen=True)
class SanityConfig:
    run_sanity_checks: bool
    run_prevout_db_sanity: bool
    prevout_sanity_sample_limit: int
    prevout_sanity_parquet: str | None


@dataclass(frozen=True)
class HeuristicToggles:
    enable_coinjoin_filter: bool
    enable_multi_input: bool
    enable_change: bool
    enable_merge_guards: bool
    precreate_nodes_for_all_output_addrs: bool
    create_nodes_for_all_resolved_inputs: bool


@dataclass(frozen=True)
class HeuristicParams:
    dust_sats: int
    max_fee_abs_sats: int
    max_fee_frac: float
    max_change_inputs_utxos: int
    max_change_spendable_outs: int
    change_require_2_outputs: bool
    multi_input_policy: MultiInputPolicy


UltraVoteRule = tuple[int, int]  # (threshold, votes_required)


@dataclass(frozen=True)
class MergeGuardParams:
    # Safety ceiling
    max_merged_component_size: int

    # Ratio guard (CHANGE only) + ignore tiny components (singleton change)
    merge_ratio_guard: bool
    merge_ratio_max: float
    merge_ratio_big_cluster_min: int
    merge_ratio_small_floor: int

    # Big–big gating (CHANGE only): require multiple "confirmations" per component-pair
    enable_change_merge_votes: bool
    change_vote_min_side: int
    change_vote_merged_trigger: int
    change_votes_required: int
    ultra_change_vote_rules: tuple[UltraVoteRule, ...]

    # Degree guard (CHANGE-only): "change component attempts to attach to too many distinct large anchors"
    enable_change_degree_guard: bool
    change_degree_large_min: int
    change_degree_small_min: int
    change_degree_max_distinct_large: int
    change_degree_set_cap: int

    # Diagnostics: ratio-guard sample log
    ratio_guard_sample_n: int
    ratio_guard_sample_path: Path

    # Constraint event log
    constraint_log_n: int
    constraint_log_path: Path

    # Debug logging of large merges (optional)
    debug_large_change_merges: bool
    debug_change_merge_min: int
    debug_change_merge_log_path: Path


@dataclass(frozen=True)
class ConfidenceConfig:
    """
    Proxy "probability" that an address is *meaningfully clustered* (diagnostic / downstream weight).
    This does NOT claim statistical calibration; it's an evidence-based score in [0,1].
    """
    enable_confidence_proxy: bool
    confidence_out_path: Path
    confidence_compression: str
    confidence_write_batch: int

    # scoring knobs
    size_norm: int
    size_bonus_max: float
    p_singleton: float
    p_base_0: float
    p_base_1: float
    p_base_2plus: float

    include_cluster_size: bool
    include_evidence_bits: bool


# =============================================================================
# Change model: features + logistic scoring + Patch 2 acceptance (adaptive)
# =============================================================================

@dataclass(frozen=True)
class ChangeModelConfig:
    """
    Deterministic logistic scoring + deterministic year-adaptive acceptance via rolling quantiles.

    Score:
      p = sigmoid(intercept + sum w_i * x_i)

    Acceptance (Patch 2):
      - maintain rolling window of p_best on "easy" txs:
          easy := 2-out, non-mixing, resolved-all (and model invoked)
      - p_accept := high quantile of p_best (bounded)
      - gap threshold:
          either fixed min_p_gap_fixed, or rolling quantile of p_gap := p_best - p_second (bounded)
    """
    intercept: float
    weights: dict[str, float]

    # Fixed gap option (always available)
    min_p_gap_fixed: float

    # Optional quantile-based gap threshold
    p_gap_quantile: float | None
    p_gap_window: int
    p_gap_update_every: int
    p_gap_warmup_min_samples: int
    p_gap_init: float
    p_gap_min: float
    p_gap_max: float

    # Quantile-based acceptance threshold
    p_accept_quantile: float
    p_accept_window: int
    p_accept_update_every: int
    p_accept_warmup_min_samples: int
    p_accept_init: float
    p_accept_min: float
    p_accept_max: float

    # Optional soft "optimal change" slack (kept as a feature)
    optimal_change_slack_sats: int

    # Feature normalization knobs
    log1p_fee_norm: float
    log1p_value_norm: float


def sigmoid(x: float) -> float:
    if x >= 0:
        z = math.exp(-x)
        return 1.0 / (1.0 + z)
    z = math.exp(x)
    return z / (1.0 + z)


def extract_change_features(
    *,
    n_in_utxos: int,
    n_spendable_out: int,
    fee_sats: int,
    sum_inputs_sats: int,
    sum_outputs_sats: int,
    min_input_sats: int,
    in_type_uniform: bool,
    out_type_match_in: bool,
    candidate_new: bool,
    candidate_value_sats: int,
    other_value_sats: int,
    candidate_is_min_out: bool,
    candidate_lt_other: bool,
    is_mixing_like: bool,
    cfg: ChangeModelConfig,
) -> dict[str, float]:
    fee_frac = (fee_sats / max(1, sum_inputs_sats)) if sum_inputs_sats > 0 else 0.0

    log_fee = math.log1p(max(0, fee_sats)) / math.log1p(cfg.log1p_fee_norm)
    log_cand = math.log1p(max(0, candidate_value_sats)) / math.log1p(cfg.log1p_value_norm)
    log_other = math.log1p(max(0, other_value_sats)) / math.log1p(cfg.log1p_value_norm)

    optimal_ok = 0.0
    if min_input_sats > 0:
        if candidate_value_sats <= (min_input_sats - fee_sats + int(cfg.optimal_change_slack_sats)):
            optimal_ok = 1.0

    return {
        "bias": 1.0,
        "n_in": float(n_in_utxos),
        "n_out": float(n_spendable_out),
        "fee_frac": float(fee_frac),
        "log_fee": float(log_fee),
        "log_cand": float(log_cand),
        "log_other": float(log_other),
        "in_type_uniform": 1.0 if in_type_uniform else 0.0,
        "out_type_match_in": 1.0 if out_type_match_in else 0.0,
        "cand_new": 1.0 if candidate_new else 0.0,
        "cand_is_min_out": 1.0 if candidate_is_min_out else 0.0,
        "cand_lt_other": 1.0 if candidate_lt_other else 0.0,
        "optimal_ok": float(optimal_ok),
        "is_mixing_like": 1.0 if is_mixing_like else 0.0,
        "sum_in_gt_sum_out": 1.0 if (sum_inputs_sats > sum_outputs_sats) else 0.0,
    }


def score_change_candidate(feats: dict[str, float], cfg: ChangeModelConfig) -> float:
    s = float(cfg.intercept)
    for k, v in feats.items():
        w = cfg.weights.get(k, 0.0)
        s += w * float(v)
    return float(sigmoid(s))


# =============================================================================
# Utility helpers
# =============================================================================

def extract_day_from_path(path: str) -> date | None:
    p = Path(path)
    for part in p.parts:
        if part.startswith("day="):
            day_str = part.split("=", 1)[1]
            try:
                return date.fromisoformat(day_str)
            except ValueError:
                return None
    return None


def addr_type(addr: str) -> str:
    if addr.startswith("1"):
        return "p2pkh"
    if addr.startswith("3"):
        return "p2sh"
    if addr.startswith("bc1q"):
        return "bech32_p2wpkh"
    if addr.startswith("bc1p"):
        return "taproot"
    return "other"


def infer_value_unit_from_sample(sample_vals: list[float]) -> str:
    if not sample_vals:
        return "btc"
    mx = max(sample_vals)
    return "sats" if mx >= 1e6 else "btc"


class UnionFind:
    """
    Union-Find with rank, path compression, component sizes,
    plus degree-guard metadata.
    """
    def __init__(self):
        self.parent: list[int] = []
        self.rank: list[int] = []
        self.size: list[int] = []
        self.change_large_anchor_roots: dict[int, set[int]] = {}

    def make_set(self) -> int:
        idx = len(self.parent)
        self.parent.append(idx)
        self.rank.append(0)
        self.size.append(1)
        return idx

    def find(self, x: int) -> int:
        parent = self.parent
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union_roots(self, rx: int, ry: int, *, change_degree_set_cap: int) -> int:
        if rx == ry:
            return rx

        parent = self.parent
        rank = self.rank
        size = self.size

        if rank[rx] < rank[ry]:
            parent[rx] = ry
            size[ry] += size[rx]
            new_root, old_root = ry, rx
        elif rank[rx] > rank[ry]:
            parent[ry] = rx
            size[rx] += size[ry]
            new_root, old_root = rx, ry
        else:
            parent[ry] = rx
            rank[rx] += 1
            size[rx] += size[ry]
            new_root, old_root = rx, ry

        sa = self.change_large_anchor_roots.get(new_root)
        sb = self.change_large_anchor_roots.get(old_root)
        if sa is None and sb is None:
            return new_root

        merged: set[int] = set()
        if sa:
            merged |= sa
        if sb:
            merged |= sb

        if len(merged) > change_degree_set_cap:
            merged = set(sorted(merged)[:change_degree_set_cap])

        self.change_large_anchor_roots[new_root] = merged
        self.change_large_anchor_roots.pop(old_root, None)
        return new_root


def _count_dupe_values(values: list[int], dust_sats: int) -> tuple[int, int]:
    vals = [v for v in values if v >= dust_sats]
    if not vals:
        return 0, 0
    cnt = Counter(vals)
    return max(cnt.values()), len(cnt)


def detect_mixing_like(n_in_utxos: int, spendable_outs: list[tuple[str, int]], dust_sats: int) -> bool:
    n_out = len(spendable_outs)
    if n_in_utxos < 3 or n_out < 3:
        return False

    out_vals = [v for (_a, v) in spendable_outs]
    out_types = [addr_type(a) for (a, _v) in spendable_outs if a is not None]
    unique_vals = len(set(out_vals))
    unique_types = len(set(out_types)) if out_types else 0
    max_dupe, distinct_vals_non_dust = _count_dupe_values(out_vals, dust_sats)

    if max_dupe >= 2 and distinct_vals_non_dust >= 2:
        return True
    if abs(n_in_utxos - n_out) <= 1 and min(n_in_utxos, n_out) >= 3:
        return True
    if n_out >= 4 and unique_vals <= (n_out // 2):
        return True
    if n_out >= 4 and unique_types == 1 and max_dupe >= 2:
        return True
    return False


def multi_input_is_safe(policy: MultiInputPolicy, n_in_utxos: int, n_spendable_out: int, is_mixing_like: bool) -> bool:
    if n_in_utxos < 2:
        return False
    if policy == "one_output":
        return n_spendable_out == 1
    if policy == "one_or_two_nonmix":
        if n_spendable_out == 1:
            return True
        if n_spendable_out == 2 and not is_mixing_like:
            return True
        return False
    raise ValueError(f"Unknown multi_input_policy={policy}")


# =============================================================================
# SQLite outpoint DB + TEMP join buffers
# =============================================================================

def open_outpoint_db(db_path: Path, reset: bool) -> sqlite3.Connection:
    if reset and db_path.exists():
        db_path.unlink()

    conn = sqlite3.connect(str(db_path))
    cur = conn.cursor()

    cur.execute("PRAGMA journal_mode=WAL;")
    cur.execute("PRAGMA synchronous=OFF;")
    cur.execute("PRAGMA temp_store=MEMORY;")
    cur.execute("PRAGMA cache_size=-2000000;")
    cur.execute("PRAGMA wal_autocheckpoint=5000;")

    conn.isolation_level = None
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS outpoints (
            txid TEXT NOT NULL,
            n    INTEGER NOT NULL,
            address TEXT NOT NULL,
            value_sats INTEGER NOT NULL,
            PRIMARY KEY(txid, n)
        ) WITHOUT ROWID;
        """
    )
    return conn


def init_lookup_tables(conn: sqlite3.Connection) -> None:
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS keybuf;")
    cur.execute(
        """
        CREATE TEMP TABLE keybuf (
            txid TEXT NOT NULL,
            n    INTEGER NOT NULL
        );
        """
    )

    cur.execute("DROP TABLE IF EXISTS vinbuf;")
    cur.execute(
        """
        CREATE TEMP TABLE vinbuf (
            spend_txid TEXT NOT NULL,
            prev_txid  TEXT NOT NULL,
            prev_n     INTEGER NOT NULL
        );
        """
    )


def reset_vinbuf(conn: sqlite3.Connection) -> None:
    conn.execute("DELETE FROM vinbuf;")


def insert_outpoints_no_commit(conn: sqlite3.Connection, rows: list[tuple[str, int, str, int]]) -> None:
    if rows:
        conn.executemany(
            "INSERT OR IGNORE INTO outpoints(txid, n, address, value_sats) VALUES (?, ?, ?, ?);",
            rows,
        )


def lookup_outpoints_or(conn: sqlite3.Connection, keys: list[tuple[str, int]], chunk_size: int) -> dict[tuple[str, int], tuple[str, int]]:
    if not keys:
        return {}
    keys = list(dict.fromkeys(keys))
    cur = conn.cursor()
    out: dict[tuple[str, int], tuple[str, int]] = {}

    for i in range(0, len(keys), chunk_size):
        chunk = keys[i : i + chunk_size]
        where = " OR ".join(["(txid=? AND n=?)"] * len(chunk))
        params = [p for k in chunk for p in k]
        cur.execute(f"SELECT txid, n, address, value_sats FROM outpoints WHERE {where};", params)
        for txid, n, address, value_sats in cur.fetchall():
            out[(txid, int(n))] = (address, int(value_sats))
    return out


def lookup_outpoints_join(conn: sqlite3.Connection, keys: list[tuple[str, int]], chunk_size: int) -> dict[tuple[str, int], tuple[str, int]]:
    if not keys:
        return {}
    keys = list(dict.fromkeys(keys))
    cur = conn.cursor()
    out: dict[tuple[str, int], tuple[str, int]] = {}

    for i in range(0, len(keys), chunk_size):
        chunk = keys[i : i + chunk_size]
        cur.execute("DELETE FROM keybuf;")
        cur.executemany("INSERT INTO keybuf(txid, n) VALUES (?, ?);", chunk)
        cur.execute(
            """
            SELECT k.txid, k.n, o.address, o.value_sats
            FROM keybuf k
            JOIN outpoints o
              ON o.txid = k.txid AND o.n = k.n;
            """
        )
        for txid, n, address, value_sats in cur.fetchall():
            out[(txid, int(n))] = (address, int(value_sats))
    return out


def lookup_outpoints_hybrid(conn: sqlite3.Connection, keys: list[tuple[str, int]], hybrid_threshold: int, join_chunk: int) -> dict[tuple[str, int], tuple[str, int]]:
    if not keys:
        return {}
    if len(keys) < hybrid_threshold:
        return lookup_outpoints_or(conn, keys, chunk_size=500)
    return lookup_outpoints_join(conn, keys, chunk_size=join_chunk)


# =============================================================================
# Patch 3: Resolve + aggregate inputs inside SQLite, transport to Polars
# =============================================================================

def fetch_prevouts_aggs_polars(conn: sqlite3.Connection, fetch_chunk: int) -> pl.DataFrame:
    cur = conn.cursor()
    cur.execute(
        """
        SELECT
            v.spend_txid                                 AS txid,
            COUNT(*)                                      AS resolved_cnt,
            SUM(o.value_sats)                             AS sum_inputs_sats,
            MIN(o.value_sats)                             AS min_input_sats,
            group_concat(DISTINCT o.address)              AS in_addrs_csv
        FROM vinbuf v
        JOIN outpoints o
          ON o.txid = v.prev_txid AND o.n = v.prev_n
        GROUP BY v.spend_txid;
        """
    )

    dfs: list[pl.DataFrame] = []
    while True:
        rows = cur.fetchmany(fetch_chunk)
        if not rows:
            break
        dfs.append(
            pl.DataFrame(
                rows,
                schema=["txid", "resolved_cnt", "sum_inputs_sats", "min_input_sats", "in_addrs_csv"],
                orient="row",
            )
        )

    if not dfs:
        return pl.DataFrame(
            schema={
                "txid": pl.Utf8,
                "resolved_cnt": pl.Int64,
                "sum_inputs_sats": pl.Int64,
                "min_input_sats": pl.Int64,
                "in_addrs": pl.List(pl.Utf8),
            }
        )

    df = pl.concat(dfs, how="vertical", rechunk=True)

    df = df.with_columns(
        pl.col("in_addrs_csv")
        .fill_null("")
        .str.split(",")
        .alias("in_addrs")
    ).drop("in_addrs_csv")

    df = df.with_columns(
        pl.when(pl.col("in_addrs") == [""])
        .then(pl.lit([], dtype=pl.List(pl.Utf8)))
        .otherwise(pl.col("in_addrs"))
        .alias("in_addrs")
    )

    df = df.with_columns(pl.col("in_addrs").list.sort().alias("in_addrs"))
    return df


# =============================================================================
# Sanity checks + prevout join sanity
# =============================================================================

OUTPUT_DIR_VALUES = {"out", "vout", "output", "o"}
INPUT_DIR_VALUES  = {"in", "vin", "input", "i"}


def run_sanity_checks(
    n_nodes: int,
    node_to_entity: list[int] | np.ndarray | None = None,
    cluster_size_counter: Counter | None = None,
    prevout_lookups: int | None = None,
    prevout_hits: int | None = None,
    top_k: int = 20,
) -> None:
    if cluster_size_counter is None:
        if node_to_entity is None:
            raise ValueError("Provide either node_to_entity or cluster_size_counter.")
        cluster_size_counter = Counter(list(node_to_entity))

    sizes = list(cluster_size_counter.values())
    if not sizes:
        print("[SANITY] No clusters found (sizes empty).", flush=True)
        return

    sizes_sorted = sorted(sizes, reverse=True)
    total_nodes_from_sizes = sum(sizes_sorted)
    largest = sizes_sorted[0]
    frac = largest / total_nodes_from_sizes if total_nodes_from_sizes else float("nan")

    print("\n[SANITY] Cluster summary", flush=True)
    print(f"  UF nodes (n_nodes): {n_nodes:,}", flush=True)
    print(f"  Total nodes from cluster sizes: {total_nodes_from_sizes:,}", flush=True)
    if total_nodes_from_sizes != n_nodes:
        print("  [WARN] sum(cluster_sizes) != n_nodes  -> mismatch suggests a bug in mapping logic.", flush=True)
    print(f"  Entities (clusters): {len(cluster_size_counter):,}", flush=True)
    print(f"  Largest cluster size: {largest:,}", flush=True)
    print(f"  Largest cluster fraction of nodes: {frac:.2%}", flush=True)

    print(f"\n[SANITY] Top {top_k} cluster sizes:", flush=True)
    print(" ", sizes_sorted[:top_k], flush=True)

    def pct(p: float) -> int:
        if not sizes_sorted:
            return 0
        s_asc = sorted(sizes_sorted)
        idx = max(0, min(len(s_asc) - 1, math.ceil(p * len(s_asc)) - 1))
        return int(s_asc[idx])

    med = int(sorted(sizes_sorted)[len(sizes_sorted) // 2])
    print("\n[SANITY] Quick distribution stats", flush=True)
    print(f"  Median cluster size: {med:,}", flush=True)
    print(f"  90th percentile cluster size: {pct(0.90):,}", flush=True)
    print(f"  99th percentile cluster size: {pct(0.99):,}", flush=True)

    if prevout_lookups is not None and prevout_hits is not None:
        rate = (prevout_hits / prevout_lookups) if prevout_lookups else float("nan")
        print("\n[SANITY] Prevout lookup hit-rate (DB)", flush=True)
        print(f"  Lookups:  {prevout_lookups:,}", flush=True)
        print(f"  Hits:     {prevout_hits:,}", flush=True)
        print(f"  Hit-rate: {rate:.2%}", flush=True)


def prevout_join_sanity_db(
    conn: sqlite3.Connection,
    parquet_path: str,
    sample_limit: int,
    hybrid_threshold: int,
    join_chunk: int,
) -> None:
    df = pl.read_parquet(parquet_path, columns=["dir", "txid", "prev_txid", "prev_vout"])
    df = df.with_columns(pl.col("dir").cast(pl.Utf8, strict=False).str.to_lowercase().alias("dir"))
    vin = (
        df.filter(pl.col("dir").is_in(list(INPUT_DIR_VALUES)))
        .select([pl.col("txid").alias("spend_txid"), "prev_txid", "prev_vout"])
        .filter(pl.col("prev_txid").is_not_null() & pl.col("prev_vout").is_not_null())
        .head(sample_limit)
    )
    print("\n[PREVOUT-SANITY | DB]", flush=True)
    print("File:", parquet_path, flush=True)
    print("Vin sample rows:", vin.height, flush=True)
    if vin.height == 0:
        return

    frac_integerlike = (
        vin.with_columns(((pl.col("prev_vout") - pl.col("prev_vout").floor()).abs() < 1e-9).alias("is_intlike"))
           .select(pl.col("is_intlike").mean())
           .item()
    )
    print(f"prev_vout integer-like fraction (sample): {float(frac_integerlike):.6f}", flush=True)

    vin2 = vin.with_columns(pl.col("prev_vout").cast(pl.Int64, strict=False).alias("prev_vout_i64")).filter(
        pl.col("prev_vout_i64").is_not_null()
    )

    keys = [(tx, int(n)) for tx, n in vin2.select(["prev_txid", "prev_vout_i64"]).iter_rows(named=False, buffer_size=200_000)]
    hits = lookup_outpoints_hybrid(conn, keys, hybrid_threshold=hybrid_threshold, join_chunk=join_chunk)
    print(f"DB prevout hit-rate (sample): {len(hits)}/{len(keys)} = {len(hits)/len(keys):.2%}" if keys else "No keys", flush=True)

    unresolved = []
    for k in keys[:5000]:
        if k not in hits:
            unresolved.append(k)
            if len(unresolved) >= 10:
                break
    if unresolved:
        print("Sample unresolved (prev_txid, prev_n):", flush=True)
        for k in unresolved:
            print(" ", k, flush=True)
    else:
        print("No unresolved keys in first 5000 keys (good sign).", flush=True)


# =============================================================================
# Plot helpers
# =============================================================================

def _log_bins(max_val: int, bins: int) -> np.ndarray:
    max_val = max(1, int(max_val))
    return np.logspace(0, math.log10(max_val), num=bins)


def _maybe_save_show(save_path: Optional[Path], show: bool) -> None:
    if save_path is not None:
        save_path.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(str(save_path), dpi=140)
    if show:
        plt.show()
    plt.close()


def plot_hist_logbins(
    sizes: np.ndarray,
    title: str,
    color: str,
    bins: int,
    max_x: int | None,
    *,
    save_path: Optional[Path],
    show: bool,
) -> None:
    if sizes.size == 0:
        print(f"[PLOT] Skipping empty plot: {title}", flush=True)
        return
    if max_x is None:
        max_x = int(sizes.max())
    b = _log_bins(max_x, bins)
    plt.figure(figsize=(11, 6))
    plt.hist(sizes, bins=b, log=True, color=color, edgecolor="black", linewidth=0.7)
    plt.xscale("log")
    plt.xlabel("Cluster size (addresses per entity)")
    plt.ylabel("Frequency (log scale)")
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", linewidth=0.35, alpha=0.6)
    plt.tight_layout()
    _maybe_save_show(save_path, show)


def plot_zipf(
    sizes: np.ndarray,
    title: str,
    color: str,
    top_k: int,
    *,
    save_path: Optional[Path],
    show: bool,
) -> None:
    if sizes.size == 0:
        print(f"[PLOT] Skipping empty plot: {title}", flush=True)
        return
    top_k = min(top_k, sizes.size)
    top = np.partition(sizes, -top_k)[-top_k:]
    top_sorted = np.sort(top)[::-1]
    ranks = np.arange(1, top_sorted.size + 1)

    plt.figure(figsize=(11, 6))
    plt.plot(ranks, top_sorted, color=color, linewidth=1.3)
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel("Rank (log scale)")
    plt.ylabel("Cluster size (log scale)")
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", linewidth=0.35, alpha=0.6)
    plt.tight_layout()
    _maybe_save_show(save_path, show)


def plot_logit_eval(
    scores: np.ndarray,
    labels: np.ndarray,
    *,
    roc_points: int,
    calib_bins: int,
    title_prefix: str,
    save_roc: Optional[Path],
    save_calib: Optional[Path],
    show: bool,
) -> None:
    if scores.size == 0:
        print("[PLOT] No logit eval samples.", flush=True)
        return

    # ROC
    thr = np.linspace(0.0, 1.0, int(roc_points))
    P = max(1, int(labels.sum()))
    N = max(1, int((1 - labels).sum()))

    tpr = np.empty_like(thr)
    fpr = np.empty_like(thr)
    for i, t in enumerate(thr):
        pred = scores >= t
        tp = int(np.sum(pred & (labels == 1)))
        fp = int(np.sum(pred & (labels == 0)))
        tpr[i] = tp / P
        fpr[i] = fp / N

    order = np.argsort(fpr)
    fpr_s = fpr[order]
    tpr_s = tpr[order]
    auc = float(np.trapz(tpr_s, fpr_s))

    plt.figure(figsize=(8, 6))
    plt.plot(fpr_s, tpr_s, linewidth=1.5)
    plt.plot([0, 1], [0, 1], linestyle="--", linewidth=1.0)
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title(f"{title_prefix} ROC (proxy AUC={auc:.3f})")
    plt.grid(True, linestyle="--", linewidth=0.35, alpha=0.6)
    plt.tight_layout()
    _maybe_save_show(save_roc, show)

    # Calibration
    bins = np.linspace(0.0, 1.0, int(calib_bins) + 1)
    which = np.digitize(scores, bins) - 1  # [0..calib_bins-1]
    xs, ys = [], []
    for b in range(int(calib_bins)):
        m = which == b
        if int(np.sum(m)) < 50:
            continue
        xs.append(float(np.mean(scores[m])))
        ys.append(float(np.mean(labels[m])))

    plt.figure(figsize=(8, 6))
    plt.plot(xs, ys, marker="o", linewidth=1.2)
    plt.plot([0, 1], [0, 1], linestyle="--", linewidth=1.0)
    plt.xlabel("Predicted p_best")
    plt.ylabel("Empirical proxy rate (label)")
    plt.title(f"{title_prefix} Calibration (proxy label)")
    plt.grid(True, linestyle="--", linewidth=0.35, alpha=0.6)
    plt.tight_layout()
    _maybe_save_show(save_calib, show)


# =============================================================================
# Reporting helpers (all final prints live here)
# =============================================================================

def compute_top_cluster_evidence_composition(
    *,
    node_to_entity: np.ndarray,
    multi_change_flags: bytearray,
    top_entity_id: int,
) -> dict[str, float | int]:
    n_nodes = int(node_to_entity.size)
    if n_nodes == 0:
        return {
            "top_entity_id": int(top_entity_id),
            "n_top": 0,
            "n_bit1": 0, "n_bit2": 0, "n_bit4": 0,
            "n_bit2_only": 0,
            "share_bit1": float("nan"),
            "share_bit2": float("nan"),
            "share_bit4": float("nan"),
            "share_bit2_only": float("nan"),
        }

    bits = np.frombuffer(multi_change_flags, dtype=np.uint8, count=n_nodes)
    mask = (node_to_entity == int(top_entity_id))
    n_top = int(mask.sum())
    if n_top == 0:
        return {
            "top_entity_id": int(top_entity_id),
            "n_top": 0,
            "n_bit1": 0, "n_bit2": 0, "n_bit4": 0,
            "n_bit2_only": 0,
            "share_bit1": float("nan"),
            "share_bit2": float("nan"),
            "share_bit4": float("nan"),
            "share_bit2_only": float("nan"),
        }

    top_bits = bits[mask]
    n_bit1 = int(np.count_nonzero(top_bits & 1))
    n_bit2 = int(np.count_nonzero(top_bits & 2))
    n_bit4 = int(np.count_nonzero(top_bits & 4))
    n_bit2_only = int(np.count_nonzero(top_bits == 2))

    return {
        "top_entity_id": int(top_entity_id),
        "n_top": n_top,
        "n_bit1": n_bit1,
        "n_bit2": n_bit2,
        "n_bit4": n_bit4,
        "n_bit2_only": n_bit2_only,
        "share_bit1": float(n_bit1 / n_top),
        "share_bit2": float(n_bit2 / n_top),
        "share_bit4": float(n_bit4 / n_top),
        "share_bit2_only": float(n_bit2_only / n_top),
    }


def print_final_report(
    *,
    n_nodes: int,
    n_entities: int,
    node_to_entity: np.ndarray,
    counts: np.ndarray,
    cluster_sizes: np.ndarray,
    stats: dict[str, int],
    multi_change_flags: bytearray,
    outputs: OutputConfig,
    guards: MergeGuardParams,
    enable_constraints_diag: bool,
    constraint_unique_pairs: set[tuple[int, int]],
    constraint_gate_evals_total: int,
    constraint_pairs_repeated: int,
    change_merge_votes: dict[tuple[int, int], int],
    prevout_lookups: int,
    prevout_hits: int,
    # Patch 2 / model diagnostics
    model_diag: dict[str, float | int],
) -> int:
    print("\nFinalizing entity mapping (compressing components)...", flush=True)
    print(f"Number of unique addresses with UF nodes: {n_nodes:,}", flush=True)

    hit_rate = prevout_hits / max(1, prevout_lookups)
    print(f"[INFO] prevout lookups: {prevout_lookups:,}  hits: {prevout_hits:,}  hit-rate: {hit_rate:.3%}", flush=True)
    print(f"Number of entities (clusters): {n_entities:,}", flush=True)

    if n_entities == 0:
        print("[WARN] No entities computed.", flush=True)
        return -1

    top_k = min(outputs.top_k_clusters_print, n_entities)
    top_ids = np.argpartition(counts, -top_k)[-top_k:]
    top_ids_sorted = top_ids[np.argsort(counts[top_ids])[::-1]]
    top_entity_id = int(top_ids_sorted[0])

    print("\n[TOP CLUSTERS]", flush=True)
    for rank, eid in enumerate(top_ids_sorted, start=1):
        sz = int(counts[eid])
        frac = (sz / n_nodes) if n_nodes else float("nan")
        print(f"  #{rank:02d}  entity_id={int(eid):>8d}  size={sz:>10,d}  frac={frac:>7.2%}", flush=True)

    largest = int(cluster_sizes.max()) if cluster_sizes.size else 0
    largest_frac = (largest / n_nodes) if n_nodes else float("nan")
    q50, q90, q99 = np.quantile(cluster_sizes, [0.5, 0.9, 0.99]) if cluster_sizes.size else (0, 0, 0)

    print("\n[CLUSTER STATS]", flush=True)
    print(f"  Entities: {n_entities:,}", flush=True)
    print(f"  Largest cluster size: {largest:,}", flush=True)
    print(f"  Largest cluster fraction of nodes: {largest_frac:.2%}", flush=True)
    print(f"  Median cluster size: {int(q50)}", flush=True)
    print(f"  90th percentile: {int(q90)}", flush=True)
    print(f"  99th percentile: {int(q99)}", flush=True)

    comp = compute_top_cluster_evidence_composition(
        node_to_entity=node_to_entity,
        multi_change_flags=multi_change_flags,
        top_entity_id=top_entity_id,
    )

    print("\n[TOP CLUSTER EVIDENCE COMPOSITION]", flush=True)
    print(f"  Top entity_id: {comp['top_entity_id']:,}", flush=True)
    print(f"  Addresses in top cluster: {comp['n_top']:,}", flush=True)
    print(f"  bit1 (multi-input)   : {comp['n_bit1']:,}  share={comp['share_bit1']:.2%}", flush=True)
    print(f"  bit2 (change-output) : {comp['n_bit2']:,}  share={comp['share_bit2']:.2%}", flush=True)
    print(f"  bit4 (change-anchor) : {comp['n_bit4']:,}  share={comp['share_bit4']:.2%}", flush=True)
    print(f"  bit2-only            : {comp['n_bit2_only']:,}  share={comp['share_bit2_only']:.2%}", flush=True)
    print("  (collapse signature often: very high bit2-only share + low bit1 share)", flush=True)

    flags_view = multi_change_flags[:n_nodes]
    n_addrs_multi = sum(1 for v in flags_view if (v & 1))
    n_addrs_change_out = sum(1 for v in flags_view if (v & 2))
    n_addrs_change_anchor = sum(1 for v in flags_view if (v & 4))
    n_addrs_touched = sum(1 for v in flags_view if (v & 7))

    print("\n[HEURISTIC COVERAGE (node-level)]", flush=True)
    print(f"  Total txs processed (>=1 input UTXO): {stats['n_txs_total']:,}", flush=True)
    print(f"  Mixing-like skipped: {stats['n_txs_coinjoin_flagged']:,}", flush=True)
    print(f"  Multi-input txs (SAFE policy applied): {stats['n_txs_with_multiinput']:,}", flush=True)
    print(f"  Change detected (model): {stats['n_txs_with_change_detected']:,}", flush=True)
    print(f"  Nodes marked multi-input: {n_addrs_multi:,}", flush=True)
    print(f"  Nodes marked change-output: {n_addrs_change_out:,}", flush=True)
    print(f"  Nodes marked change-anchor: {n_addrs_change_anchor:,}", flush=True)
    print(f"  Nodes touched by any heuristic: {n_addrs_touched:,}", flush=True)

    print("\n[MODEL CALIBRATION (Patch 2)]", flush=True)
    print(f"  easy txs seen (calibration stream): {int(model_diag.get('easy_seen', 0)):,}", flush=True)
    print(f"  easy txs accepted: {int(model_diag.get('easy_accepted', 0)):,}", flush=True)
    denom = max(1, int(model_diag.get('easy_seen', 0)))
    print(f"  easy accept-rate: {int(model_diag.get('easy_accepted', 0))/denom:.4%}", flush=True)
    print(f"  p_accept (rolling quantile): {float(model_diag.get('p_accept', float('nan'))):.4f}", flush=True)
    print(f"  p_gap threshold: {float(model_diag.get('p_gap', float('nan'))):.4f}", flush=True)
    print(f"  logit eval samples stored: {int(model_diag.get('eval_samples', 0)):,}", flush=True)

    print("\n[UNION DIAGNOSTICS]", flush=True)
    print(f"  Union attempts: {stats['union_attempts']:,}", flush=True)
    print(f"  Unions applied: {stats['unions_applied']:,}", flush=True)
    print(f"    - applied via H1:     {stats['unions_applied_h1']:,}", flush=True)
    print(f"    - applied via CHANGE: {stats['unions_applied_change']:,}", flush=True)

    print(f"  skip_same_component (already merged): {stats['skip_same_component']:,}", flush=True)
    print(f"    - same_component via H1:     {stats['skip_same_component_h1']:,}", flush=True)
    print(f"    - same_component via CHANGE: {stats['skip_same_component_change']:,}", flush=True)

    print(f"  Unions skipped by guards/votes/constraints: {stats['unions_skipped']:,}", flush=True)
    print(f"    - skipped (H1):     {stats['unions_skipped_h1']:,}", flush=True)
    print(f"    - skipped (CHANGE): {stats['unions_skipped_change']:,}", flush=True)

    print("  Skip breakdown:", flush=True)
    print(f"    - skip_abs_cap: {stats['skip_abs_cap']:,}  (H1={stats['skip_abs_cap_h1']:,}, CHANGE={stats['skip_abs_cap_change']:,})", flush=True)
    print(f"    - skip_ratio_guard: {stats['skip_ratio_guard']:,}  (CHANGE-only)", flush=True)
    print(f"    - skip_constraint: {stats['skip_constraint']:,}  (CHANGE-only; vote pending)", flush=True)
    print(f"    - skip_degree_guard: {stats['skip_degree_guard']:,}  (CHANGE-only)", flush=True)
    print(f"    - skip_vote_pending: {stats['skip_vote_pending']:,}", flush=True)
    print(f"    - skip_vote_failed:  {stats['skip_vote_failed']:,}", flush=True)

    if enable_constraints_diag:
        print("\n[CONSTRAINT DIAGNOSTICS]", flush=True)
        print(f"  Vote-gating pairs seen (unique): {len(constraint_unique_pairs):,}", flush=True)
        print(f"  Vote-gating evaluations (total): {constraint_gate_evals_total:,}", flush=True)
        print(f"  Vote-gating pairs that repeated: {constraint_pairs_repeated:,}", flush=True)
        active_pairs = sum(1 for _k, v in change_merge_votes.items() if v > 0)
        print(f"  Vote pairs tracked (total): {active_pairs:,}", flush=True)

    return top_entity_id


# =============================================================================
# Main executor
# =============================================================================

def run_entity_clustering(
    paths: PathsConfig,
    analysis: AnalysisConfig,
    outputs: OutputConfig,
    plots: PlotConfig,
    perf: PerfConfig,
    sanity: SanityConfig,
    toggles: HeuristicToggles,
    params: HeuristicParams,
    guards: MergeGuardParams,
    confidence: ConfidenceConfig,
    change_model: ChangeModelConfig,
    *,
    determinism_seed: int,
) -> None:
    set_determinism(determinism_seed)

    try:
        sys.stdout.reconfigure(line_buffering=True)
        sys.stderr.reconfigure(line_buffering=True)
    except Exception:
        pass

    pl.Config.set_tbl_rows(20)
    pl.Config.set_fmt_str_lengths(80)

    paths.outputs_dir.mkdir(parents=True, exist_ok=True)

    io_paths = sorted(glob.glob(paths.parquet_io_glob))
    print("Parquet file counts:", {"io": len(io_paths)}, flush=True)
    print(f"[INFO] Analysis window: {analysis.analysis_start} .. {analysis.analysis_end_exclusive} (exclusive)", flush=True)

    index_start = analysis.analysis_start - timedelta(days=analysis.outpoint_db_lookback_days)
    print(f"[INFO] Outpoint DB preload start: {index_start} .. {analysis.analysis_end_exclusive} (exclusive)", flush=True)
    print(f"[INFO] Outpoint DB path: {paths.outpoint_db_path} (reset={outputs.reset_outpoint_db})", flush=True)

    eligible: list[tuple[str, date]] = []
    for path in io_paths:
        d = extract_day_from_path(path)
        if d is None:
            continue
        if index_start <= d < analysis.analysis_end_exclusive:
            eligible.append((path, d))

    n_preload_total = sum(1 for _p, d in eligible if d < analysis.analysis_start)
    n_analysis_total = len(eligible) - n_preload_total
    print(f"[INFO] Eligible files in window: {len(eligible)}  (preload={n_preload_total}, analysis={n_analysis_total})", flush=True)

    if not eligible:
        print("[WARN] No eligible parquet files found for the requested window.", flush=True)
        return

    sanity_parquet = sanity.prevout_sanity_parquet
    if sanity_parquet is None:
        for p, d in eligible:
            if analysis.analysis_start <= d < analysis.analysis_end_exclusive:
                sanity_parquet = p
                break
        if sanity_parquet is None and eligible:
            sanity_parquet = eligible[0][0]
    if sanity.run_sanity_checks and sanity_parquet:
        print(f"[INFO] Sanity parquet selected: {sanity_parquet}", flush=True)

    conn = open_outpoint_db(paths.outpoint_db_path, reset=outputs.reset_outpoint_db)
    init_lookup_tables(conn)

    conn.execute("BEGIN;")
    pending_outpoint_rows = 0

    uf = UnionFind()
    addr_to_id: dict[str, int] = {}

    # Evidence flags:
    # bit 1: multi-input evidence
    # bit 2: change-output evidence
    # bit 4: change-anchor evidence
    multi_change_flags = bytearray()
    seen_output_flags = bytearray()

    seen_output_addrs: set[str] | None
    if toggles.precreate_nodes_for_all_output_addrs:
        seen_output_addrs = None
    else:
        seen_output_addrs = set()

    def ensure_flag_capacity(idx: int) -> None:
        needed = idx + 1
        cur = len(seen_output_flags)
        if cur < needed:
            delta = needed - cur
            seen_output_flags.extend(b"\x00" * delta)
            multi_change_flags.extend(b"\x00" * delta)

    def get_addr_id(addr: str) -> int:
        idx = addr_to_id.get(addr)
        if idx is None:
            idx = uf.make_set()
            addr_to_id[addr] = idx
            ensure_flag_capacity(idx)
        return idx

    # value unit mode
    if analysis.value_unit_mode == "btc":
        value_unit: str | None = "btc"
    elif analysis.value_unit_mode == "sats":
        value_unit = "sats"
    elif analysis.value_unit_mode == "infer":
        value_unit = None
    else:
        raise ValueError(f"Unknown value_unit_mode={analysis.value_unit_mode}")

    def value_expr_to_sats() -> pl.Expr:
        nonlocal value_unit
        if value_unit is None:
            return (pl.col("value").cast(pl.Float64, strict=False) * 100_000_000).round(0).cast(pl.Int64, strict=False)
        if value_unit == "sats":
            return pl.col("value").cast(pl.Float64, strict=False).round(0).cast(pl.Int64, strict=False)
        return (pl.col("value").cast(pl.Float64, strict=False) * 100_000_000).round(0).cast(pl.Int64, strict=False)

    stats = {
        "n_files_indexed": 0,
        "n_files_analyzed": 0,
        "n_txs_total": 0,
        "n_txs_coinjoin_flagged": 0,
        "n_txs_with_multiinput": 0,
        "n_txs_with_change_detected": 0,
        "n_prevout_lookups": 0,
        "n_prevout_hits": 0,

        "union_attempts": 0,
        "unions_applied": 0,
        "unions_applied_h1": 0,
        "unions_applied_change": 0,

        "skip_same_component": 0,
        "skip_same_component_h1": 0,
        "skip_same_component_change": 0,

        "unions_skipped": 0,
        "unions_skipped_h1": 0,
        "unions_skipped_change": 0,

        "skip_abs_cap": 0,
        "skip_abs_cap_h1": 0,
        "skip_abs_cap_change": 0,

        "skip_ratio_guard": 0,
        "skip_ratio_guard_change": 0,

        "skip_constraint": 0,
        "skip_constraint_change": 0,

        "skip_degree_guard": 0,
        "skip_degree_guard_change": 0,

        "skip_vote_pending": 0,
        "skip_vote_failed": 0,
    }

    # Patch 2 calibrators (year-adaptive acceptance)
    p_accept_cal = OnlineQuantileCalibrator(
        quantile=change_model.p_accept_quantile,
        window=change_model.p_accept_window,
        update_every=change_model.p_accept_update_every,
        warmup_min_samples=change_model.p_accept_warmup_min_samples,
        init_value=change_model.p_accept_init,
        lo=change_model.p_accept_min,
        hi=change_model.p_accept_max,
    )

    p_gap_cal: OnlineQuantileCalibrator | None = None
    if change_model.p_gap_quantile is not None:
        p_gap_cal = OnlineQuantileCalibrator(
            quantile=float(change_model.p_gap_quantile),
            window=change_model.p_gap_window,
            update_every=change_model.p_gap_update_every,
            warmup_min_samples=change_model.p_gap_warmup_min_samples,
            init_value=change_model.p_gap_init,
            lo=change_model.p_gap_min,
            hi=change_model.p_gap_max,
        )

    easy_seen = 0
    easy_accepted = 0

    # Logit eval collection (proxy labels)
    eval_col = LogitEvalCollector(max_samples=plots.logit_eval_max_samples, seed=determinism_seed + analysis.analysis_start.year)

    change_merge_votes: dict[tuple[int, int], int] = {}
    constraint_gate_evals_total = 0
    constraint_pair_counts: dict[tuple[int, int], int] = {}
    constraint_unique_pairs: set[tuple[int, int]] = set()
    constraint_pairs_repeated = 0

    ratio_guard_samples_written = 0
    ratio_guard_sample_fh = None
    try:
        if guards.ratio_guard_sample_n > 0:
            ratio_guard_sample_fh = open(guards.ratio_guard_sample_path, "w", encoding="utf-8")
            ratio_guard_sample_fh.write(
                "txid\tsa\tsb\ts_big\ts_small\tratio\tchange_side\tbig_side\tbig_is_change\tchange_comp_size\tinput_comp_size\n"
            )
    except Exception:
        ratio_guard_sample_fh = None

    constraint_log_written = 0
    constraint_log_fh = None
    try:
        if guards.constraint_log_n > 0:
            constraint_log_fh = open(guards.constraint_log_path, "w", encoding="utf-8")
            constraint_log_fh.write(
                "txid\tevent\treason\tsa\tsb\tmerged\tmin_side\tbig_side\tratio\t"
                "votes\tvotes_required\tkey_r0\tkey_r1\ttriggered_by\tbig_is_change\t"
                "pair_seen\tis_repeat\tchange_root\tanchor_root\tchange_distinct_large\n"
            )
    except Exception:
        constraint_log_fh = None

    def _votes_required_for_sizes(min_side: int, big_side: int, merged: int) -> int:
        req = guards.change_votes_required
        for thr, req_thr in guards.ultra_change_vote_rules:
            if big_side >= thr or merged >= thr:
                if req_thr > req:
                    req = req_thr
        return req

    def _log_constraint_event(
        *,
        txid: str | None,
        event: str,
        reason: str,
        sa: int,
        sb: int,
        merged: int,
        min_side: int,
        big_side: int,
        ratio: float,
        votes: int,
        votes_required: int,
        key_r0: int,
        key_r1: int,
        triggered_by: str,
        big_is_change: int,
        pair_seen: int,
        is_repeat: int,
        change_root: int,
        anchor_root: int,
        change_distinct_large: int,
    ) -> None:
        nonlocal constraint_log_written
        if (
            constraint_log_fh is None
            or constraint_log_written >= guards.constraint_log_n
            or txid is None
        ):
            return
        constraint_log_fh.write(
            f"{txid}\t{event}\t{reason}\t{sa}\t{sb}\t{merged}\t{min_side}\t{big_side}\t{ratio}\t"
            f"{votes}\t{votes_required}\t{key_r0}\t{key_r1}\t{triggered_by}\t{big_is_change}\t"
            f"{pair_seen}\t{is_repeat}\t{change_root}\t{anchor_root}\t{change_distinct_large}\n"
        )
        constraint_log_written += 1

    def union_guarded(a: int, b: int, reason: Literal["H1", "CHANGE"], txid: str | None) -> bool:
        nonlocal ratio_guard_samples_written
        nonlocal constraint_gate_evals_total, constraint_pairs_repeated

        stats["union_attempts"] += 1
        ra = uf.find(a)
        rb = uf.find(b)
        if ra == rb:
            stats["skip_same_component"] += 1
            if reason == "H1":
                stats["skip_same_component_h1"] += 1
            else:
                stats["skip_same_component_change"] += 1
            return False

        sa = uf.size[ra]
        sb = uf.size[rb]
        merged = sa + sb

        if not toggles.enable_merge_guards:
            uf.union_roots(ra, rb, change_degree_set_cap=guards.change_degree_set_cap)
            stats["unions_applied"] += 1
            if reason == "H1":
                stats["unions_applied_h1"] += 1
            else:
                stats["unions_applied_change"] += 1
            return True

        if merged > guards.max_merged_component_size:
            stats["unions_skipped"] += 1
            if reason == "H1":
                stats["unions_skipped_h1"] += 1
                stats["skip_abs_cap_h1"] += 1
            else:
                stats["unions_skipped_change"] += 1
                stats["skip_abs_cap_change"] += 1
            stats["skip_abs_cap"] += 1
            return False

        if guards.merge_ratio_guard and reason == "CHANGE":
            s_big = sa if sa >= sb else sb
            s_small = sb if sa >= sb else sa

            if s_small >= guards.merge_ratio_small_floor and s_big >= guards.merge_ratio_big_cluster_min:
                ratio = (s_big / s_small) if s_small > 0 else float("inf")
                if ratio > guards.merge_ratio_max:
                    stats["unions_skipped"] += 1
                    stats["unions_skipped_change"] += 1
                    stats["skip_ratio_guard"] += 1
                    stats["skip_ratio_guard_change"] += 1

                    if (
                        ratio_guard_sample_fh is not None
                        and ratio_guard_samples_written < guards.ratio_guard_sample_n
                        and txid is not None
                    ):
                        big_root = ra if sa >= sb else rb
                        big_is_change = 1 if big_root == rb else 0
                        big_side = "b" if big_is_change else "a"
                        ratio_guard_sample_fh.write(
                            f"{txid}\t{sa}\t{sb}\t{s_big}\t{s_small}\t{ratio}\t"
                            f"b\t{big_side}\t{big_is_change}\t{sb}\t{sa}\n"
                        )
                        ratio_guard_samples_written += 1
                    return False

        if guards.enable_change_degree_guard and reason == "CHANGE":
            anchor_root = ra
            change_root = rb
            anchor_size = sa
            change_size = sb

            change_distinct_large = 0
            if (anchor_size >= guards.change_degree_large_min) and (change_size >= guards.change_degree_small_min):
                s = uf.change_large_anchor_roots.get(change_root)
                if s is None:
                    s = set()
                    uf.change_large_anchor_roots[change_root] = s
                s.add(anchor_root)
                if len(s) > guards.change_degree_set_cap:
                    uf.change_large_anchor_roots[change_root] = set(sorted(s)[:guards.change_degree_set_cap])
                    s = uf.change_large_anchor_roots[change_root]
                change_distinct_large = len(s)

                if change_distinct_large > guards.change_degree_max_distinct_large:
                    stats["unions_skipped"] += 1
                    stats["unions_skipped_change"] += 1
                    stats["skip_degree_guard"] += 1
                    stats["skip_degree_guard_change"] += 1

                    s_big = max(sa, sb)
                    s_small = min(sa, sb)
                    ratio = (s_big / s_small) if s_small > 0 else float("inf")
                    big_root = ra if sa >= sb else rb
                    big_is_change = 1 if big_root == rb else 0
                    r0, r1 = (anchor_root, change_root) if anchor_root < change_root else (change_root, anchor_root)

                    _log_constraint_event(
                        txid=txid,
                        event="DEGREE_GUARD",
                        reason=reason,
                        sa=sa, sb=sb,
                        merged=merged,
                        min_side=min(sa, sb),
                        big_side=max(sa, sb),
                        ratio=ratio,
                        votes=0,
                        votes_required=0,
                        key_r0=r0,
                        key_r1=r1,
                        triggered_by="degree_guard",
                        big_is_change=big_is_change,
                        pair_seen=0,
                        is_repeat=0,
                        change_root=change_root,
                        anchor_root=anchor_root,
                        change_distinct_large=change_distinct_large,
                    )
                    return False

        if guards.enable_change_merge_votes and reason == "CHANGE":
            min_side = sa if sa <= sb else sb
            big_side = sa if sa >= sb else sb

            triggered = False
            triggered_by = "none"

            if guards.change_vote_min_side > 0 and min_side >= guards.change_vote_min_side:
                triggered = True
                triggered_by = "min_side"
            if (not triggered) and guards.change_vote_merged_trigger > 0 and merged >= guards.change_vote_merged_trigger:
                triggered = True
                triggered_by = "merged"

            if triggered:
                r0, r1 = (ra, rb) if ra < rb else (rb, ra)
                k = (r0, r1)

                constraint_gate_evals_total += 1
                prev_seen = constraint_pair_counts.get(k, 0)
                pair_seen = prev_seen + 1
                constraint_pair_counts[k] = pair_seen
                if pair_seen == 1:
                    constraint_unique_pairs.add(k)
                elif pair_seen == 2:
                    constraint_pairs_repeated += 1
                is_repeat = 1 if pair_seen > 1 else 0

                prev_votes = change_merge_votes.get(k, 0)
                votes = prev_votes + 1
                change_merge_votes[k] = votes

                votes_required = _votes_required_for_sizes(min_side=min_side, big_side=big_side, merged=merged)

                if votes < votes_required:
                    stats["unions_skipped"] += 1
                    stats["unions_skipped_change"] += 1
                    stats["skip_constraint"] += 1
                    stats["skip_constraint_change"] += 1
                    stats["skip_vote_pending"] += 1

                    s_big = max(sa, sb)
                    s_small = min(sa, sb)
                    ratio = (s_big / s_small) if s_small > 0 else float("inf")
                    big_root = ra if sa >= sb else rb
                    big_is_change = 1 if big_root == rb else 0

                    _log_constraint_event(
                        txid=txid,
                        event="VOTE_PENDING",
                        reason=reason,
                        sa=sa, sb=sb,
                        merged=merged,
                        min_side=min_side,
                        big_side=big_side,
                        ratio=ratio,
                        votes=votes,
                        votes_required=votes_required,
                        key_r0=r0,
                        key_r1=r1,
                        triggered_by=triggered_by,
                        big_is_change=big_is_change,
                        pair_seen=pair_seen,
                        is_repeat=is_repeat,
                        change_root=rb,
                        anchor_root=ra,
                        change_distinct_large=0,
                    )
                    return False

        if guards.debug_large_change_merges and reason == "CHANGE" and merged >= guards.debug_change_merge_min and txid is not None:
            try:
                with open(guards.debug_change_merge_log_path, "a", encoding="utf-8") as f:
                    f.write(f"{txid}\t{sa}\t{sb}\t{merged}\n")
            except Exception:
                pass

        uf.union_roots(ra, rb, change_degree_set_cap=guards.change_degree_set_cap)
        stats["unions_applied"] += 1
        if reason == "H1":
            stats["unions_applied_h1"] += 1
        else:
            stats["unions_applied_change"] += 1
        return True

    pbar = tqdm(
        eligible,
        total=len(eligible),
        unit="file",
        dynamic_ncols=True,
        mininterval=perf.tqdm_mininterval,
        miniters=perf.tqdm_miniters,
        smoothing=0,
        desc="Processing parquet",
        file=sys.stdout,
        leave=True,
    )

    DIR_NEEDS_NORMALIZATION: bool | None = None

    def log(msg: str) -> None:
        tqdm.write(msg, file=sys.stdout)

    def mark_outputs_seen(outputs_list: list[tuple[str | None, int]]) -> None:
        if toggles.precreate_nodes_for_all_output_addrs:
            for a, _v in outputs_list:
                if a is None:
                    continue
                a_id = get_addr_id(a)
                seen_output_flags[a_id] = 1
        else:
            assert seen_output_addrs is not None
            for a, _v in outputs_list:
                if a is not None:
                    seen_output_addrs.add(a)

    for i, (path, file_day) in enumerate(pbar, start=1):
        in_analysis = (analysis.analysis_start <= file_day < analysis.analysis_end_exclusive)

        if (i % perf.tqdm_postfix_every) == 0:
            pbar.set_postfix_str(f"{'analysis' if in_analysis else 'preload'} day={file_day}")

        if in_analysis:
            stats["n_files_analyzed"] += 1
            if stats["n_files_analyzed"] <= 5:
                log(f"[{stats['n_files_analyzed']}] Processing {path} (day={file_day}) ...")
            elif (stats["n_files_analyzed"] % 250) == 0:
                log(f"[{stats['n_files_analyzed']}] Processing ... (day={file_day})")
        else:
            stats["n_files_indexed"] += 1
            if stats["n_files_indexed"] <= 3:
                log(f"[preload {stats['n_files_indexed']}] Indexing outputs only: {path} (day={file_day}) ...")

        df = pl.read_parquet(path, columns=["dir", "txid", "n", "prev_txid", "prev_vout", "address", "value"])

        if DIR_NEEDS_NORMALIZATION is None:
            dir_uniques = df.select(pl.col("dir").cast(pl.Utf8, strict=False).unique()).to_series().to_list()
            DIR_NEEDS_NORMALIZATION = any(
                (d is not None and str(d).lower() in ("in", "out") and str(d) not in ("in", "out")) for d in dir_uniques
            )
            if in_analysis and stats["n_files_analyzed"] == 1:
                log(f"  [debug] distinct dir values in first processed file: {dir_uniques}")

        if DIR_NEEDS_NORMALIZATION:
            df = df.with_columns(pl.col("dir").cast(pl.Utf8).str.to_lowercase().alias("dir"))

        if value_unit is None:
            sample = (
                df.filter(pl.col("dir") == "out")
                .select(pl.col("value").cast(pl.Float64, strict=False))
                .drop_nulls()
                .head(5000)
                .to_series()
                .to_list()
            )
            value_unit = infer_value_unit_from_sample(sample)
            log(f"[INFO] Detected value unit: {value_unit}  (all internal amount logic uses satoshis)")

        out_all = (
            df.filter(pl.col("dir") == "out")
            .filter(pl.col("txid").is_not_null())
            .filter(pl.col("n").is_not_null())
            .select(
                pl.col("txid"),
                pl.col("n").cast(pl.Int64, strict=False).alias("n"),
                pl.col("address"),
                value_expr_to_sats().alias("value_sats"),
            )
        )

        out_for_db = out_all.filter(pl.col("address").is_not_null()).filter(pl.col("value_sats").is_not_null())
        if out_for_db.height > 0:
            rows = [
                (txid, int(n), addr, int(vs))
                for txid, n, addr, vs in out_for_db.select(["txid", "n", "address", "value_sats"]).iter_rows(
                    named=False, buffer_size=perf.iter_buffer_out
                )
            ]
            insert_outpoints_no_commit(conn, rows)
            pending_outpoint_rows += len(rows)

            if pending_outpoint_rows >= perf.outpoint_commit_every_rows:
                conn.execute("COMMIT;")
                conn.execute("BEGIN;")
                pending_outpoint_rows = 0

            if toggles.precreate_nodes_for_all_output_addrs:
                out_u = out_for_db.select(pl.col("address").unique()).to_series().to_list()
                for addr in sorted(out_u):
                    _ = get_addr_id(addr)

        if not in_analysis:
            del df, out_all, out_for_db
            if (i % perf.gc_every_n_files) == 0:
                gc.collect()
            continue

        in_df = (
            df.filter(pl.col("dir") == "in")
            .filter(pl.col("txid").is_not_null())
            .filter(pl.col("prev_txid").is_not_null())
            .filter(pl.col("prev_vout").is_not_null())
            .select(
                pl.col("txid").alias("spend_txid"),
                pl.col("prev_txid"),
                pl.col("prev_vout").cast(pl.Int64, strict=False).alias("prev_n"),
            )
            .filter(pl.col("prev_n").is_not_null())
        )

        del df
        if (i % perf.gc_every_n_files) == 0:
            gc.collect()

        if in_df.height == 0:
            del in_df
            del out_all, out_for_db
            if (i % perf.gc_every_n_files) == 0:
                gc.collect()
            continue

        in_counts = (
            in_df.group_by("spend_txid")
            .len()
            .rename({"spend_txid": "txid", "len": "n_in_utxos"})
        )

        if not perf.use_sql_prevout_join:
            raise RuntimeError("perf.use_sql_prevout_join=False is not supported in this build; set it True.")

        reset_vinbuf(conn)

        cur = conn.cursor()
        insert_sql = "INSERT INTO vinbuf(spend_txid, prev_txid, prev_n) VALUES (?, ?, ?);"
        batch: list[tuple[str, str, int]] = []
        total_vin = 0

        for spend_txid, prev_txid, prev_n in in_df.select(["spend_txid", "prev_txid", "prev_n"]).iter_rows(
            named=False, buffer_size=perf.iter_buffer_in
        ):
            batch.append((str(spend_txid), str(prev_txid), int(prev_n)))
            if len(batch) >= perf.vinbuf_insert_chunk:
                cur.executemany(insert_sql, batch)
                total_vin += len(batch)
                batch.clear()

        if batch:
            cur.executemany(insert_sql, batch)
            total_vin += len(batch)
            batch.clear()

        stats["n_prevout_lookups"] += total_vin

        resolved_aggs = fetch_prevouts_aggs_polars(conn, fetch_chunk=perf.vinbuf_fetch_chunk)

        if resolved_aggs.height > 0:
            resolved_inputs_sum = resolved_aggs.select(pl.col("resolved_cnt").sum()).item()
            stats["n_prevout_hits"] += int(resolved_inputs_sum) if resolved_inputs_sum is not None else 0

        del in_df

        if toggles.create_nodes_for_all_resolved_inputs and resolved_aggs.height > 0:
            u = (
                resolved_aggs
                .select(pl.col("in_addrs").explode().drop_nulls().unique())
                .to_series()
                .to_list()
            )
            for a in sorted(u):
                _ = get_addr_id(a)

        if out_all.height == 0:
            del out_all, out_for_db, in_counts, resolved_aggs
            if (i % perf.gc_every_n_files) == 0:
                gc.collect()
            continue

        vout_grouped = (
            out_all.group_by("txid")
            .agg(
                pl.col("address").alias("out_addrs_all"),
                pl.col("value_sats").alias("out_values_sats_all"),
            )
        )
        del out_all, out_for_db

        tx_joined = (
            vout_grouped
            .join(in_counts, on="txid", how="inner")
            .join(resolved_aggs, on="txid", how="left")
        )
        del vout_grouped, in_counts, resolved_aggs

        for (
            txid,
            out_addrs_all,
            out_values_all,
            n_in_utxos,
            resolved_cnt,
            sum_inputs_sats,
            min_input_sats,
            in_addrs,
        ) in tx_joined.select(
            ["txid", "out_addrs_all", "out_values_sats_all", "n_in_utxos", "resolved_cnt", "sum_inputs_sats", "min_input_sats", "in_addrs"]
        ).iter_rows(named=False, buffer_size=perf.iter_buffer_grouped):

            if not out_addrs_all or not out_values_all:
                continue

            n_in_utxos_i = int(n_in_utxos) if n_in_utxos is not None else 0
            if n_in_utxos_i == 0:
                continue

            stats["n_txs_total"] += 1

            outputs_list: list[tuple[str | None, int]] = []
            sum_outputs_sats = 0
            for a, v in zip(out_addrs_all, out_values_all):
                if v is None:
                    continue
                vi = int(v)
                outputs_list.append((a, vi))
                sum_outputs_sats += vi

            spendable = [(a, v) for (a, v) in outputs_list if a is not None and v >= params.dust_sats]
            if not spendable:
                mark_outputs_seen(outputs_list)
                continue

            is_mixing_like = False
            if toggles.enable_coinjoin_filter:
                is_mixing_like = detect_mixing_like(n_in_utxos_i, spendable, dust_sats=params.dust_sats)
                if is_mixing_like:
                    stats["n_txs_coinjoin_flagged"] += 1
                    mark_outputs_seen(outputs_list)
                    continue

            if in_addrs is None:
                mark_outputs_seen(outputs_list)
                continue

            in_addrs_list = list(in_addrs)
            if not in_addrs_list:
                mark_outputs_seen(outputs_list)
                continue

            # ---- Multi-input (H1) ----
            if toggles.enable_multi_input:
                safe = multi_input_is_safe(params.multi_input_policy, n_in_utxos_i, len(spendable), is_mixing_like)
                if safe:
                    stats["n_txs_with_multiinput"] += 1
                    if len(in_addrs_list) >= 2:
                        in_ids = [get_addr_id(a) for a in in_addrs_list]
                        for idx in in_ids:
                            multi_change_flags[idx] |= 1
                        first_id = in_ids[0]
                        for idx in in_ids[1:]:
                            union_guarded(first_id, idx, reason="H1", txid=str(txid))
                    else:
                        idx = get_addr_id(in_addrs_list[0])
                        multi_change_flags[idx] |= 1

            # ---- Change (logit + Patch 2 acceptance) ----
            if toggles.enable_change:
                n_spendable_out = len(spendable)

                if params.change_require_2_outputs and n_spendable_out != 2:
                    mark_outputs_seen(outputs_list)
                    continue

                if n_in_utxos_i > params.max_change_inputs_utxos:
                    mark_outputs_seen(outputs_list)
                    continue
                if n_spendable_out > params.max_change_spendable_outs:
                    mark_outputs_seen(outputs_list)
                    continue

                resolved_cnt_i = int(resolved_cnt) if resolved_cnt is not None else 0
                resolved_all = (resolved_cnt_i == n_in_utxos_i)
                if not resolved_all:
                    mark_outputs_seen(outputs_list)
                    continue

                in_types = [addr_type(a) for a in in_addrs_list]
                in_type_uniform = (len(set(in_types)) == 1)
                majority_type = in_types[0] if in_types else "other"
                if not in_type_uniform:
                    mark_outputs_seen(outputs_list)
                    continue

                spendable_addrs = [a for (a, _v) in spendable]
                if len(spendable_addrs) != len(set(spendable_addrs)):
                    mark_outputs_seen(outputs_list)
                    continue

                in_addr_set_fast = set(in_addrs_list)

                candidates: list[tuple[str, int, bool]] = []
                for a, v in spendable:
                    if a in in_addr_set_fast:
                        continue
                    if addr_type(a) != majority_type:
                        continue

                    is_new = True
                    if toggles.precreate_nodes_for_all_output_addrs:
                        a_id = get_addr_id(a)
                        if seen_output_flags[a_id]:
                            is_new = False
                    else:
                        assert seen_output_addrs is not None
                        if a in seen_output_addrs:
                            is_new = False

                    candidates.append((a, v, is_new))

                if len(candidates) != 2:
                    mark_outputs_seen(outputs_list)
                    continue

                (cand_a, cand_v, cand_new), (cand_b, cand_w, cand_new2) = candidates

                sum_in_i = int(sum_inputs_sats) if sum_inputs_sats is not None else 0
                fee = sum_in_i - sum_outputs_sats
                if sum_in_i <= 0 or fee < 0:
                    mark_outputs_seen(outputs_list)
                    continue
                if fee > int(params.max_fee_abs_sats) or fee > int(params.max_fee_frac * sum_in_i):
                    mark_outputs_seen(outputs_list)
                    continue

                other_a, other_v, other_new = cand_b, cand_w, cand_new2
                min_in = int(min_input_sats) if min_input_sats is not None else 0

                feats1 = extract_change_features(
                    n_in_utxos=n_in_utxos_i,
                    n_spendable_out=n_spendable_out,
                    fee_sats=fee,
                    sum_inputs_sats=sum_in_i,
                    sum_outputs_sats=sum_outputs_sats,
                    min_input_sats=min_in,
                    in_type_uniform=in_type_uniform,
                    out_type_match_in=True,
                    candidate_new=cand_new,
                    candidate_value_sats=cand_v,
                    other_value_sats=other_v,
                    candidate_is_min_out=(cand_v <= other_v),
                    candidate_lt_other=(cand_v < other_v),
                    is_mixing_like=is_mixing_like,
                    cfg=change_model,
                )
                p1 = score_change_candidate(feats1, change_model)

                feats2 = extract_change_features(
                    n_in_utxos=n_in_utxos_i,
                    n_spendable_out=n_spendable_out,
                    fee_sats=fee,
                    sum_inputs_sats=sum_in_i,
                    sum_outputs_sats=sum_outputs_sats,
                    min_input_sats=min_in,
                    in_type_uniform=in_type_uniform,
                    out_type_match_in=True,
                    candidate_new=other_new,
                    candidate_value_sats=other_v,
                    other_value_sats=cand_v,
                    candidate_is_min_out=(other_v <= cand_v),
                    candidate_lt_other=(other_v < cand_v),
                    is_mixing_like=is_mixing_like,
                    cfg=change_model,
                )
                p2 = score_change_candidate(feats2, change_model)

                # best vs runner-up
                if p2 > p1:
                    p_best, change_addr, change_val = p2, other_a, other_v
                    p_second = p1
                    chosen_is_min = (other_v <= cand_v)
                else:
                    p_best, change_addr, change_val = p1, cand_a, cand_v
                    p_second = p2
                    chosen_is_min = (cand_v <= other_v)

                gap = float(p_best - p_second)

                # proxy eval label: "chosen change is the min output"
                eval_col.add(float(p_best), 1 if chosen_is_min else 0)

                # Patch 2: acceptance thresholds from calibrators (no year-specific caps)
                thr_accept = float(p_accept_cal.value)
                thr_gap = float(p_gap_cal.value) if p_gap_cal is not None else float(change_model.min_p_gap_fixed)

                accept = (float(p_best) >= thr_accept) and (gap >= thr_gap)

                # define "easy" stream: 2-out, non-mixing, resolved-all, model invoked
                easy_seen += 1
                if accept:
                    easy_accepted += 1

                # update calibrators AFTER using current threshold (no lookahead)
                p_accept_cal.update(float(p_best))
                if p_gap_cal is not None:
                    p_gap_cal.update(gap)

                if not accept:
                    mark_outputs_seen(outputs_list)
                    continue

                stats["n_txs_with_change_detected"] += 1

                anchor_id = get_addr_id(in_addrs_list[0])
                change_id = get_addr_id(change_addr)

                multi_change_flags[change_id] |= 2
                multi_change_flags[anchor_id] |= 4

                union_guarded(anchor_id, change_id, reason="CHANGE", txid=str(txid))

            mark_outputs_seen(outputs_list)

        del tx_joined
        if (i % perf.gc_every_n_files) == 0:
            gc.collect()

    pbar.close()
    conn.execute("COMMIT;")

    try:
        if ratio_guard_sample_fh is not None:
            ratio_guard_sample_fh.close()
            log(f"[DIAG] Wrote ratio-guard sample log: {guards.ratio_guard_sample_path}  (n={ratio_guard_samples_written})")
    except Exception:
        pass
    try:
        if constraint_log_fh is not None:
            constraint_log_fh.close()
            log(f"[DIAG] Wrote constraint-event log: {guards.constraint_log_path}  (n={constraint_log_written})")
    except Exception:
        pass

    # Finalize entity mapping
    n_nodes = len(addr_to_id)
    if n_nodes == 0:
        print("No union-find nodes created.", flush=True)
        conn.close()
        return

    node_to_entity = np.empty(n_nodes, dtype=np.int32)
    root_to_entity: dict[int, int] = {}
    next_entity_id = 0
    for node in range(n_nodes):
        root = uf.find(node)
        ent = root_to_entity.get(root)
        if ent is None:
            ent = next_entity_id
            root_to_entity[root] = ent
            next_entity_id += 1
        node_to_entity[node] = ent

    n_entities = next_entity_id
    counts = np.bincount(node_to_entity, minlength=n_entities).astype(np.int64)
    cluster_sizes = counts[counts > 0]

    scores_arr, labels_arr = eval_col.arrays()
    model_diag = {
        "easy_seen": int(easy_seen),
        "easy_accepted": int(easy_accepted),
        "p_accept": float(p_accept_cal.value),
        "p_gap": float(p_gap_cal.value) if p_gap_cal is not None else float(change_model.min_p_gap_fixed),
        "eval_samples": int(0 if scores_arr is None else scores_arr.size),
    }

    top_entity_id = print_final_report(
        n_nodes=n_nodes,
        n_entities=n_entities,
        node_to_entity=node_to_entity,
        counts=counts,
        cluster_sizes=cluster_sizes,
        stats=stats,
        multi_change_flags=multi_change_flags,
        outputs=outputs,
        guards=guards,
        enable_constraints_diag=(guards.enable_change_merge_votes or guards.enable_change_degree_guard),
        constraint_unique_pairs=constraint_unique_pairs,
        constraint_gate_evals_total=constraint_gate_evals_total,
        constraint_pairs_repeated=constraint_pairs_repeated,
        change_merge_votes=change_merge_votes,
        prevout_lookups=stats["n_prevout_lookups"],
        prevout_hits=stats["n_prevout_hits"],
        model_diag=model_diag,
    )

    # Sanity
    if sanity.run_sanity_checks:
        run_sanity_checks(
            n_nodes=n_nodes,
            node_to_entity=node_to_entity,
            prevout_lookups=stats["n_prevout_lookups"],
            prevout_hits=stats["n_prevout_hits"],
            top_k=20,
        )
        if sanity.run_prevout_db_sanity and sanity_parquet:
            prevout_join_sanity_db(
                conn,
                sanity_parquet,
                sample_limit=sanity.prevout_sanity_sample_limit,
                hybrid_threshold=perf.prevout_hybrid_threshold,
                join_chunk=perf.prevout_lookup_chunk,
            )

    # Write mapping (address -> entity_id)
    if outputs.write_entity_mapping:
        print(f"\n[WRITE] Writing entity mapping to: {paths.entity_map_out_path}", flush=True)
        if paths.entity_map_out_path.exists():
            paths.entity_map_out_path.unlink()

        writer: pq.ParquetWriter | None = None
        batch_addrs: list[str] = []
        batch_eids: list[int] = []
        written = 0

        for addr, node_id in addr_to_id.items():
            batch_addrs.append(addr)
            batch_eids.append(int(node_to_entity[node_id]))

            if len(batch_addrs) >= outputs.entity_write_batch:
                table = pa.table({"address": batch_addrs, "entity_id": batch_eids})
                if writer is None:
                    writer = pq.ParquetWriter(
                        str(paths.entity_map_out_path),
                        table.schema,
                        compression=outputs.entity_map_compression,
                        use_dictionary=True,
                    )
                writer.write_table(table)
                written += len(batch_addrs)
                batch_addrs.clear()
                batch_eids.clear()
                print(f"  [WRITE] rows written: {written:,}", flush=True)

        if batch_addrs:
            table = pa.table({"address": batch_addrs, "entity_id": batch_eids})
            if writer is None:
                writer = pq.ParquetWriter(
                    str(paths.entity_map_out_path),
                    table.schema,
                    compression=outputs.entity_map_compression,
                    use_dictionary=True,
                )
            writer.write_table(table)
            written += len(batch_addrs)
            print(f"  [WRITE] rows written: {written:,}", flush=True)

        if writer is not None:
            writer.close()

        print("[WRITE] Done.", flush=True)

    # Confidence proxy output (optional)
    if confidence.enable_confidence_proxy:
        print(f"\n[CONF] Writing confidence proxy to: {confidence.confidence_out_path}", flush=True)
        if confidence.confidence_out_path.exists():
            confidence.confidence_out_path.unlink()

        def proxy_prob(node_id: int) -> float:
            eid = int(node_to_entity[node_id])
            sz = int(counts[eid])
            if sz <= 1:
                return float(confidence.p_singleton)

            bits = int(multi_change_flags[node_id])
            e = 0
            if bits & 1:
                e += 1
            if bits & 2:
                e += 1
            if bits & 4:
                e += 1

            if e <= 0:
                base = float(confidence.p_base_0)
            elif e == 1:
                base = float(confidence.p_base_1)
            else:
                base = float(confidence.p_base_2plus)

            denom = math.log1p(max(1, confidence.size_norm))
            frac = (math.log1p(sz) / denom) if denom > 0 else 1.0
            frac = min(1.0, max(0.0, frac))
            p = base + float(confidence.size_bonus_max) * frac
            return float(min(0.999, max(0.001, p)))

        writer2: pq.ParquetWriter | None = None
        b_addr: list[str] = []
        b_eid: list[int] = []
        b_p: list[float] = []
        b_sz: list[int] = []
        b_bits: list[int] = []
        written2 = 0

        for addr, node_id in addr_to_id.items():
            eid = int(node_to_entity[node_id])
            b_addr.append(addr)
            b_eid.append(eid)
            b_p.append(proxy_prob(node_id))
            if confidence.include_cluster_size:
                b_sz.append(int(counts[eid]))
            if confidence.include_evidence_bits:
                b_bits.append(int(multi_change_flags[node_id]))

            if len(b_addr) >= confidence.confidence_write_batch:
                data = {"address": b_addr, "entity_id": b_eid, "p_clustered_proxy": b_p}
                if confidence.include_cluster_size:
                    data["cluster_size"] = b_sz
                if confidence.include_evidence_bits:
                    data["evidence_bits"] = b_bits
                table = pa.table(data)
                if writer2 is None:
                    writer2 = pq.ParquetWriter(
                        str(confidence.confidence_out_path),
                        table.schema,
                        compression=confidence.confidence_compression,
                        use_dictionary=True,
                    )
                writer2.write_table(table)
                written2 += len(b_addr)
                b_addr.clear()
                b_eid.clear()
                b_p.clear()
                b_sz.clear()
                b_bits.clear()
                print(f"  [CONF] rows written: {written2:,}", flush=True)

        if b_addr:
            data = {"address": b_addr, "entity_id": b_eid, "p_clustered_proxy": b_p}
            if confidence.include_cluster_size:
                data["cluster_size"] = b_sz
            if confidence.include_evidence_bits:
                data["evidence_bits"] = b_bits
            table = pa.table(data)
            if writer2 is None:
                writer2 = pq.ParquetWriter(
                    str(confidence.confidence_out_path),
                    table.schema,
                    compression=confidence.confidence_compression,
                    use_dictionary=True,
                )
            writer2.write_table(table)
            written2 += len(b_addr)
            print(f"  [CONF] rows written: {written2:,}", flush=True)

        if writer2 is not None:
            writer2.close()

        print("[CONF] Done.", flush=True)

    # Plots:
    # - keep: cluster hist (all) + zipf
    # - replace: (excl largest) and (<=focus_max_size) with logit eval (ROC + calibration)
    if plots.enable_plots:
        print("\n[PLOT] Generating plots...", flush=True)
        y = analysis.analysis_start.year

        save_hist = (paths.outputs_dir / f"cluster_hist_{y}.png") if plots.save_plots else None
        save_zipf = (paths.outputs_dir / f"cluster_zipf_{y}.png") if plots.save_plots else None
        save_roc  = (paths.outputs_dir / f"logit_roc_{y}.png") if plots.save_plots else None
        save_cal  = (paths.outputs_dir / f"logit_calibration_{y}.png") if plots.save_plots else None

        plot_hist_logbins(
            sizes=cluster_sizes,
            title=f"Entity Cluster Size Distribution ({y}) — log bins (all clusters)",
            color="tab:blue",
            bins=plots.log_bins,
            max_x=None,
            save_path=save_hist,
            show=plots.show_plots,
        )

        plot_zipf(
            sizes=cluster_sizes,
            title=f"Rank–Size (Zipf) Plot ({y}) — top {min(plots.zipf_top_k, cluster_sizes.size):,} clusters",
            color="tab:red",
            top_k=plots.zipf_top_k,
            save_path=save_zipf,
            show=plots.show_plots,
        )

        if scores_arr is not None and labels_arr is not None:
            plot_logit_eval(
                scores=scores_arr.astype(np.float32, copy=False),
                labels=labels_arr.astype(np.int8, copy=False),
                roc_points=plots.logit_roc_points,
                calib_bins=plots.logit_calibration_bins,
                title_prefix=f"Logit ({y})",
                save_roc=save_roc,
                save_calib=save_cal,
                show=plots.show_plots,
            )
        else:
            print("[PLOT] No logit eval samples collected; skipping ROC/calibration.", flush=True)

    conn.close()
    print("[INFO] Done.", flush=True)


# =============================================================================
# MAIN: run two years (2013 + 2 => 2013, 2014)
# =============================================================================

if __name__ == "__main__":
    outputs_dir = Path("/media/vatereal/Main/outputs")

    start_year = 2013
    n_years = 2
    base_seed = 1337

    for analysis_year in range(start_year, start_year + n_years):
        paths = PathsConfig(
            parquet_io_glob=str(Path("/media/vatereal/Main/parquet") / "io/day=*/io-*.parquet"),
            outputs_dir=outputs_dir,
            outpoint_db_path=outputs_dir / f"outpoints_{analysis_year}.sqlite",
            entity_map_out_path=outputs_dir / f"entities_model_features_{analysis_year}.parquet",
        )

        analysis = AnalysisConfig(
            analysis_start=date(analysis_year, 1, 1),
            analysis_end_exclusive=date(analysis_year + 1, 1, 1),
            outpoint_db_lookback_days=365,
            value_unit_mode="infer",
        )

        outputs = OutputConfig(
            reset_outpoint_db=True,
            write_entity_mapping=True,
            entity_map_compression="zstd",
            entity_write_batch=1_000_000,
            top_k_clusters_print=20,
        )

        plots = PlotConfig(
            enable_plots=True,
            zipf_top_k=200_000,
            focus_max_size=2048,  # kept for compatibility; no longer used for plotting
            log_bins=40,

            # logit eval plots
            logit_eval_max_samples=500_000,
            logit_calibration_bins=20,
            logit_roc_points=250,

            # save/display
            save_plots=True,
            show_plots=False,
        )

        perf = PerfConfig(
            outpoint_commit_every_rows=500_000,
            prevout_hybrid_threshold=5_000,
            prevout_lookup_chunk=50_000,
            iter_buffer_in=200_000,
            iter_buffer_out=200_000,
            iter_buffer_grouped=50_000,
            gc_every_n_files=100,
            tqdm_mininterval=2.0,
            tqdm_miniters=50,
            tqdm_postfix_every=250,
            use_sql_prevout_join=True,
            vinbuf_insert_chunk=1_000_000,
            vinbuf_fetch_chunk=1_000_000,
        )

        sanity = SanityConfig(
            run_sanity_checks=True,
            run_prevout_db_sanity=True,
            prevout_sanity_sample_limit=200_000,
            prevout_sanity_parquet=None,
        )

        toggles = HeuristicToggles(
            enable_coinjoin_filter=True,
            enable_multi_input=True,
            enable_change=True,
            enable_merge_guards=True,
            precreate_nodes_for_all_output_addrs=False,
            create_nodes_for_all_resolved_inputs=True,
        )

        params = HeuristicParams(
            dust_sats=546,
            max_fee_abs_sats=50_000_000,
            max_fee_frac=0.05,
            max_change_inputs_utxos=10,
            max_change_spendable_outs=2,
            change_require_2_outputs=True,
            multi_input_policy="one_output",
        )

        guards = MergeGuardParams(
            max_merged_component_size=10_000_000,
            merge_ratio_guard=True,
            merge_ratio_max=200.0,
            merge_ratio_big_cluster_min=50_000,
            merge_ratio_small_floor=50,

            enable_change_merge_votes=True,
            change_vote_min_side=25_000,
            change_vote_merged_trigger=0,
            change_votes_required=2,
            ultra_change_vote_rules=((250_000, 3), (500_000, 4), (1_000_000, 5)),

            enable_change_degree_guard=True,
            change_degree_large_min=25_000,
            change_degree_small_min=50,
            change_degree_max_distinct_large=3,
            change_degree_set_cap=64,

            ratio_guard_sample_n=1000,
            ratio_guard_sample_path=outputs_dir / f"ratio_guard_samples_{analysis_year}.tsv",

            constraint_log_n=200_000,
            constraint_log_path=outputs_dir / f"constraint_events_{analysis_year}.tsv",

            debug_large_change_merges=False,
            debug_change_merge_min=250_000,
            debug_change_merge_log_path=outputs_dir / f"large_change_merges_{analysis_year}.tsv",
        )

        confidence = ConfidenceConfig(
            enable_confidence_proxy=True,
            confidence_out_path=outputs_dir / f"address_confidence_{analysis_year}.parquet",
            confidence_compression="zstd",
            confidence_write_batch=1_000_000,

            size_norm=100_000,
            size_bonus_max=0.15,
            p_singleton=0.02,
            p_base_0=0.25,
            p_base_1=0.65,
            p_base_2plus=0.85,

            include_cluster_size=True,
            include_evidence_bits=True,
        )

        # Logistic weights (featureized rules). Tune later if desired.
        change_model = ChangeModelConfig(
            intercept=-1.00,
            weights={
                "cand_new": 0.55,
                "cand_is_min_out": 0.55,
                "cand_lt_other": 0.20,
                "fee_frac": -2.50,
                "log_fee": -0.35,
                "in_type_uniform": 0.25,
                "out_type_match_in": 0.35,
                "optimal_ok": 0.55,
                "is_mixing_like": -2.00,
                "sum_in_gt_sum_out": 0.50,
                "n_in": 0.05,
                "n_out": -0.25,
            },

            # Patch 2 (adaptive acceptance)
            min_p_gap_fixed=0.10,

            p_gap_quantile=None,          # set e.g. 0.90 to quantile-calibrate gap
            p_gap_window=200_000,
            p_gap_update_every=2000,
            p_gap_warmup_min_samples=20_000,
            p_gap_init=0.10,
            p_gap_min=0.05,
            p_gap_max=0.60,

            p_accept_quantile=0.99,       # top 1% by p_best on easy stream
            p_accept_window=200_000,
            p_accept_update_every=2000,
            p_accept_warmup_min_samples=20_000,
            p_accept_init=0.85,
            p_accept_min=0.70,
            p_accept_max=0.995,

            optimal_change_slack_sats=0,

            log1p_fee_norm=1e6,
            log1p_value_norm=1e8,
        )

        print(f"\n============================")
        print(f" RUN YEAR {analysis_year}")
        print(f"============================", flush=True)

        run_entity_clustering(
            paths=paths,
            analysis=analysis,
            outputs=outputs,
            plots=plots,
            perf=perf,
            sanity=sanity,
            toggles=toggles,
            params=params,
            guards=guards,
            confidence=confidence,
            change_model=change_model,
            determinism_seed=base_seed,
        )



 RUN YEAR 2013
Parquet file counts: {'io': 7678}
[INFO] Analysis window: 2013-01-01 .. 2014-01-01 (exclusive)
[INFO] Outpoint DB preload start: 2012-01-02 .. 2014-01-01 (exclusive)
[INFO] Outpoint DB path: /media/vatereal/Main/outputs/outpoints_2013.sqlite (reset=True)
[INFO] Eligible files in window: 753  (preload=375, analysis=378)
[INFO] Sanity parquet selected: /media/vatereal/Main/parquet/io/day=2013-01-01/io-000214563-000214724.parquet


Processing parquet:   0%|          | 0/753 [00:00<?, ?file/s]

[preload 1] Indexing outputs only: /media/vatereal/Main/parquet/io/day=2012-01-02/io-000160190-000160340.parquet (day=2012-01-02) ...
[INFO] Detected value unit: btc  (all internal amount logic uses satoshis)
[preload 2] Indexing outputs only: /media/vatereal/Main/parquet/io/day=2012-01-03/io-000160341-000160499.parquet (day=2012-01-03) ...
[preload 3] Indexing outputs only: /media/vatereal/Main/parquet/io/day=2012-01-04/io-000160500-000160648.parquet (day=2012-01-04) ...
[1] Processing /media/vatereal/Main/parquet/io/day=2013-01-01/io-000214563-000214724.parquet (day=2013-01-01) ...
[2] Processing /media/vatereal/Main/parquet/io/day=2013-01-02/io-000214725-000214877.parquet (day=2013-01-02) ...
[3] Processing /media/vatereal/Main/parquet/io/day=2013-01-03/io-000214878-000215032.parquet (day=2013-01-03) ...
[4] Processing /media/vatereal/Main/parquet/io/day=2013-01-03/io-000215033-000215039.parquet (day=2013-01-03) ...
[5] Processing /media/vatereal/Main/parquet/io/day=2013-01-04/io-00

/tmp/ipykernel_111889/2214432097.py:980: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = float(np.trapz(tpr_s, fpr_s))


[INFO] Done.

 RUN YEAR 2014
Parquet file counts: {'io': 7678}
[INFO] Analysis window: 2014-01-01 .. 2015-01-01 (exclusive)
[INFO] Outpoint DB preload start: 2013-01-01 .. 2015-01-01 (exclusive)
[INFO] Outpoint DB path: /media/vatereal/Main/outputs/outpoints_2014.sqlite (reset=True)
[INFO] Eligible files in window: 755  (preload=378, analysis=377)
[INFO] Sanity parquet selected: /media/vatereal/Main/parquet/io/day=2014-01-01/io-000277996-000278200.parquet


Processing parquet:   0%|          | 0/755 [00:00<?, ?file/s]

[preload 1] Indexing outputs only: /media/vatereal/Main/parquet/io/day=2013-01-01/io-000214563-000214724.parquet (day=2013-01-01) ...
[INFO] Detected value unit: btc  (all internal amount logic uses satoshis)
[preload 2] Indexing outputs only: /media/vatereal/Main/parquet/io/day=2013-01-02/io-000214725-000214877.parquet (day=2013-01-02) ...
[preload 3] Indexing outputs only: /media/vatereal/Main/parquet/io/day=2013-01-03/io-000214878-000215032.parquet (day=2013-01-03) ...
[1] Processing /media/vatereal/Main/parquet/io/day=2014-01-01/io-000277996-000278200.parquet (day=2014-01-01) ...
[2] Processing /media/vatereal/Main/parquet/io/day=2014-01-02/io-000278201-000278334.parquet (day=2014-01-02) ...
[3] Processing /media/vatereal/Main/parquet/io/day=2014-01-03/io-000278335-000278495.parquet (day=2014-01-03) ...
[4] Processing /media/vatereal/Main/parquet/io/day=2014-01-04/io-000278496-000278636.parquet (day=2014-01-04) ...
[5] Processing /media/vatereal/Main/parquet/io/day=2014-01-05/io-00

# Change-output Logit Model (Featureized Heuristic)

This pipeline replaces brittle, year-specific hard rules for change detection with a **deterministic logistic model** plus an **online acceptance calibrator**.

The change step is used only to propose a *single* additional union per eligible transaction:
`anchor_input_address  ↔  selected_change_output_address`.

---

## 1) Candidate generation (eligibility)

A transaction is eligible for change inference only if:

- Not mixing-like (if coinjoin filter is enabled)
- Inputs are fully resolved via the outpoint DB (all inputs have known previous outputs)
- The transaction has exactly 2 spendable outputs (when `change_require_2_outputs=True`)
- Input address types are uniform (e.g., all P2PKH), and candidate outputs match that type
- Candidate outputs are not among the input addresses
- Candidate outputs are distinct (no duplicate output addresses)

From the 2 outputs, we build **two candidates**: score output A as “change” and score output B as “change”.

---

## 2) Feature extraction

For each candidate output, we compute a feature vector `x`:

### Structural / count features
- `n_in`: number of input UTXOs
- `n_out`: number of spendable outputs (typically 2 here)

### Fee / amount features
- `fee_frac`: `fee_sats / sum_inputs_sats`
- `log_fee`: `log1p(fee_sats) / log1p(fee_norm)`
- `log_cand`: `log1p(candidate_value_sats) / log1p(value_norm)`
- `log_other`: `log1p(other_output_value_sats) / log1p(value_norm)`

### Type / novelty features
- `in_type_uniform`: inputs share a single script/address type
- `out_type_match_in`: candidate output type matches the input type
- `cand_new`: candidate address has not appeared as an output earlier in the run

### Relative-output features
- `cand_is_min_out`: candidate value is <= the other output value
- `cand_lt_other`: candidate value is strictly < the other output value

### “Optimal change” feature (soft constraint)
- `optimal_ok`: `1` if `candidate_value_sats <= min_input_sats - fee_sats (+ slack)`, else `0`

### Sanity / consistency
- `sum_in_gt_sum_out`: `1` if `sum_inputs_sats > sum_outputs_sats`, else `0`

### Mixing indicator (should be 0 in eligible stream)
- `is_mixing_like`: included for completeness; acts as a strong negative weight

---

## 3) Logistic scoring

The model computes:

`score = intercept + Σ_i w_i * x_i`

Probability:

`p = sigmoid(score) = 1 / (1 + exp(-score))`

We score both candidates:
- `p1 = P(output A is change)`
- `p2 = P(output B is change)`

Select the best candidate:
- `p_best = max(p1, p2)`
- `p_second = min(p1, p2)`

Apply a *separation constraint*:
- accept only if `p_best - p_second >= min_p_gap`

This prevents ambiguous “coin-flip” assignments.

---

## 4) Patch 2: online year-adaptive acceptance threshold

Instead of fixed thresholds per year, we use an **online calibrator**:

- Maintain a rolling window of `p_best` values for *easy* transactions:
  - 2 spendable outputs
  - not mixing-like
  - all inputs resolved
  - candidate set is exactly 2 outputs

- Set the acceptance threshold dynamically:
  - `p_accept = quantile(p_best_window, q)` (e.g., q = 0.99)
  - clamp to `[p_accept_min, p_accept_max]`

Decision rule becomes:

Accept a change merge iff:
1) the tx is eligible
2) `p_gap = p_best - p_second >= min_p_gap`
3) `p_best >= p_accept` (rolling quantile threshold)

This yields a deterministic, testable policy that adapts naturally across years
without guessing “correct” thresholds for 2013 vs 2014.

---

## 5) Effect on clustering

When accepted:
- Mark evidence bits:
  - change output node gets bit2
  - anchor/input node gets bit4
- Apply a single union:
  - `union(anchor_input_addr, selected_change_output_addr)`

This keeps change-based linking sparse and reduces risk of runaway component growth.
